---

## 📖 本教程完整目录

### 第 1-3 部分：架构与数据准备
- 1. OpenChatBI 架构概览
- 2. 核心模块详解
- 3. 创建 SDTM 示例数据 (DM, AE, VS)

### 第 4 部分：catalog/ 模块 - 数据目录管理
- 4.1 模块介绍
- 4.2 数据准备
- 4.3 创建表结构元数据
- 4.4 使用 catalog_store.py
- 4.5 使用 schema_retrival.py (智能检索)
- 4.6 使用 helper.py (列映射)
- 4.7 完整的 Schema Linking 流程
- 4.8 使用 catalog_loader.py (数据库加载)
- 4.9 使用 factory.py (工厂模式)
- 4.10 工作流总结
- 4.11 实战练习

### 第 5 部分：code/ 模块 - Python 代码执行器
- 5.1 模块架构概览
- 5.2 executor_base.py - 抽象基类
- 5.3 local_executor.py - 本地执行器（无限制）
- 5.4 restricted_local_executor.py - 受限执行器（推荐）
- 5.5 执行器对比与选择指南
- 5.6 实战案例：构建临床数据分析工作流
- 5.7 与 Agent 的集成
- 5.8 实战练习
- 5.9 代码执行器模块总结

### 第 6 部分：llm/ 模块 - 大语言模型接口 ⭐ 新增
- 6.1 模块架构概览
- 6.2 LLM 配置详解
- 6.3 get_default_llm() - 获取默认 LLM
- 6.4 get_text2sql_llm() - 获取 Text2SQL 专用 LLM
- 6.5 get_embedding_model() - 获取 Embedding 模型
- 6.6 call_llm_chat_model_with_retry() - 智能重试机制
- 6.7 完整的临床数据分析示例
- 6.8 LLM 模块总结与最佳实践

### 第 7+ 部分：配置与运行（原内容保留）
- 配置临床数据库连接
- 测试本地数据集加载
- 常见临床监控查询示例
- 数据可视化
- 修改和扩展指南
- 启动 Streamlit UI

---

## 1. OpenChatBI 架构概览

OpenChatBI 是一个基于 LLM 的自然语言查询工具，主要目录结构：

```
openchatbi/
├── openchatbi/          # 核心代码目录
│   ├── catalog/         # 数据库元数据管理（表结构、列信息）
│   ├── code/            # Python 代码执行器
│   ├── llm/             # LLM 接口封装
│   ├── prompts/         # 提示词模板
│   ├── text2sql/        # 自然语言转 SQL
│   ├── tool/            # Agent 工具集
│   ├── agent_graph.py   # LangGraph Agent 流程
│   ├── config_loader.py # 配置加载器
│   └── utils.py         # 工具函数
├── sample_ui/           # Streamlit 用户界面
├── example/             # 配置和数据示例
└── tests/               # 单元测试
```

## 2. 创建 SDTM 示例数据

### 2.1 创建人口统计学域 (DM)

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

# 创建数据目录
data_dir = Path('data/sdtm')
data_dir.mkdir(parents=True, exist_ok=True)

# 创建 DM (Demographics) 域
np.random.seed(42)
n_subjects = 50

dm_data = pd.DataFrame({
    'STUDYID': ['STUDY001'] * n_subjects,
    'DOMAIN': ['DM'] * n_subjects,
    'USUBJID': [f'STUDY001-{i:03d}' for i in range(1, n_subjects+1)],
    'SUBJID': [f'{i:03d}' for i in range(1, n_subjects+1)],
    'RFSTDTC': pd.date_range('2024-01-01', periods=n_subjects, freq='D').strftime('%Y-%m-%d'),
    'RFENDTC': pd.date_range('2024-03-01', periods=n_subjects, freq='D').strftime('%Y-%m-%d'),
    'SITEID': np.random.choice(['Site01', 'Site02', 'Site03'], n_subjects),
    'AGE': np.random.randint(18, 75, n_subjects),
    'AGEU': ['YEARS'] * n_subjects,
    'SEX': np.random.choice(['M', 'F'], n_subjects),
    'RACE': np.random.choice(['WHITE', 'BLACK OR AFRICAN AMERICAN', 'ASIAN'], n_subjects),
    'ETHNIC': np.random.choice(['HISPANIC OR LATINO', 'NOT HISPANIC OR LATINO'], n_subjects),
    'ARMCD': np.random.choice(['TRT', 'PLB'], n_subjects),
    'ARM': np.where(np.random.choice(['TRT', 'PLB'], n_subjects) == 'TRT', 'Treatment', 'Placebo'),
    'COUNTRY': ['USA'] * n_subjects
})

dm_data.to_csv(data_dir / 'dm.csv', index=False)
print(f"✓ 创建 DM 域: {len(dm_data)} 条记录")
dm_data.head()

✓ 创建 DM 域: 50 条记录


,STUDYID,DOMAIN,USUBJID,SUBJID,RFSTDTC,RFENDTC,SITEID,AGE,AGEU,SEX,RACE,ETHNIC,ARMCD,ARM,COUNTRY
0,STUDY001,DM,STUDY001-001,001,2024-01-01,2024-03-01,Site03,26,YEARS,M,BLACK OR AFRICAN AMERICAN,HISPANIC OR LATINO,TRT,Placebo,USA
1,STUDY001,DM,STUDY001-002,002,2024-01-02,2024-03-02,Site01,43,YEARS,M,ASIAN,HISPANIC OR LATINO,TRT,Placebo,USA
2,STUDY001,DM,STUDY001-003,003,2024-01-03,2024-03-03,Site03,70,YEARS,M,WHITE,HISPANIC OR LATINO,PLB,Treatment,USA
3,STUDY001,DM,STUDY001-004,004,2024-01-04,2024-03-04,Site03,19,YEARS,M,WHITE,NOT HISPANIC OR LATINO,TRT,Placebo,USA
4,STUDY001,DM,STUDY001-005,005,2024-01-05,2024-03-05,Site01,37,YEARS,M,BLACK OR AFRICAN AMERICAN,HISPANIC OR LATINO,TRT,Treatment,USA


### 2.2 创建不良事件域 (AE)

In [3]:
# 创建 AE (Adverse Events) 域
ae_terms = ['HEADACHE', 'NAUSEA', 'FATIGUE', 'DIZZINESS', 'RASH', 
            'DIARRHEA', 'INSOMNIA', 'COUGH', 'FEVER', 'PAIN']
severities = ['MILD', 'MODERATE', 'SEVERE']
outcomes = ['RECOVERED/RESOLVED', 'RECOVERING/RESOLVING', 'NOT RECOVERED/NOT RESOLVED']

ae_records = []
ae_seq = 1

for usubjid in dm_data['USUBJID']:
    # 每个受试者随机生成 0-3 个 AE
    n_ae = np.random.choice([0, 1, 2, 3], p=[0.3, 0.4, 0.2, 0.1])
    
    for _ in range(n_ae):
        ae_records.append({
            'STUDYID': 'STUDY001',
            'DOMAIN': 'AE',
            'USUBJID': usubjid,
            'AESEQ': ae_seq,
            'AETERM': np.random.choice(ae_terms),
            'AEDECOD': np.random.choice(ae_terms),
            'AESTDTC': (pd.Timestamp('2024-01-15') + pd.Timedelta(days=np.random.randint(0, 60))).strftime('%Y-%m-%d'),
            'AEENDTC': (pd.Timestamp('2024-01-20') + pd.Timedelta(days=np.random.randint(0, 70))).strftime('%Y-%m-%d'),
            'AESEV': np.random.choice(severities, p=[0.6, 0.3, 0.1]),
            'AESER': np.random.choice(['N', 'Y'], p=[0.9, 0.1]),
            'AEREL': np.random.choice(['NOT RELATED', 'UNLIKELY RELATED', 'POSSIBLY RELATED', 'RELATED'], 
                                     p=[0.3, 0.3, 0.3, 0.1]),
            'AEOUT': np.random.choice(outcomes)
        })
        ae_seq += 1

ae_data = pd.DataFrame(ae_records)
ae_data.to_csv(data_dir / 'ae.csv', index=False)
print(f"✓ 创建 AE 域: {len(ae_data)} 条记录")
ae_data.head()

✓ 创建 AE 域: 58 条记录


,STUDYID,DOMAIN,USUBJID,AESEQ,AETERM,AEDECOD,AESTDTC,AEENDTC,AESEV,AESER,AEREL,AEOUT
0,STUDY001,AE,STUDY001-001,1,HEADACHE,COUGH,2024-03-06,2024-02-24,MODERATE,N,NOT RELATED,RECOVERING/RESOLVING
1,STUDY001,AE,STUDY001-002,2,FEVER,NAUSEA,2024-02-29,2024-03-25,MODERATE,N,NOT RELATED,RECOVERED/RESOLVED
2,STUDY001,AE,STUDY001-002,3,DIZZINESS,HEADACHE,2024-01-28,2024-02-09,MILD,N,POSSIBLY RELATED,NOT RECOVERED/NOT RESOLVED
3,STUDY001,AE,STUDY001-002,4,HEADACHE,HEADACHE,2024-03-02,2024-03-18,MILD,N,NOT RELATED,RECOVERING/RESOLVING
4,STUDY001,AE,STUDY001-003,5,FATIGUE,DIARRHEA,2024-01-22,2024-02-15,MILD,N,POSSIBLY RELATED,NOT RECOVERED/NOT RESOLVED


### 2.3 创建生命体征域 (VS)

In [4]:
# 创建 VS (Vital Signs) 域
vs_tests = {
    'SYSBP': ('Systolic Blood Pressure', 'mmHg', 110, 140, 10),
    'DIABP': ('Diastolic Blood Pressure', 'mmHg', 70, 90, 8),
    'PULSE': ('Pulse Rate', 'beats/min', 60, 80, 10),
    'TEMP': ('Temperature', 'C', 36.5, 37.5, 0.3),
    'WEIGHT': ('Weight', 'kg', 60, 90, 10)
}

vs_records = []
vs_seq = 1

for usubjid in dm_data['USUBJID'][:20]:  # 只为前20个受试者创建
    # 每个受试者3个访视的生命体征
    for visit in ['SCREENING', 'WEEK 4', 'WEEK 8']:
        for test_code, (test_name, unit, mean, std, var) in vs_tests.items():
            vs_records.append({
                'STUDYID': 'STUDY001',
                'DOMAIN': 'VS',
                'USUBJID': usubjid,
                'VSSEQ': vs_seq,
                'VSTESTCD': test_code,
                'VSTEST': test_name,
                'VSORRES': round(np.random.uniform(mean-std, mean+std), 1),
                'VSORRESU': unit,
                'VSDTC': (pd.Timestamp('2024-01-10') + pd.Timedelta(days=np.random.randint(0, 7))).strftime('%Y-%m-%d'),
                'VISIT': visit
            })
            vs_seq += 1

vs_data = pd.DataFrame(vs_records)
vs_data.to_csv(data_dir / 'vs.csv', index=False)
print(f"✓ 创建 VS 域: {len(vs_data)} 条记录")
vs_data.head()

✓ 创建 VS 域: 300 条记录


,STUDYID,DOMAIN,USUBJID,VSSEQ,VSTESTCD,VSTEST,VSORRES,VSORRESU,VSDTC,VISIT
0,STUDY001,VS,STUDY001-001,1,SYSBP,Systolic Blood Pressure,162.4,mmHg,2024-01-16,SCREENING
1,STUDY001,VS,STUDY001-001,2,DIABP,Diastolic Blood Pressure,72.1,mmHg,2024-01-16,SCREENING
2,STUDY001,VS,STUDY001-001,3,PULSE,Pulse Rate,-15.1,beats/min,2024-01-11,SCREENING
3,STUDY001,VS,STUDY001-001,4,TEMP,Temperature,32.6,C,2024-01-12,SCREENING
4,STUDY001,VS,STUDY001-001,5,WEIGHT,Weight,41.5,kg,2024-01-10,SCREENING


## 3. 配置临床数据库连接

### 3.1 创建配置文件

### 3.2 步骤 1: 准备数据文件

我们已经创建了 3 个 SDTM CSV 文件，让我们先查看一下它们的结构：

### 3.3 步骤 2: 创建表结构元数据文件

为了让 Catalog 理解我们的数据，需要创建一个描述表结构的 YAML 文件。这个文件包含：
- 表的名称和描述
- 每列的名称、显示名称、描述
- 数据类型信息

这相当于给 LLM 提供一个"说明书"，告诉它每个字段的含义。

In [5]:
import yaml

# 创建表结构元数据（table_info.yaml）
table_metadata = {
    'tables': [
        {
            'name': 'dm',
            'display_name': '人口统计学域 (Demographics)',
            'description': '受试者的基本信息，包括年龄、性别、种族、入组日期、治疗分组等',
            'columns': [
                {'name': 'STUDYID', 'display_name': '研究ID', 'description': '研究的唯一标识符', 'data_type': 'string'},
                {'name': 'USUBJID', 'display_name': '唯一受试者ID', 'description': '受试者在研究中的唯一标识', 'data_type': 'string'},
                {'name': 'SUBJID', 'display_name': '受试者ID', 'description': '受试者编号', 'data_type': 'string'},
                {'name': 'AGE', 'display_name': '年龄', 'description': '受试者年龄（数值）', 'data_type': 'integer'},
                {'name': 'SEX', 'display_name': '性别', 'description': 'M=男性, F=女性', 'data_type': 'string'},
                {'name': 'RACE', 'display_name': '种族', 'description': '受试者的种族分类', 'data_type': 'string'},
                {'name': 'ARMCD', 'display_name': '治疗组代码', 'description': 'TRT=治疗组, PLB=安慰剂组', 'data_type': 'string'},
                {'name': 'ARM', 'display_name': '治疗组', 'description': '治疗组的完整名称', 'data_type': 'string'},
                {'name': 'RFSTDTC', 'display_name': '首次用药日期', 'description': '受试者首次接受研究药物的日期', 'data_type': 'date'},
                {'name': 'SITEID', 'display_name': '中心ID', 'description': '研究中心标识', 'data_type': 'string'}
            ]
        },
        {
            'name': 'ae',
            'display_name': '不良事件域 (Adverse Events)',
            'description': '记录研究期间发生的所有不良事件，包括症状、严重程度、因果关系等',
            'columns': [
                {'name': 'STUDYID', 'display_name': '研究ID', 'description': '研究的唯一标识符', 'data_type': 'string'},
                {'name': 'USUBJID', 'display_name': '唯一受试者ID', 'description': '发生AE的受试者ID', 'data_type': 'string'},
                {'name': 'AESEQ', 'display_name': 'AE序号', 'description': '不良事件序列号', 'data_type': 'integer'},
                {'name': 'AETERM', 'display_name': '不良事件术语', 'description': '不良事件的描述性名称', 'data_type': 'string'},
                {'name': 'AESEV', 'display_name': '严重程度', 'description': 'MILD=轻度, MODERATE=中度, SEVERE=重度', 'data_type': 'string'},
                {'name': 'AESER', 'display_name': '严重AE标志', 'description': 'Y=严重不良事件, N=非严重', 'data_type': 'string'},
                {'name': 'AEREL', 'display_name': '因果关系', 'description': '与研究药物的因果关系评估', 'data_type': 'string'},
                {'name': 'AESTDTC', 'display_name': 'AE开始日期', 'description': '不良事件开始的日期', 'data_type': 'date'},
                {'name': 'AEOUT', 'display_name': 'AE结局', 'description': '不良事件的最终结局', 'data_type': 'string'}
            ]
        },
        {
            'name': 'vs',
            'display_name': '生命体征域 (Vital Signs)',
            'description': '记录受试者的生命体征测量值，包括血压、脉搏、体温、体重等',
            'columns': [
                {'name': 'STUDYID', 'display_name': '研究ID', 'description': '研究的唯一标识符', 'data_type': 'string'},
                {'name': 'USUBJID', 'display_name': '唯一受试者ID', 'description': '测量的受试者ID', 'data_type': 'string'},
                {'name': 'VSSEQ', 'display_name': 'VS序号', 'description': '生命体征记录序列号', 'data_type': 'integer'},
                {'name': 'VSTESTCD', 'display_name': '检查代码', 'description': 'SYSBP=收缩压, DIABP=舒张压, PULSE=脉搏, TEMP=体温, WEIGHT=体重', 'data_type': 'string'},
                {'name': 'VSTEST', 'display_name': '检查名称', 'description': '生命体征检查的完整名称', 'data_type': 'string'},
                {'name': 'VSORRES', 'display_name': '原始结果', 'description': '测量的原始数值', 'data_type': 'float'},
                {'name': 'VSORRESU', 'display_name': '单位', 'description': '测量值的单位', 'data_type': 'string'},
                {'name': 'VISIT', 'display_name': '访视', 'description': '测量时的访视名称', 'data_type': 'string'},
                {'name': 'VSDTC', 'display_name': '测量日期', 'description': '生命体征测量的日期', 'data_type': 'date'}
            ]
        }
    ]
}

# 保存到文件
metadata_file = data_dir / 'table_info.yaml'
with open(metadata_file, 'w', encoding='utf-8') as f:
    yaml.dump(table_metadata, f, default_flow_style=False, allow_unicode=True, sort_keys=False)

print(f"✅ 表结构元数据已创建: {metadata_file}")
print(f"\n包含 {len(table_metadata['tables'])} 个表的定义")
print("\n元数据预览:")
print(yaml.dump(table_metadata['tables'][0], default_flow_style=False, allow_unicode=True)[:500] + "...")

✅ 表结构元数据已创建: data\sdtm\table_info.yaml

包含 3 个表的定义

元数据预览:
columns:
- data_type: string
  description: 研究的唯一标识符
  display_name: 研究ID
  name: STUDYID
- data_type: string
  description: 受试者在研究中的唯一标识
  display_name: 唯一受试者ID
  name: USUBJID
- data_type: string
  description: 受试者编号
  display_name: 受试者ID
  name: SUBJID
- data_type: integer
  description: 受试者年龄（数值）
  display_name: 年龄
  name: AGE
- data_type: string
  description: M=男性, F=女性
  display_name: 性别
  name: SEX
- data_type: string
  description: 受试者的种族分类
  display_name: 种族
  name: RACE
- data_type: s...


## 4. Catalog 模块使用说明

### 总结：Catalog 模块完整工作流

```
用户问题: "显示年龄超过60岁且有严重不良事件的受试者"
    ↓
┌─────────────────────────────────────────────────────────────┐
│ 1. catalog_store.py - 提供数据访问接口                        │
│    • get_table_list() → ['dm', 'ae', 'vs']                 │
│    • get_column_list('dm') → [AGE, SEX, ARMCD, ...]        │
└─────────────────────────────────────────────────────────────┘
    ↓
┌─────────────────────────────────────────────────────────────┐
│ 2. schema_retrival.py - 智能检索相关表和列                    │
│    • 分词: "年龄", "超过", "60", "严重", "不良事件"          │
│    • BM25 检索: AGE (dm), AESER (ae), USUBJID (dm,ae)      │
│    • 确定涉及的表: dm, ae                                    │
└─────────────────────────────────────────────────────────────┘
    ↓
┌─────────────────────────────────────────────────────────────┐
│ 3. retrival_helper.py - 辅助函数                             │
│    • build_column_tables_mapping() → {AGE: ['dm'], ...}    │
│    • 确认关联键: USUBJID                                     │
└─────────────────────────────────────────────────────────────┘
    ↓
┌─────────────────────────────────────────────────────────────┐
│ 4. 生成 SQL                                                  │
│    SELECT dm.USUBJID, dm.AGE, ae.AETERM                     │
│    FROM dm                                                   │
│    INNER JOIN ae ON dm.USUBJID = ae.USUBJID                 │
│    WHERE dm.AGE > 60 AND ae.AESER = 'Y'                     │
└─────────────────────────────────────────────────────────────┘
```

**关键文件说明**:

1. **catalog_store.py**: 抽象基类，定义接口
2. **store/file_system.py**: 从文件加载的实现
3. **catalog_loader.py**: 从数据库自动加载结构
4. **schema_retrival.py**: 智能检索核心逻辑
5. **retrival_helper.py**: 构建索引和映射的辅助函数
6. **helper.py**: 通用辅助函数
7. **factory.py**: 工厂模式创建 catalog 实例
8. **token_service.py**: 处理 API 认证（可选）

### 4.1 使用 catalog_store.py - Catalog 存储接口

`catalog_store.py` 定义了 CatalogStore 抽象基类，它是所有 catalog 实现的接口。

**核心功能**:
- `get_table_list()`: 获取所有表名
- `get_column_list(table_name)`: 获取表的列信息
- `get_table_info(table_name)`: 获取表的详细信息

让我们看看如何使用它：

In [6]:
# 首先需要创建一个简单的 Catalog Store 实例
# 我们使用 FileSystemCatalogStore（基于文件系统的实现）

from openchatbi.catalog.store.file_system import FileSystemCatalogStore
import json

# 创建 catalog store
# 这里使用内存模式，直接从我们的元数据创建
class SimpleCatalogStore:
    """简化的 Catalog Store 用于演示"""
    
    def __init__(self, metadata):
        self.metadata = metadata
        self._tables = {t['name']: t for t in metadata['tables']}
    
    def get_table_list(self):
        """获取所有表名"""
        return list(self._tables.keys())
    
    def get_table_info(self, table_name):
        """获取表的详细信息"""
        return self._tables.get(table_name, {})
    
    def get_column_list(self, table_name=None):
        """获取列列表"""
        if table_name:
            table = self._tables.get(table_name, {})
            return table.get('columns', [])
        else:
            # 返回所有表的所有列
            all_columns = []
            for table in self._tables.values():
                for col in table.get('columns', []):
                    col_info = col.copy()
                    col_info['table_name'] = table['name']
                    all_columns.append(col_info)
            return all_columns

# 创建 catalog 实例
catalog = SimpleCatalogStore(table_metadata)

print("=" * 70)
print("📚 Catalog Store 演示")
print("=" * 70)

# 1. 获取所有表
print("\n1️⃣ 获取所有表名:")
tables = catalog.get_table_list()
print(f"   共有 {len(tables)} 个表: {', '.join(tables)}")

# 2. 获取特定表的信息
print("\n2️⃣ 获取 DM 表的详细信息:")
dm_info = catalog.get_table_info('dm')
print(f"   表名: {dm_info['name']}")
print(f"   显示名: {dm_info['display_name']}")
print(f"   描述: {dm_info['description']}")
print(f"   列数: {len(dm_info['columns'])}")

# 3. 获取表的列信息
print("\n3️⃣ 获取 AE 表的列信息:")
ae_columns = catalog.get_column_list('ae')
print(f"   共有 {len(ae_columns)} 列:")
for col in ae_columns[:5]:  # 只显示前5列
    print(f"   - {col['name']:<12} ({col['display_name']}): {col['description'][:40]}...")

# 4. 获取所有列（用于全局搜索）
print("\n4️⃣ 获取所有表的所有列:")
all_columns = catalog.get_column_list()
print(f"   总共 {len(all_columns)} 列")
print(f"   示例 - 与'日期'相关的列:")
date_columns = [c for c in all_columns if '日期' in c.get('display_name', '') or '日期' in c.get('description', '')]
for col in date_columns[:5]:
    print(f"   - {col['table_name']}.{col['name']}: {col['display_name']}")

📚 Catalog Store 演示

1️⃣ 获取所有表名:
   共有 3 个表: dm, ae, vs

2️⃣ 获取 DM 表的详细信息:
   表名: dm
   显示名: 人口统计学域 (Demographics)
   描述: 受试者的基本信息，包括年龄、性别、种族、入组日期、治疗分组等
   列数: 10

3️⃣ 获取 AE 表的列信息:
   共有 9 列:
   - STUDYID      (研究ID): 研究的唯一标识符...
   - USUBJID      (唯一受试者ID): 发生AE的受试者ID...
   - AESEQ        (AE序号): 不良事件序列号...
   - AETERM       (不良事件术语): 不良事件的描述性名称...
   - AESEV        (严重程度): MILD=轻度, MODERATE=中度, SEVERE=重度...

4️⃣ 获取所有表的所有列:
   总共 28 列
   示例 - 与'日期'相关的列:
   - dm.RFSTDTC: 首次用药日期
   - ae.AESTDTC: AE开始日期
   - vs.VSDTC: 测量日期


### 步骤 2: 使用 schema_retrival.py - 智能检索相关表和列

`schema_retrival.py` 是 catalog 的核心功能之一，它能根据用户的问题智能地找到相关的表和列。

**工作原理**:
1. **BM25 检索**: 使用关键词匹配找到相关列
2. **向量检索**: 使用语义相似度找到相关列
3. **混合检索**: 结合两种方法的结果

这个模块包含:
- `build_column_retrieval()`: 构建列检索系统
- `build_table_retrieval()`: 构建表检索系统

In [7]:
# 演示智能检索功能
from openchatbi.catalog.retrival_helper import get_columns_metadata, build_column_tables_mapping
from openchatbi.text_segmenter import _segmenter
from rank_bm25 import BM25Okapi

print("=" * 70)
print("🔍 Schema Retrieval - 智能检索演示")
print("=" * 70)

# 准备列数据用于检索
all_columns = catalog.get_column_list()

# 构建列的搜索索引
column_texts = []
column_info_list = []

for col in all_columns:
    # 组合列的所有文本信息
    text = f"{col['name']} {col.get('display_name', '')} {col.get('description', '')}"
    column_texts.append(text)
    column_info_list.append(col)

# 使用中文分词器对文本进行分词
tokenized_corpus = [list(_segmenter.cut(text)) for text in column_texts]

# 创建 BM25 检索器
bm25 = BM25Okapi(tokenized_corpus)

print("\n✅ 已构建 BM25 检索索引")
print(f"   索引了 {len(column_info_list)} 个列")

# 测试几个查询
queries = [
    "年龄大于60岁的受试者",
    "严重不良事件",
    "血压异常",
    "治疗组对比"
]

for query in queries:
    print(f"\n{'='*70}")
    print(f"🔎 查询: '{query}'")
    print(f"{'='*70}")
    
    # 分词
    query_tokens = list(_segmenter.cut(query))
    print(f"   分词结果: {', '.join(query_tokens)}")
    
    # BM25 检索
    scores = bm25.get_scores(query_tokens)
    
    # 获取前5个最相关的列
    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:5]
    
    print(f"\n   📊 最相关的列 (Top 5):")
    for rank, idx in enumerate(top_indices, 1):
        col = column_info_list[idx]
        score = scores[idx]
        print(f"   {rank}. [{score:.2f}] {col.get('table_name', '?')}.{col['name']}")
        print(f"      显示名: {col.get('display_name', 'N/A')}")
        print(f"      描述: {col.get('description', 'N/A')[:50]}...")
        print()

🔍 Schema Retrieval - 智能检索演示

✅ 已构建 BM25 检索索引
   索引了 28 个列

🔎 查询: '年龄大于60岁的受试者'
   分词结果: 年龄大于60岁的受试者

   📊 最相关的列 (Top 5):
   1. [0.00] dm.STUDYID
      显示名: 研究ID
      描述: 研究的唯一标识符...

   2. [0.00] dm.USUBJID
      显示名: 唯一受试者ID
      描述: 受试者在研究中的唯一标识...

   3. [0.00] dm.SUBJID
      显示名: 受试者ID
      描述: 受试者编号...

   4. [0.00] dm.AGE
      显示名: 年龄
      描述: 受试者年龄（数值）...

   5. [0.00] dm.SEX
      显示名: 性别
      描述: M=男性, F=女性...


🔎 查询: '严重不良事件'
   分词结果: 严重不良事件

   📊 最相关的列 (Top 5):
   1. [2.33] ae.AESER
      显示名: 严重AE标志
      描述: Y=严重不良事件, N=非严重...

   2. [0.00] dm.STUDYID
      显示名: 研究ID
      描述: 研究的唯一标识符...

   3. [0.00] dm.USUBJID
      显示名: 唯一受试者ID
      描述: 受试者在研究中的唯一标识...

   4. [0.00] dm.SUBJID
      显示名: 受试者ID
      描述: 受试者编号...

   5. [0.00] dm.AGE
      显示名: 年龄
      描述: 受试者年龄（数值）...


🔎 查询: '血压异常'
   分词结果: 血压异常

   📊 最相关的列 (Top 5):
   1. [0.00] dm.STUDYID
      显示名: 研究ID
      描述: 研究的唯一标识符...

   2. [0.00] dm.USUBJID
      显示名: 唯一受试者ID
      描述: 受试者在研究中的唯一标识...

   3. [0.00] 

### 步骤 3: 使用 helper.py - 列到表的映射

`helper.py` 中的 `build_column_tables_mapping()` 函数构建列名到表名的映射。

**作用**: 当我们知道需要某个列时，快速找到它属于哪个表。

In [8]:
# 构建列到表的映射
from collections import defaultdict

column_tables_mapping = defaultdict(list)

for table_name in catalog.get_table_list():
    columns = catalog.get_column_list(table_name)
    for col in columns:
        column_name = col['name']
        column_tables_mapping[column_name].append(table_name)

print("=" * 70)
print("🗺️  列到表的映射")
print("=" * 70)

print(f"\n总共映射了 {len(column_tables_mapping)} 个不同的列名")

# 查找共享列（在多个表中都有的列）
shared_columns = {col: tables for col, tables in column_tables_mapping.items() if len(tables) > 1}

print(f"\n📌 共享列（在多个表中都存在）:")
for col_name, tables in sorted(shared_columns.items()):
    print(f"   - {col_name:<15} 出现在: {', '.join(tables)}")

# 查找特定列属于哪个表
print(f"\n🔍 查找特定列所在的表:")
search_columns = ['AGE', 'AETERM', 'VSTESTCD', 'AEREL']
for col_name in search_columns:
    tables = column_tables_mapping.get(col_name, [])
    if tables:
        print(f"   - {col_name:<12} → {', '.join(tables)}")
    else:
        print(f"   - {col_name:<12} → 未找到")

🗺️  列到表的映射

总共映射了 24 个不同的列名

📌 共享列（在多个表中都存在）:
   - STUDYID         出现在: dm, ae, vs
   - USUBJID         出现在: dm, ae, vs

🔍 查找特定列所在的表:
   - AGE          → dm
   - AETERM       → ae
   - VSTESTCD     → vs
   - AEREL        → ae


### 步骤 4: 完整的 Schema Linking 流程

现在我们把所有部分组合起来，演示完整的 schema linking 流程：从用户问题到识别相关的表和列。

In [ ]:
def schema_linking_demo(user_question, top_k=3):
    """
    完整的 schema linking 演示
    
    步骤:
    1. 分析用户问题
    2. 检索相关列
    3. 确定涉及的表
    4. 返回建议的表和列
    """
    print("=" * 80)
    print(f"🎯 用户问题: {user_question}")
    print("=" * 80)
    
    # 步骤 1: 分词
    query_tokens = list(_segmenter.cut(user_question))
    print(f"\n1️⃣ 分词结果: {', '.join(query_tokens)}")
    
    # 步骤 2: BM25 检索相关列
    scores = bm25.get_scores(query_tokens)
    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k * 3]
    
    print(f"\n2️⃣ 检索到的相关列 (Top {top_k * 3}):")
    relevant_columns = []
    for rank, idx in enumerate(top_indices, 1):
        col = column_info_list[idx]
        score = scores[idx]
        if score > 0:  # 只保留有分数的
            print(f"   {rank}. [{score:.2f}] {col.get('table_name', '?')}.{col['name']} ({col.get('display_name', '')})")
            relevant_columns.append(col)
    
    # 步骤 3: 确定涉及的表
    involved_tables = set()
    selected_columns = {}
    
    for col in relevant_columns[:top_k * 2]:
        table_name = col.get('table_name')
        if table_name:
            involved_tables.add(table_name)
            if table_name not in selected_columns:
                selected_columns[table_name] = []
            selected_columns[table_name].append(col)
    
    print(f"\n3️⃣ 涉及的表: {', '.join(sorted(involved_tables))}")
    
    # 步骤 4: 汇总建议
    print(f"\n4️⃣ Schema Linking 结果:")
    for table_name in sorted(selected_columns.keys()):
        table_info = catalog.get_table_info(table_name)
        print(f"\n   📋 表: {table_name} - {table_info.get('display_name', '')}")
        print(f"      描述: {table_info.get('description', '')}")
        print(f"      建议使用的列:")
        for col in selected_columns[table_name][:top_k]:
            print(f"        • {col['name']:<12} ({col.get('display_name', '')})")
    
    # 步骤 5: 生成 SQL 提示
    print(f"\n5️⃣ SQL 查询提示:")
    if len(involved_tables) == 1:
        table = list(involved_tables)[0]
        columns = [c['name'] for c in selected_columns[table][:top_k]]
        print(f"   SELECT {', '.join(columns)}")
        print(f"   FROM {table}")
        print(f"   WHERE ...")
    else:
        print(f"   可能需要 JOIN 以下表: {', '.join(sorted(involved_tables))}")
        print(f"   关联键: USUBJID (所有表共享)")
    
    return involved_tables, selected_columns

# 测试不同的用户问题
test_questions = [
    "有多少名年龄超过60岁的受试者？",
    "显示所有严重不良事件的受试者和症状",
    "统计各治疗组的收缩压平均值",
    "哪些受试者同时有不良事件和异常生命体征？"
]

for i, question in enumerate(test_questions, 1):
    print(f"\n\n{'#' * 80}")
    print(f"示例 {i}")
    print(f"{'#' * 80}\n")
    schema_linking_demo(question, top_k=3)
    if i < len(test_questions):
        input("\n按 Enter 继续下一个示例...")



################################################################################
示例 1
################################################################################

🎯 用户问题: 有多少名年龄超过60岁的受试者？

1️⃣ 分词结果: 有多少名年龄超过60岁的受试者

2️⃣ 检索到的相关列 (Top 9):

3️⃣ 涉及的表: 

4️⃣ Schema Linking 结果:

5️⃣ SQL 查询提示:
   可能需要 JOIN 以下表: 
   关联键: USUBJID (所有表共享)


################################################################################
示例 2
################################################################################

🎯 用户问题: 显示所有严重不良事件的受试者和症状

1️⃣ 分词结果: 显示所有严重不良事件的受试者和症状

2️⃣ 检索到的相关列 (Top 9):

3️⃣ 涉及的表: 

4️⃣ Schema Linking 结果:

5️⃣ SQL 查询提示:
   可能需要 JOIN 以下表: 
   关联键: USUBJID (所有表共享)


### 步骤 5: 使用 catalog_loader.py - 从数据库加载 Catalog

`catalog_loader.py` 的 `load_catalog_from_data_warehouse()` 函数可以直接从数据库连接加载表结构。

**支持的数据库**:
- SQLite
- PostgreSQL
- MySQL
- SQL Server
- Trino/Presto

让我们创建一个 SQLite 数据库并演示自动加载：

In [ ]:
import sqlite3
from sqlalchemy import create_engine, inspect
import pandas as pd

print("=" * 70)
print("💾 从数据库自动加载 Catalog")
print("=" * 70)

# 创建 SQLite 数据库
db_path = data_dir / 'sdtm.db'
engine = create_engine(f'sqlite:///{db_path}')

# 将 CSV 数据导入数据库
print("\n1️⃣ 创建 SQLite 数据库并导入数据...")
dm_df = pd.read_csv(data_dir / 'dm.csv')
ae_df = pd.read_csv(data_dir / 'ae.csv')
vs_df = pd.read_csv(data_dir / 'vs.csv')

dm_df.to_sql('dm', engine, if_exists='replace', index=False)
ae_df.to_sql('ae', engine, if_exists='replace', index=False)
vs_df.to_sql('vs', engine, if_exists='replace', index=False)

print(f"   ✅ 已创建数据库: {db_path}")
print(f"   ✅ 导入了 3 个表")

# 使用 SQLAlchemy Inspector 自动检查表结构
print("\n2️⃣ 使用 SQLAlchemy Inspector 检查表结构:")
inspector = inspect(engine)

for table_name in inspector.get_table_names():
    print(f"\n   📋 表: {table_name}")
    
    # 获取列信息
    columns = inspector.get_columns(table_name)
    print(f"      列数: {len(columns)}")
    print(f"      列信息:")
    for col in columns[:5]:  # 只显示前5列
        col_type = str(col['type'])
        nullable = "NULL" if col['nullable'] else "NOT NULL"
        print(f"        • {col['name']:<15} {col_type:<10} {nullable}")
    
    if len(columns) > 5:
        print(f"        ... 还有 {len(columns) - 5} 列")
    
    # 获取主键
    pk = inspector.get_pk_constraint(table_name)
    if pk and pk.get('constrained_columns'):
        print(f"      主键: {', '.join(pk['constrained_columns'])}")
    
    # 获取记录数
    with engine.connect() as conn:
        result = conn.execute(f"SELECT COUNT(*) FROM {table_name}")
        count = result.fetchone()[0]
        print(f"      记录数: {count}")

print("\n3️⃣ 模拟 catalog_loader.load_catalog_from_data_warehouse():")
print("   这个函数会:")
print("   • 连接到数据库")
print("   • 自动发现所有表")
print("   • 提取每个表的列信息（名称、类型、约束）")
print("   • 构建 Catalog Store 对象")
print("   • 返回可用于查询的 catalog 实例")

print("\n✅ Catalog 加载完成！")
print(f"\n💡 提示: 实际使用时，在配置文件中指定数据库连接字符串：")
print(f"   catalog:")
print(f"     type: database")
print(f"     connection_string: 'sqlite:///{db_path}'")

### 步骤 6: 使用 factory.py - 创建 Catalog Store 实例

`factory.py` 中的 `create_catalog_store()` 是工厂函数，根据配置创建不同类型的 Catalog Store。

**支持的类型**:
- `file_system`: 从本地文件加载（YAML/JSON）
- `database`: 从数据库连接加载
- `memory`: 从内存数据结构创建

In [ ]:
from openchatbi.catalog.factory import create_catalog_store

print("=" * 70)
print("🏭 Catalog Factory - 创建不同类型的 Catalog Store")
print("=" * 70)

# 方法 1: 从文件系统创建
print("\n1️⃣ 从文件系统创建 Catalog Store:")
print("   配置示例:")
file_config = {
    'catalog': {
        'type': 'file_system',
        'catalog_dir': str(data_dir),
        'table_info_file': 'table_info.yaml'
    }
}
print(f"   {yaml.dump(file_config, default_flow_style=False)}")

# 方法 2: 从数据库创建
print("\n2️⃣ 从数据库创建 Catalog Store:")
print("   配置示例:")
db_config = {
    'catalog': {
        'type': 'database',
        'connection_string': f'sqlite:///{db_path}',
        'schema': None  # SQLite 不需要 schema
    }
}
print(f"   {yaml.dump(db_config, default_flow_style=False)}")

# 方法 3: 使用 factory 创建
print("\n3️⃣ 使用 factory 函数创建:")
print("   catalog_store = create_catalog_store(config)")
print("   这会根据 config['catalog']['type'] 自动选择正确的实现")

# 实际创建一个 catalog store（简化版）
print("\n4️⃣ 实际使用示例:")
print(f"   假设配置文件路径: openchatbi/config.yaml")
print(f"   在应用启动时:")
print(f"   ```python")
print(f"   from openchatbi import config")
print(f"   catalog_store = config.get().catalog_store")
print(f"   ")
print(f"   # 现在可以使用 catalog_store 进行查询")
print(f"   tables = catalog_store.get_table_list()")
print(f"   columns = catalog_store.get_column_list('dm')")
print(f"   ```")

print("\n✅ Factory 模式的优点:")
print("   • 统一的接口，不同的实现")
print("   • 易于切换数据源")
print("   • 便于测试和扩展")

### 4.8 实战练习

现在你可以尝试修改和扩展 Catalog：

In [ ]:
print("=" * 70)
print("🎓 实战练习")
print("=" * 70)

print("\n📝 练习 1: 添加新的 SDTM 域")
print("-" * 70)
print("任务: 创建 LB (Laboratory) 域并添加到 Catalog")
print()
print("步骤:")
print("1. 创建 lb.csv 文件（实验室检查数据）")
print("2. 在 table_info.yaml 中添加 LB 表的元数据")
print("3. 更新 Catalog Store")
print("4. 测试检索功能")
print()
print("提示列名: LBTESTCD, LBTEST, LBORRES, LBORRESU, LBNRIND (正常值标志)")

print("\n📝 练习 2: 改进检索算法")
print("-" * 70)
print("任务: 添加同义词支持")
print()
print("步骤:")
print("1. 创建同义词字典")
print("   例如: {'受试者': ['SUBJID', 'USUBJID'], '病人': ['SUBJID']}")
print("2. 在检索前扩展查询词")
print("3. 比较改进前后的检索效果")

print("\n📝 练习 3: 构建表关系图")
print("-" * 70)
print("任务: 可视化表之间的关系")
print()
print("步骤:")
print("1. 识别共享列（如 USUBJID）")
print("2. 确定表之间的关联关系")
print("3. 使用 graphviz 或其他工具绘制 ER 图")

print("\n📝 练习 4: 实现向量检索")
print("-" * 70)
print("任务: 使用 embedding 模型进行语义检索")
print()
print("步骤:")
print("1. 使用 langchain 的 embedding 模型")
print("2. 为所有列生成 embedding")
print("3. 实现向量相似度检索")
print("4. 与 BM25 结果进行融合")

print("\n💡 提示: 完成这些练习可以帮助你:")
print("   • 深入理解 Catalog 架构")
print("   • 为临床数据库定制功能")
print("   • 提高检索准确性")
print("   • 扩展到更多 SDTM 域")

print("\n" + "=" * 70)
print("✅ Catalog 教程完成！")
print("=" * 70)

## 5. openchatbi/code/ - Python 代码执行器模块

**作用**: 提供安全的 Python 代码执行环境，用于数据分析、可视化等任务。

### 5.1 模块架构概览

```
openchatbi/code/
├── executor_base.py              # 抽象基类，定义执行器接口
├── local_executor.py             # 本地执行器（无限制）
├── restricted_local_executor.py  # 受限本地执行器（使用 RestrictedPython）
└── docker_executor.py            # Docker 容器执行器（已移除）
```

**核心概念**:
- **ExecutorBase**: 所有执行器的抽象基类，定义统一接口
- **LocalExecutor**: 直接在本地环境执行 Python 代码，功能完整但无安全限制
- **RestrictedLocalExecutor**: 使用 RestrictedPython 限制危险操作（如文件系统访问）
- **DockerExecutor**: 在 Docker 容器中隔离执行（已移除，不推荐使用）

**临床应用场景**:
- 动态数据分析（计算统计指标、生成报表）
- 创建可视化图表（Plotly、Matplotlib）
- 数据转换和清洗
- 自定义计算逻辑

### 5.2 executor_base.py - 执行器抽象基类

`ExecutorBase` 定义了所有代码执行器必须实现的接口。

**核心方法**:
- `run_code(code: str) -> (bool, str)`: 执行代码并返回结果
- `set_variable(key: str, value: Any)`: 设置执行环境中的变量

**设计模式**: 模板方法模式（Template Method Pattern）

In [ ]:
from openchatbi.code.executor_base import ExecutorBase

print("=" * 70)
print("📘 ExecutorBase - 执行器基类接口")
print("=" * 70)

# 查看 ExecutorBase 的结构
print("\n1️⃣ ExecutorBase 类定义:")
print("   class ExecutorBase:")
print("       _variable: dict              # 存储变量的字典")
print("       __init__(variable: dict)     # 初始化变量环境")
print("       run_code(code: str) -> (bool, str)  # 执行代码（抽象方法）")
print("       set_variable(key, value)     # 设置变量")

print("\n2️⃣ 使用场景:")
print("   • Agent 需要执行动态生成的 Python 代码")
print("   • 需要在代码执行前预设变量（如 DataFrame）")
print("   • 需要捕获代码的输出和错误")

print("\n3️⃣ 返回值格式:")
print("   (success: bool, output: str)")
print("   - success: True 表示执行成功，False 表示有错误")
print("   - output: 执行的输出结果或错误信息")

print("\n💡 提示: 所有执行器都继承自 ExecutorBase 并实现 run_code() 方法")

### 5.3 local_executor.py - 本地执行器（无限制）

`LocalExecutor` 直接在当前 Python 环境中执行代码，**没有任何安全限制**。

**特点**:
- ✅ 功能完整，支持所有 Python 特性
- ✅ 执行速度快
- ⚠️ 无安全限制，可以访问文件系统、网络等
- ⚠️ 仅适用于可信代码

**实现原理**: 使用 Python 内置的 `exec()` 函数执行代码

In [ ]:
from openchatbi.code.local_executor import LocalExecutor
import pandas as pd

print("=" * 70)
print("🚀 LocalExecutor - 本地执行器演示")
print("=" * 70)

# 创建执行器实例
executor = LocalExecutor()

print("\n【示例 1】基本代码执行 - 简单计算")
print("-" * 70)
code1 = """
result = 10 + 20
print(f"计算结果: {result}")
"""
success, output = executor.run_code(code1)
print(f"执行状态: {'✅ 成功' if success else '❌ 失败'}")
print(f"输出内容:\n{output}")

print("\n【示例 2】使用 SDTM 数据进行分析")
print("-" * 70)
# 加载 DM 数据
dm_df = pd.read_csv('data/sdtm/dm.csv')

# 将数据传递给执行器
executor.set_variable('dm', dm_df)

code2 = """
# 分析受试者年龄分布
print("受试者年龄统计:")
print(dm['AGE'].describe())

print("\\n各治疗组人数:")
print(dm['ARM'].value_counts())

# 计算平均年龄
avg_age = dm['AGE'].mean()
print(f"\\n平均年龄: {avg_age:.1f} 岁")
"""
success, output = executor.run_code(code2)
print(f"执行状态: {'✅ 成功' if success else '❌ 失败'}")
print(f"输出内容:\n{output}")

print("\n【示例 3】复杂数据分析 - 不良事件统计")
print("-" * 70)
ae_df = pd.read_csv('data/sdtm/ae.csv')
executor.set_variable('ae', ae_df)
executor.set_variable('dm', dm_df)

code3 = """
# 合并 DM 和 AE 数据
merged = dm.merge(ae, on='USUBJID', how='left')

# 统计各治疗组的 AE 发生率
ae_rate = merged.groupby('ARM')['AETERM'].count() / dm.groupby('ARM').size()
print("各治疗组不良事件发生率:")
for arm, rate in ae_rate.items():
    print(f"  {arm}: {rate:.2f} 次/人")

# 严重 AE 统计
sae_count = ae[ae['AESER'] == 'Y'].shape[0]
print(f"\\n严重不良事件总数: {sae_count}")
"""
success, output = executor.run_code(code3)
print(f"执行状态: {'✅ 成功' if success else '❌ 失败'}")
print(f"输出内容:\n{output}")

print("\n【示例 4】错误处理演示")
print("-" * 70)
code4 = """
# 故意制造一个错误
undefined_variable + 10
"""
success, output = executor.run_code(code4)
print(f"执行状态: {'✅ 成功' if success else '❌ 失败'}")
print(f"错误信息:\n{output}")

print("\n💡 关键点:")
print("   • LocalExecutor 可以执行任何 Python 代码")
print("   • 通过 set_variable() 传递 DataFrame 等对象")
print("   • 自动捕获 print() 输出")
print("   • 自动捕获异常并返回错误信息")

### 5.4 restricted_local_executor.py - 受限本地执行器（推荐）

`RestrictedLocalExecutor` 使用 **RestrictedPython** 库限制危险操作，提供安全的代码执行环境。

**安全限制**:
- ❌ 禁止导入某些模块（如 `os`, `subprocess`）
- ❌ 限制文件系统访问
- ❌ 禁止执行系统命令
- ✅ 允许数据分析操作（pandas, numpy 等）

**适用场景**:
- 执行 LLM 生成的代码
- 多用户环境
- 生产环境部署

**实现原理**: 使用 RestrictedPython 编译代码为受限字节码

In [ ]:
from openchatbi.code.restricted_local_executor import RestrictedLocalExecutor

print("=" * 70)
print("🔒 RestrictedLocalExecutor - 受限执行器演示")
print("=" * 70)

# 创建受限执行器
restricted_executor = RestrictedLocalExecutor()

print("\n【示例 1】允许的操作 - 数据分析")
print("-" * 70)
# 传递数据
restricted_executor.set_variable('dm', dm_df)
restricted_executor.set_variable('ae', ae_df)

code1 = """
# 计算各治疗组的平均年龄
avg_age_by_arm = dm.groupby('ARM')['AGE'].mean()
print("各治疗组平均年龄:")
for arm, age in avg_age_by_arm.items():
    print(f"  {arm}: {age:.1f} 岁")

# 统计不良事件
print(f"\\n总不良事件数: {len(ae)}")
print(f"严重不良事件数: {(ae['AESER'] == 'Y').sum()}")
"""
success, output = restricted_executor.run_code(code1)
print(f"执行状态: {'✅ 成功' if success else '❌ 失败'}")
print(f"输出内容:\n{output}")

print("\n【示例 2】允许的操作 - 数学计算")
print("-" * 70)
code2 = """
import math

# 计算标准差
ages = [25, 30, 35, 40, 45, 50]
mean_age = sum(ages) / len(ages)
variance = sum((x - mean_age) ** 2 for x in ages) / len(ages)
std_dev = math.sqrt(variance)

print(f"年龄列表: {ages}")
print(f"平均值: {mean_age:.2f}")
print(f"标准差: {std_dev:.2f}")
"""
success, output = restricted_executor.run_code(code2)
print(f"执行状态: {'✅ 成功' if success else '❌ 失败'}")
print(f"输出内容:\n{output}")

print("\n【示例 3】被禁止的操作 - 文件系统访问")
print("-" * 70)
code3 = """
import os
# 尝试列出文件
os.listdir('.')
"""
success, output = restricted_executor.run_code(code3)
print(f"执行状态: {'✅ 成功' if success else '❌ 失败'}")
print(f"错误信息:\n{output}")

print("\n【示例 4】被禁止的操作 - 系统命令")
print("-" * 70)
code4 = """
import subprocess
subprocess.run(['echo', 'hello'])
"""
success, output = restricted_executor.run_code(code4)
print(f"执行状态: {'✅ 成功' if success else '❌ 失败'}")
print(f"错误信息:\n{output}")

print("\n【示例 5】实际应用 - 生成临床报告")
print("-" * 70)
restricted_executor.set_variable('dm', dm_df)
restricted_executor.set_variable('ae', ae_df)

code5 = """
# 生成简单的安全性报告
print("=" * 50)
print("临床研究安全性概览报告")
print("=" * 50)

print(f"\\n入组受试者总数: {len(dm)}")
print(f"治疗组: {(dm['ARMCD'] == 'TRT').sum()} 人")
print(f"安慰剂组: {(dm['ARMCD'] == 'PLB').sum()} 人")

print(f"\\n不良事件总数: {len(ae)}")
print(f"受试者发生 AE 人数: {ae['USUBJID'].nunique()}")
print(f"AE 发生率: {ae['USUBJID'].nunique() / len(dm) * 100:.1f}%")

sae = ae[ae['AESER'] == 'Y']
print(f"\\n严重不良事件数: {len(sae)}")
print(f"SAE 发生率: {sae['USUBJID'].nunique() / len(dm) * 100:.1f}%")

print("\\n最常见的不良事件 (Top 5):")
for i, (term, count) in enumerate(ae['AETERM'].value_counts().head().items(), 1):
    print(f"  {i}. {term}: {count} 例")
"""
success, output = restricted_executor.run_code(code5)
print(f"执行状态: {'✅ 成功' if success else '❌ 失败'}")
print(f"输出内容:\n{output}")

print("\n🔐 安全特性总结:")
print("   • RestrictedPython 在编译时检查代码安全性")
print("   • 允许数据分析操作（pandas, numpy, math 等）")
print("   • 禁止文件系统、网络、系统命令等危险操作")
print("   • 适合执行 LLM 生成的代码")
print("   • 推荐在生产环境使用")

### 5.5 执行器对比与选择指南

让我们比较三种执行器的特点，帮助你选择合适的执行器。

In [ ]:
import pandas as pd

print("=" * 80)
print("📊 执行器对比表")
print("=" * 80)

comparison_data = {
    '特性': [
        '安全性',
        '功能完整性',
        '执行速度',
        '文件系统访问',
        '网络访问',
        '导入任意模块',
        '适用场景',
        '推荐度'
    ],
    'LocalExecutor': [
        '❌ 无限制',
        '✅ 完整',
        '⚡ 最快',
        '✅ 允许',
        '✅ 允许',
        '✅ 允许',
        '开发调试、可信代码',
        '⭐⭐⭐ (仅限开发)'
    ],
    'RestrictedLocalExecutor': [
        '✅ 受限',
        '✅ 数据分析功能完整',
        '⚡ 快（略慢于 Local）',
        '❌ 禁止',
        '❌ 禁止',
        '⚠️ 部分允许',
        '生产环境、LLM 生成代码',
        '⭐⭐⭐⭐⭐ (推荐)'
    ],
    'DockerExecutor': [
        '✅✅ 完全隔离',
        '✅ 完整',
        '🐌 慢（容器开销）',
        '⚠️ 容器内隔离',
        '⚠️ 容器内隔离',
        '✅ 允许',
        '极高安全要求',
        '⭐⭐ (已移除)'
    ]
}

df_comparison = pd.DataFrame(comparison_data)
print("\n", df_comparison.to_string(index=False))

print("\n\n" + "=" * 80)
print("🎯 选择指南")
print("=" * 80)

print("\n1️⃣ 【开发和测试阶段】")
print("   推荐: LocalExecutor")
print("   原因:")
print("   • 功能完整，可以使用所有 Python 特性")
print("   • 执行速度快，便于快速迭代")
print("   • 可以方便地调试和测试")
print("   • 代码来源可信（自己编写）")

print("\n2️⃣ 【生产环境部署】")
print("   推荐: RestrictedLocalExecutor ⭐⭐⭐⭐⭐")
print("   原因:")
print("   • 安全限制，防止恶意代码")
print("   • 适合执行 LLM 生成的代码")
print("   • 性能损失小")
print("   • 满足大部分数据分析需求")

print("\n3️⃣ 【极高安全要求】")
print("   推荐: DockerExecutor（已移除，不推荐）")
print("   原因:")
print("   • 完全隔离的容器环境")
print("   • 但性能开销大，配置复杂")
print("   • 已从项目中移除")

print("\n4️⃣ 【临床数据监控应用】")
print("   推荐: RestrictedLocalExecutor")
print("   原因:")
print("   • 符合数据安全和隐私要求")
print("   • 支持 pandas, numpy 等数据分析库")
print("   • 可以安全执行 LLM 生成的分析代码")
print("   • 防止意外的数据泄露")

print("\n\n" + "=" * 80)
print("⚙️ 配置示例")
print("=" * 80)

print("\n在 config.yaml 中配置执行器:")
print("""
code_executor:
  type: restricted_local          # 可选: local, restricted_local
  allowed_packages:                # RestrictedLocalExecutor 允许的包
    - pandas
    - numpy
    - matplotlib
    - plotly
    - seaborn
  timeout: 30                      # 执行超时时间（秒）
""")

### 5.6 实战案例：构建临床数据分析工作流

结合 SDTM 数据，演示如何在实际应用中使用执行器。

In [ ]:
print("=" * 80)
print("🏥 实战案例：临床数据分析工作流")
print("=" * 80)

# 场景：模拟 Agent 收到用户请求后，动态生成并执行分析代码

print("\n📋 场景描述:")
print("   用户问题: '请分析治疗组和安慰剂组的不良事件发生情况，")
print("              包括总体发生率、严重AE比例，并生成对比表格'")

print("\n" + "=" * 80)
print("步骤 1: 准备数据环境")
print("=" * 80)

# 加载数据
dm_df = pd.read_csv('data/sdtm/dm.csv')
ae_df = pd.read_csv('data/sdtm/ae.csv')

# 创建执行器并传递数据
executor = RestrictedLocalExecutor()
executor.set_variable('dm', dm_df)
executor.set_variable('ae', ae_df)
executor.set_variable('pd', pd)  # 传递 pandas 模块

print("✅ 数据加载完成")
print(f"   DM: {len(dm_df)} 条记录")
print(f"   AE: {len(ae_df)} 条记录")

print("\n" + "=" * 80)
print("步骤 2: 执行动态生成的分析代码")
print("=" * 80)

# 模拟 LLM 生成的分析代码
analysis_code = """
# 合并 DM 和 AE 数据
merged = dm.merge(ae, on='USUBJID', how='left')

# 计算各治疗组的 AE 统计
stats = []

for arm in dm['ARM'].unique():
    arm_subjects = dm[dm['ARM'] == arm]
    arm_ae = merged[merged['ARM'] == arm]
    
    # 有 AE 的受试者
    subjects_with_ae = arm_ae[arm_ae['AETERM'].notna()]['USUBJID'].nunique()
    ae_rate = subjects_with_ae / len(arm_subjects) * 100
    
    # 严重 AE
    sae_subjects = arm_ae[arm_ae['AESER'] == 'Y']['USUBJID'].nunique()
    sae_rate = sae_subjects / len(arm_subjects) * 100
    
    # AE 总数
    total_ae = arm_ae['AETERM'].notna().sum()
    
    stats.append({
        '治疗组': arm,
        '受试者数': len(arm_subjects),
        '发生AE人数': subjects_with_ae,
        'AE发生率(%)': f'{ae_rate:.1f}',
        'SAE人数': sae_subjects,
        'SAE发生率(%)': f'{sae_rate:.1f}',
        'AE总数': total_ae,
        '人均AE数': f'{total_ae/len(arm_subjects):.2f}'
    })

result_df = pd.DataFrame(stats)
print("\\n不良事件发生情况对比表:")
print("=" * 80)
print(result_df.to_string(index=False))

# 额外分析
print("\\n\\n关键发现:")
print("-" * 80)
trt_ae_rate = float(result_df[result_df['治疗组'] == 'Treatment']['AE发生率(%)'].values[0])
plb_ae_rate = float(result_df[result_df['治疗组'] == 'Placebo']['AE发生率(%)'].values[0])
print(f"• 治疗组 AE 发生率 ({trt_ae_rate:.1f}%) vs 安慰剂组 ({plb_ae_rate:.1f}%)")

if trt_ae_rate > plb_ae_rate:
    diff = trt_ae_rate - plb_ae_rate
    print(f"• 治疗组 AE 发生率高 {diff:.1f} 个百分点")
else:
    diff = plb_ae_rate - trt_ae_rate
    print(f"• 安慰剂组 AE 发生率高 {diff:.1f} 个百分点")
"""

print("🤖 执行 LLM 生成的代码...")
success, output = executor.run_code(analysis_code)

if success:
    print("✅ 执行成功\n")
    print(output)
else:
    print(f"❌ 执行失败: {output}")

print("\n" + "=" * 80)
print("步骤 3: 生成可视化代码")
print("=" * 80)

visualization_code = """
# 生成对比数据
ae_comparison = {
    '治疗组': [],
    '安慰剂组': []
}

trt_subjects = dm[dm['ARMCD'] == 'TRT']['USUBJID'].unique()
plb_subjects = dm[dm['ARMCD'] == 'PLB']['USUBJID'].unique()

ae_comparison['治疗组'].append(len([s for s in trt_subjects if s in ae['USUBJID'].values]))
ae_comparison['安慰剂组'].append(len([s for s in plb_subjects if s in ae['USUBJID'].values]))

print("\\n可视化数据准备完成:")
print(f"治疗组发生AE人数: {ae_comparison['治疗组'][0]}")
print(f"安慰剂组发生AE人数: {ae_comparison['安慰剂组'][0]}")
"""

success, output = executor.run_code(visualization_code)
print("✅ 可视化数据生成\n")
print(output)

print("\n" + "=" * 80)
print("💡 工作流总结")
print("=" * 80)
print("""
完整流程:
1. Agent 收到用户自然语言请求
2. LLM 生成相应的 Python 分析代码
3. 使用 RestrictedLocalExecutor 安全执行代码
4. 捕获输出结果并返回给用户
5. 可选：生成可视化图表

关键优势:
• 动态响应用户需求，无需预定义所有分析
• 安全执行 LLM 生成的代码
• 自动处理数据转换和计算
• 生成专业的分析报告
""")

### 5.7 与 Agent 的集成

执行器如何与 Agent 系统集成，实现完整的自动化分析流程。

In [ ]:
print("=" * 80)
print("🤖 代码执行器与 Agent 的集成")
print("=" * 80)

print("\n📐 架构图:")
print("""
┌─────────────────────────────────────────────────────────────┐
│                      用户输入                                │
│              "分析治疗组的不良事件"                           │
└─────────────────────┬───────────────────────────────────────┘
                      │
                      ▼
┌─────────────────────────────────────────────────────────────┐
│                   LangGraph Agent                            │
│  • 理解用户意图                                              │
│  • 决策使用哪些工具                                          │
│  • 编排执行流程                                              │
└─────────────────────┬───────────────────────────────────────┘
                      │
        ┌─────────────┼─────────────┐
        │             │             │
        ▼             ▼             ▼
  ┌──────────┐  ┌──────────┐  ┌──────────┐
  │ Text2SQL │  │ Run Code │  │  Search  │
  │   Tool   │  │   Tool   │  │   Tool   │
  └────┬─────┘  └────┬─────┘  └──────────┘
       │             │
       │             ▼
       │      ┌─────────────────┐
       │      │ Code Executor   │ ← 我们正在学习这部分
       │      │ • LocalExecutor │
       │      │ • RestrictedExec│
       │      └────────┬────────┘
       │               │
       ▼               ▼
  ┌─────────────────────────┐
  │    执行 SQL/Python      │
  │    返回结果给 Agent     │
  └──────────┬──────────────┘
             │
             ▼
  ┌─────────────────────────┐
  │    格式化输出给用户     │
  └─────────────────────────┘
""")

print("\n" + "=" * 80)
print("🔧 在 Agent 中使用执行器")
print("=" * 80)

print("\n1️⃣ 在 tool/run_python_code.py 中的实际使用:")
print("-" * 80)
print("""
from openchatbi.code.executor_base import ExecutorBase
from langchain_core.tools import StructuredTool

class RunPythonCodeTool:
    def __init__(self, executor: ExecutorBase):
        self.executor = executor
    
    def run(self, code: str) -> str:
        '''执行 Python 代码'''
        # 使用配置的执行器（可能是 Local 或 Restricted）
        success, output = self.executor.run_code(code)
        
        if success:
            return f"执行成功:\\n{output}"
        else:
            return f"执行失败:\\n{output}"
    
    def as_langchain_tool(self) -> StructuredTool:
        return StructuredTool.from_function(
            func=self.run,
            name="run_python_code",
            description="执行 Python 代码进行数据分析"
        )
""")

print("\n2️⃣ 在配置中选择执行器:")
print("-" * 80)
print("""
# openchatbi/config.yaml
code_executor:
  type: restricted_local  # 或 local
  
# 在应用启动时创建执行器
from openchatbi.config_loader import ConfigLoader

config = ConfigLoader.load_config()
if config.code_executor.type == 'local':
    executor = LocalExecutor()
elif config.code_executor.type == 'restricted_local':
    executor = RestrictedLocalExecutor()
""")

print("\n3️⃣ Agent 执行流程示例:")
print("-" * 80)
print("""
用户: "计算各治疗组的平均年龄"
  ↓
Agent: [分析] 需要查询 DM 表并计算
  ↓
Agent: [决策] 使用 run_python_code 工具
  ↓
Agent: [生成代码]
  code = '''
  avg_age = dm.groupby('ARM')['AGE'].mean()
  print(avg_age)
  '''
  ↓
Tool: run_python_code(code)
  ↓
Executor: RestrictedLocalExecutor.run_code(code)
  ↓
[执行成功] 返回结果
  ↓
Agent: [格式化] 生成友好的回复
  ↓
返回给用户: "各治疗组的平均年龄如下：
             Treatment: 45.2岁
             Placebo: 46.8岁"
""")

print("\n" + "=" * 80)
print("⚙️ 执行器的生命周期")
print("=" * 80)
print("""
1. 应用启动时:
   • 根据配置创建执行器实例
   • 初始化变量环境（可选）

2. 对话会话中:
   • 通过 set_variable() 传递 DataFrame 等对象
   • 多次调用 run_code() 执行不同代码
   • 变量在会话中保持（可以累积）

3. 会话结束:
   • 清理执行器资源
   • 释放内存

示例 - 会话中的变量保持:
executor = RestrictedLocalExecutor()

# 第一次执行：加载数据
executor.run_code("import pandas as pd")
executor.set_variable('dm', dm_df)

# 第二次执行：使用之前的数据
executor.run_code("print(f'受试者数: {len(dm)}')")

# 第三次执行：继续使用
executor.run_code("print(dm['AGE'].mean())")
""")

### 5.8 实战练习

通过练习加深对代码执行器的理解。

In [ ]:
print("=" * 80)
print("🎓 代码执行器实战练习")
print("=" * 80)

print("\n练习 1: 创建自定义执行器")
print("-" * 80)
print("""
任务: 创建一个带日志记录的执行器

要求:
1. 继承 RestrictedLocalExecutor
2. 在 run_code() 前后记录时间戳
3. 记录执行的代码和结果
4. 统计执行次数

提示代码框架:
```python
from openchatbi.code.restricted_local_executor import RestrictedLocalExecutor
import time
from datetime import datetime

class LoggingExecutor(RestrictedLocalExecutor):
    def __init__(self):
        super().__init__()
        self.execution_log = []
        self.execution_count = 0
    
    def run_code(self, code: str) -> (bool, str):
        start_time = time.time()
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # 执行代码
        success, output = super().run_code(code)
        
        # 记录日志
        elapsed = time.time() - start_time
        self.execution_log.append({
            'timestamp': timestamp,
            'code': code[:50] + '...' if len(code) > 50 else code,
            'success': success,
            'elapsed': f'{elapsed:.3f}s'
        })
        self.execution_count += 1
        
        return success, output
    
    def get_stats(self):
        return {
            'total_executions': self.execution_count,
            'recent_logs': self.execution_log[-5:]
        }
```
""")

print("\n练习 2: 实现执行超时控制")
print("-" * 80)
print("""
任务: 为执行器添加超时机制，防止代码执行时间过长

要求:
1. 使用 signal 模块（Linux）或 threading（Windows）
2. 超时后终止执行并返回错误
3. 设置默认超时为 30 秒

提示:
```python
import signal

def timeout_handler(signum, frame):
    raise TimeoutError("代码执行超时")

class TimeoutExecutor(LocalExecutor):
    def __init__(self, timeout=30):
        super().__init__()
        self.timeout = timeout
    
    def run_code(self, code: str) -> (bool, str):
        # 设置超时信号
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(self.timeout)
        
        try:
            result = super().run_code(code)
            signal.alarm(0)  # 取消超时
            return result
        except TimeoutError as e:
            return False, f"执行超时（{self.timeout}秒）"
```
""")

print("\n练习 3: 构建分析报告生成器")
print("-" * 80)
print("""
任务: 使用执行器自动生成临床研究报告

要求:
1. 读取 DM, AE, VS 三个域的数据
2. 生成包含以下内容的报告:
   • 受试者概况（人数、年龄、性别分布）
   • 不良事件统计（发生率、严重程度）
   • 生命体征趋势（血压、脉搏变化）
3. 输出格式化的文本报告

提示代码:
```python
executor = RestrictedLocalExecutor()
executor.set_variable('dm', dm_df)
executor.set_variable('ae', ae_df)
executor.set_variable('vs', vs_df)

report_code = '''
print("=" * 60)
print("临床研究安全性报告")
print("=" * 60)

# 第一部分：受试者概况
print("\\n一、受试者概况")
print(f"总人数: {len(dm)}")
# ... 继续添加更多分析

# 第二部分：不良事件
print("\\n二、不良事件分析")
# ...

# 第三部分：生命体征
print("\\n三、生命体征监测")
# ...
'''

success, report = executor.run_code(report_code)
print(report)
```
""")

print("\n练习 4: 实现变量持久化")
print("-" * 80)
print("""
任务: 在多次执行之间保持变量状态

要求:
1. 执行器可以保存变量到文件
2. 下次启动时可以恢复变量
3. 使用 pickle 序列化

提示:
```python
import pickle
from pathlib import Path

class PersistentExecutor(RestrictedLocalExecutor):
    def __init__(self, state_file='executor_state.pkl'):
        super().__init__()
        self.state_file = Path(state_file)
        self.load_state()
    
    def save_state(self):
        with open(self.state_file, 'wb') as f:
            pickle.dump(self._variable, f)
    
    def load_state(self):
        if self.state_file.exists():
            with open(self.state_file, 'rb') as f:
                self._variable = pickle.load(f)
```
""")

print("\n练习 5: 安全性测试")
print("-" * 80)
print("""
任务: 测试 RestrictedLocalExecutor 的安全限制

要求:
测试以下恶意代码是否被阻止:
1. import os; os.system('rm -rf /')
2. open('/etc/passwd', 'r').read()
3. import subprocess; subprocess.run(['ls'])
4. eval('__import__("os").system("echo hacked")')

验证:
```python
executor = RestrictedLocalExecutor()

dangerous_codes = [
    "import os; os.system('ls')",
    "open('test.txt', 'w').write('test')",
    "__import__('subprocess').run(['echo', 'test'])"
]

for code in dangerous_codes:
    success, output = executor.run_code(code)
    assert not success, f"危险代码未被阻止: {code}"
    print(f"✅ 已阻止: {code[:40]}...")
```
""")

print("\n\n" + "=" * 80)
print("💡 学习要点总结")
print("=" * 80)
print("""
通过本节学习，你应该掌握:

1. ExecutorBase 的接口设计
   • run_code() - 执行代码
   • set_variable() - 传递数据

2. LocalExecutor vs RestrictedLocalExecutor
   • 功能完整性 vs 安全性的权衡
   • 适用场景的选择

3. 与 Agent 系统的集成
   • 作为 Tool 的一部分
   • 动态执行 LLM 生成的代码

4. 实际应用
   • 临床数据分析
   • 报告自动生成
   • 安全性考虑

5. 最佳实践
   • 生产环境使用 RestrictedLocalExecutor
   • 合理设置超时
   • 记录执行日志
   • 处理异常情况
""")

### 5.9 代码执行器模块总结

完整回顾 `openchatbi/code/` 模块的功能和使用。

In [ ]:
print("=" * 80)
print("📚 openchatbi/code/ 模块总结")
print("=" * 80)

print("""
┌────────────────────────────────────────────────────────────────┐
│                    代码执行器模块架构                           │
└────────────────────────────────────────────────────────────────┘

                    ExecutorBase (抽象基类)
                           │
                ┌──────────┴──────────┐
                │                     │
        LocalExecutor        RestrictedLocalExecutor
      (无限制，快速)          (安全受限，推荐)
                │                     │
                └──────────┬──────────┘
                           │
                    Agent 工具集成
                (run_python_code tool)
                           │
                    用户自然语言查询
""")

print("\n" + "=" * 80)
print("🗂️ 文件功能速查")
print("=" * 80)

files_summary = """
1. executor_base.py
   • 抽象基类，定义执行器接口
   • 核心方法: run_code(), set_variable()
   • 作用: 统一不同执行器的接口

2. local_executor.py
   • 本地执行器，无安全限制
   • 优点: 功能完整，速度快
   • 缺点: 不安全，仅限开发使用
   • 使用场景: 开发调试、测试

3. restricted_local_executor.py
   • 受限本地执行器，使用 RestrictedPython
   • 优点: 安全可控，性能好
   • 限制: 禁止文件/网络/系统操作
   • 使用场景: 生产环境、执行 LLM 代码

4. docker_executor.py (已移除)
   • Docker 容器隔离执行
   • 已从项目中移除，不推荐使用
"""

print(files_summary)

print("\n" + "=" * 80)
print("🎯 关键使用场景")
print("=" * 80)

scenarios = """
场景 1: Agent 执行数据分析
├─ 用户问题: "计算各组的平均年龄"
├─ LLM 生成代码: dm.groupby('ARM')['AGE'].mean()
├─ 执行器: RestrictedLocalExecutor
└─ 返回结果: Treatment: 45.2, Placebo: 46.8

场景 2: 动态生成可视化
├─ 用户问题: "绘制 AE 发生率对比图"
├─ LLM 生成: plotly 绘图代码
├─ 执行器: 执行并返回图表数据
└─ 前端渲染: Streamlit 显示图表

场景 3: 自定义计算逻辑
├─ 用户问题: "计算某指标的 95% 置信区间"
├─ LLM 生成: 统计计算代码
├─ 执行器: 安全执行复杂计算
└─ 返回: 置信区间结果

场景 4: 多步骤数据处理
├─ 步骤 1: 数据清洗和转换
├─ 步骤 2: 统计分析
├─ 步骤 3: 生成报告
└─ 变量在执行器中持久化
"""

print(scenarios)

print("\n" + "=" * 80)
print("⚡ 性能与安全对比")
print("=" * 80)

print("""
┌─────────────────┬──────────────┬──────────────────┬─────────────┐
│   执行器类型    │  执行速度    │   安全等级       │  推荐指数   │
├─────────────────┼──────────────┼──────────────────┼─────────────┤
│ LocalExecutor   │ ⚡⚡⚡ 最快   │ ⚠️  无限制       │ ⭐⭐ 仅开发  │
│ RestrictedLocal │ ⚡⚡ 快       │ ✅ 受限安全      │ ⭐⭐⭐⭐⭐    │
│ DockerExecutor  │ 🐌 慢         │ ✅✅ 完全隔离    │ ⭐ 已移除    │
└─────────────────┴──────────────┴──────────────────┴─────────────┘
""")

print("\n" + "=" * 80)
print("🔑 核心 API 使用示例")
print("=" * 80)

print("""
# 1. 创建执行器
from openchatbi.code.restricted_local_executor import RestrictedLocalExecutor
executor = RestrictedLocalExecutor()

# 2. 传递数据
import pandas as pd
dm = pd.read_csv('data/sdtm/dm.csv')
executor.set_variable('dm', dm)
executor.set_variable('pd', pd)

# 3. 执行代码
code = "print(f'受试者数: {len(dm)}')"
success, output = executor.run_code(code)

# 4. 处理结果
if success:
    print(f"执行成功: {output}")
else:
    print(f"执行失败: {output}")
""")

print("\n" + "=" * 80)
print("✅ 学习检查清单")
print("=" * 80)

checklist = """
□ 理解 ExecutorBase 的抽象接口设计
□ 能区分 Local 和 Restricted 执行器的差异
□ 知道何时使用哪种执行器
□ 掌握 set_variable() 传递数据的方法
□ 理解执行器与 Agent 的集成方式
□ 了解 RestrictedPython 的安全限制
□ 能处理代码执行的成功和失败情况
□ 知道如何在临床数据分析中应用
□ 理解变量在会话中的持久化
□ 能编写安全的代码执行逻辑
"""

print(checklist)

print("\n" + "=" * 80)
print("🚀 下一步学习建议")
print("=" * 80)

print("""
1. 深入学习 RestrictedPython
   • 了解更多安全限制细节
   • 学习如何自定义安全策略

2. 探索 tool/run_python_code.py
   • 查看如何将执行器封装为 LangChain Tool
   • 理解 Agent 调用执行器的完整流程

3. 实践完整的分析流程
   • 从用户问题到代码生成
   • 从代码执行到结果展示

4. 优化性能
   • 添加缓存机制
   • 实现代码片段预编译

5. 增强安全性
   • 添加更详细的日志
   • 实现执行超时控制
   • 监控资源使用
""")

print("\n✅ openchatbi/code/ 模块学习完成！")
print("=" * 80)

## 6. openchatbi/llm/ - 大语言模型 (LLM) 接口模块

**作用**: 提供统一的 LLM 访问接口，支持多种 LLM 提供商，并实现智能重试和工具调用管理。

### 6.1 模块架构概览

```
openchatbi/llm/
└── llm.py                    # LLM 统一接口和工具调用管理
```

**核心概念**:
- **get_default_llm()**: 获取默认的通用 LLM（用于 Agent 对话、推理等）
- **get_text2sql_llm()**: 获取专门用于 Text2SQL 的 LLM（可单独配置）
- **get_embedding_model()**: 获取 Embedding 模型（用于向量检索）
- **call_llm_chat_model_with_retry()**: 带重试机制的 LLM 调用函数

**临床应用场景**:
- 理解自然语言查询（"显示严重不良事件的受试者"）
- 生成 SQL 查询语句
- 分析数据并生成报告
- 智能问答和知识检索

### 6.2 LLM 配置详解

OpenChatBI 支持多种 LLM 提供商，通过配置文件统一管理。

**支持的 LLM 提供商**:
- **Anthropic**: Claude 系列（Claude 3.5 Sonnet, Claude 3 Opus 等）
- **OpenAI**: GPT-4, GPT-3.5-turbo 等
- **Azure OpenAI**: Azure 托管的 OpenAI 模型
- **其他**: 支持 LangChain 的任何 LLM

**配置结构**:
```yaml
default_llm:
  provider: anthropic          # 提供商
  model: claude-3-5-sonnet-20241022
  api_key: ${ANTHROPIC_API_KEY}
  temperature: 0               # 0=确定性，1=创造性
  max_tokens: 4096

text2sql_llm:                  # 可选：专用于 SQL 生成
  provider: anthropic
  model: claude-3-5-sonnet-20241022
  temperature: 0

embedding_model:               # 用于向量检索
  provider: openai
  model: text-embedding-3-small
```

In [ ]:
print("=" * 80)
print("📋 LLM 配置示例")
print("=" * 80)

print("\n1️⃣ 配置文件示例 (openchatbi/config.yaml):")
print("-" * 80)

config_example = """
# 方案 1: 使用 Anthropic Claude（推荐用于临床数据分析）
default_llm:
  provider: anthropic
  model: claude-3-5-sonnet-20241022
  api_key: ${ANTHROPIC_API_KEY}  # 从环境变量读取
  temperature: 0                  # 确定性输出
  max_tokens: 4096

# 方案 2: 使用 OpenAI GPT-4
default_llm:
  provider: openai
  model: gpt-4-turbo
  api_key: ${OPENAI_API_KEY}
  temperature: 0.3
  max_tokens: 8000

# 方案 3: 使用 Azure OpenAI
default_llm:
  provider: azure_openai
  model: gpt-4
  api_key: ${AZURE_OPENAI_API_KEY}
  azure_endpoint: https://your-resource.openai.azure.com/
  api_version: 2024-02-01
  deployment_name: gpt-4-deployment

# Embedding 模型（用于向量检索）
embedding_model:
  provider: openai
  model: text-embedding-3-small
  api_key: ${OPENAI_API_KEY}

# 可选：专用于 Text2SQL 的 LLM
text2sql_llm:
  provider: anthropic
  model: claude-3-5-sonnet-20241022
  temperature: 0      # SQL 生成需要高确定性
"""

print(config_example)

print("\n2️⃣ 环境变量设置:")
print("-" * 80)
print("""
# Windows PowerShell
$env:ANTHROPIC_API_KEY = "sk-ant-..."
$env:OPENAI_API_KEY = "sk-..."

# Linux/Mac
export ANTHROPIC_API_KEY="sk-ant-..."
export OPENAI_API_KEY="sk-..."
""")

print("\n3️⃣ 临床数据分析的 LLM 选择建议:")
print("-" * 80)
print("""
任务类型                    推荐模型                   Temperature    原因
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
自然语言理解              Claude 3.5 Sonnet          0.0           准确理解复杂查询
SQL 生成                  Claude 3.5 Sonnet          0.0           高准确性，减少错误
数据分析推理              Claude 3.5 Sonnet          0.1           平衡准确性与创造性
报告生成                  GPT-4 Turbo                0.3-0.5       更流畅的文本
向量检索                  text-embedding-3-small     N/A           成本效益好
多语言支持（中文）        Claude 3.5 Sonnet          0.0           中文理解能力强
""")

print("\n💡 最佳实践:")
print("   • default_llm 使用 Claude 3.5 Sonnet（综合性能最佳）")
print("   • text2sql_llm 单独配置，temperature=0（确保 SQL 准确）")
print("   • embedding_model 使用 OpenAI（成本低，效果好）")
print("   • 临床数据分析建议 temperature=0（确定性输出）")

### 6.3 get_default_llm() - 获取默认 LLM

`get_default_llm()` 返回配置的默认 LLM 实例，用于通用任务。

**用途**:
- Agent 对话和推理
- 自然语言理解
- 工具选择决策
- 通用问答

In [ ]:
from openchatbi.llm.llm import get_default_llm

print("=" * 80)
print("🤖 get_default_llm() - 获取默认 LLM")
print("=" * 80)

# 注意：这里需要先配置 LLM，否则会报错
# 在实际使用中，需要先运行 ConfigLoader.load_config()

print("\n1️⃣ 函数说明:")
print("-" * 80)
print("""
def get_default_llm() -> BaseChatModel:
    '''获取配置的默认 LLM 实例'''
    return config.get().default_llm

返回值: BaseChatModel（LangChain 的 LLM 基类）
配置来源: config.yaml 中的 default_llm 部分
""")

print("\n2️⃣ 使用场景示例:")
print("-" * 80)
print("""
# 场景 1: Agent 对话
llm = get_default_llm()
response = llm.invoke([
    {"role": "user", "content": "分析 DM 表中受试者的年龄分布"}
])

# 场景 2: 意图识别
llm = get_default_llm()
response = llm.invoke([
    {"role": "system", "content": "你是临床数据分析助手"},
    {"role": "user", "content": "有多少受试者出现了严重不良事件？"}
])

# 场景 3: 数据解读
llm = get_default_llm()
response = llm.invoke([
    {"role": "user", "content": f"解读以下统计结果：{statistics_data}"}
])
""")

print("\n3️⃣ 实际示例：使用 LLM 理解临床问题")
print("-" * 80)

# 模拟 LLM 调用（实际需要配置）
example_query = "请找出年龄超过60岁且出现了严重不良事件的受试者"

print(f"用户问题: {example_query}")
print("\nLLM 分析:")
print("""
{
  "意图": "数据查询",
  "涉及表": ["dm", "ae"],
  "查询条件": [
    {"表": "dm", "列": "AGE", "条件": "> 60"},
    {"表": "ae", "列": "AESER", "条件": "= 'Y'"}
  ],
  "关联键": "USUBJID",
  "输出字段": ["USUBJID", "AGE", "AETERM", "AESEV"]
}
""")

print("\n4️⃣ LLM 在 OpenChatBI 中的角色:")
print("-" * 80)
print("""
┌─────────────────────────────────────────────────┐
│              用户自然语言查询                    │
│    "显示治疗组中有严重不良事件的受试者"          │
└────────────────┬────────────────────────────────┘
                 │
                 ▼
        ┌────────────────┐
        │  get_default_  │ ← 获取 LLM 实例
        │     llm()      │
        └────────┬───────┘
                 │
                 ▼
        ┌─────────────────────────────────┐
        │     LLM 理解和推理              │
        │  • 识别意图（查询 vs 分析）      │
        │  • 提取关键信息（表、列、条件）   │
        │  • 决定使用哪些工具              │
        │  • 规划执行步骤                 │
        └────────┬────────────────────────┘
                 │
        ┌────────┴─────────┬──────────┐
        │                  │          │
        ▼                  ▼          ▼
   Text2SQL Tool    Run Code    Search KB
""")

print("\n💡 关键点:")
print("   • get_default_llm() 返回已配置的 LLM 实例")
print("   • 用于 Agent 的所有推理和决策")
print("   • 支持流式输出（用于实时响应）")
print("   • 自动处理 API 密钥和连接")

### 6.4 get_text2sql_llm() - 获取 Text2SQL 专用 LLM

`get_text2sql_llm()` 返回专门用于 SQL 生成的 LLM，如果未单独配置则返回默认 LLM。

**为什么需要单独配置？**
- SQL 生成需要**极高的准确性**（temperature=0）
- 可能需要不同的模型（如针对代码生成优化的模型）
- 可以控制成本（SQL 生成频繁，可用较便宜的模型）

In [ ]:
from openchatbi.llm.llm import get_text2sql_llm

print("=" * 80)
print("📝 get_text2sql_llm() - 获取 Text2SQL 专用 LLM")
print("=" * 80)

print("\n1️⃣ 函数说明:")
print("-" * 80)
print("""
def get_text2sql_llm() -> BaseChatModel:
    '''获取 Text2SQL 专用 LLM，如果未配置则返回 default_llm'''
    return config.get().text2sql_llm or get_default_llm()

返回值: BaseChatModel
逻辑: 优先返回 text2sql_llm，否则回退到 default_llm
""")

print("\n2️⃣ 配置示例:")
print("-" * 80)
print("""
# 方案 A: 使用相同的 LLM（简单）
default_llm:
  provider: anthropic
  model: claude-3-5-sonnet-20241022
  temperature: 0

# text2sql_llm 不配置，自动使用 default_llm

# 方案 B: 使用不同的 LLM（优化）
default_llm:
  provider: anthropic
  model: claude-3-5-sonnet-20241022
  temperature: 0.1              # 对话可以有轻微创造性

text2sql_llm:
  provider: anthropic
  model: claude-3-5-sonnet-20241022
  temperature: 0                # SQL 生成必须确定
  max_tokens: 2048              # SQL 通常较短

# 方案 C: 使用更便宜的模型（成本优化）
default_llm:
  provider: anthropic
  model: claude-3-5-sonnet-20241022

text2sql_llm:
  provider: openai
  model: gpt-3.5-turbo          # 更便宜，SQL 生成够用
  temperature: 0
""")

print("\n3️⃣ Text2SQL 工作流程:")
print("-" * 80)
print("""
用户问题: "有多少受试者出现了严重不良事件？"
    ↓
┌─────────────────────────────────────────────────┐
│ 1. Schema Linking (使用 default_llm)            │
│    识别相关表: ae (不良事件表)                   │
│    识别相关列: AESER, USUBJID                   │
└────────────────┬────────────────────────────────┘
                 │
                 ▼
┌─────────────────────────────────────────────────┐
│ 2. SQL Generation (使用 text2sql_llm)          │
│    llm = get_text2sql_llm()                     │
│    生成 SQL:                                    │
│    SELECT COUNT(DISTINCT USUBJID)               │
│    FROM ae                                      │
│    WHERE AESER = 'Y'                            │
└────────────────┬────────────────────────────────┘
                 │
                 ▼
┌─────────────────────────────────────────────────┐
│ 3. 执行 SQL 并返回结果                          │
└─────────────────────────────────────────────────┘
""")

print("\n4️⃣ 临床场景示例:")
print("-" * 80)

clinical_queries = [
    {
        "query": "显示年龄大于60岁的受试者",
        "sql": "SELECT * FROM dm WHERE AGE > 60"
    },
    {
        "query": "统计各治疗组的不良事件发生率",
        "sql": """
SELECT 
  dm.ARM,
  COUNT(DISTINCT ae.USUBJID) * 100.0 / COUNT(DISTINCT dm.USUBJID) as ae_rate
FROM dm
LEFT JOIN ae ON dm.USUBJID = ae.USUBJID
GROUP BY dm.ARM
"""
    },
    {
        "query": "找出有药物相关不良事件的受试者",
        "sql": """
SELECT DISTINCT ae.USUBJID, dm.AGE, dm.SEX, ae.AETERM
FROM ae
JOIN dm ON ae.USUBJID = dm.USUBJID
WHERE ae.AEREL IN ('POSSIBLY RELATED', 'RELATED')
"""
    }
]

for i, item in enumerate(clinical_queries, 1):
    print(f"\n示例 {i}:")
    print(f"问题: {item['query']}")
    print(f"生成的 SQL:")
    for line in item['sql'].strip().split('\n'):
        print(f"  {line}")

print("\n\n5️⃣ Temperature 设置的重要性:")
print("-" * 80)
print("""
Temperature    SQL 输出                        适用场景
─────────────────────────────────────────────────────────────
0.0           SELECT * FROM dm WHERE AGE > 60  ✅ 生产环境（推荐）
              （每次完全相同）                  • 确定性输出
                                                • SQL 语法正确
                                                • 可重复验证

0.3-0.5       SELECT * FROM dm WHERE AGE > 60  ⚠️ 探索阶段
              SELECT AGE, SEX FROM dm          • 可能产生变化
              WHERE AGE >= 60                  • 不够稳定
              （可能有细微差异）                • 不推荐用于 SQL

1.0           SELECT ... WHER AGE > 60         ❌ 不适用
              （可能出现语法错误）              • 创造性太高
                                                • 容易出错
""")

print("\n💡 最佳实践:")
print("   • text2sql_llm 必须设置 temperature=0")
print("   • 使用 Claude 3.5 Sonnet（SQL 生成准确性高）")
print("   • 设置较小的 max_tokens（SQL 通常不长）")
print("   • 如果预算紧张，可以用 GPT-3.5-turbo")

### 6.5 get_embedding_model() - 获取 Embedding 模型

`get_embedding_model()` 返回用于文本向量化的 Embedding 模型，用于语义检索。

**用途**:
- 向量检索（找到语义相似的表和列）
- 知识库搜索
- 文档相似度计算
- 增强 BM25 检索效果

In [ ]:
from openchatbi.llm.llm import get_embedding_model

print("=" * 80)
print("🔍 get_embedding_model() - 获取 Embedding 模型")
print("=" * 80)

print("\n1️⃣ 函数说明:")
print("-" * 80)
print("""
def get_embedding_model() -> BaseModel:
    '''获取配置的 Embedding 模型'''
    return config.get().embedding_model

返回值: Embedding 模型实例（如 OpenAIEmbeddings）
用途: 将文本转换为向量（用于语义检索）
""")

print("\n2️⃣ 配置示例:")
print("-" * 80)
print("""
# 推荐：OpenAI Embedding（性能好，成本低）
embedding_model:
  provider: openai
  model: text-embedding-3-small
  api_key: ${OPENAI_API_KEY}
  dimensions: 1536              # 可选，默认 1536

# 备选：更高质量的模型
embedding_model:
  provider: openai
  model: text-embedding-3-large
  dimensions: 3072

# 备选：Azure OpenAI
embedding_model:
  provider: azure_openai
  model: text-embedding-3-small
  azure_endpoint: https://your-resource.openai.azure.com/
  api_key: ${AZURE_OPENAI_API_KEY}
""")

print("\n3️⃣ Embedding 工作原理:")
print("-" * 80)
print("""
文本                                    向量（1536 维）
───────────────────────────────────────────────────────────────
"受试者的年龄"           →    [0.123, -0.456, 0.789, ...]
"不良事件的严重程度"     →    [0.234, -0.567, 0.890, ...]
"生命体征测量值"         →    [0.345, -0.678, 0.901, ...]

相似度计算（余弦相似度）:
• "年龄" vs "年龄分布"     →  0.92（高度相似）
• "年龄" vs "不良事件"     →  0.35（不相似）
• "严重程度" vs "严重性"   →  0.88（相似）
""")

print("\n4️⃣ 在 Schema Retrieval 中的应用:")
print("-" * 80)
print("""
场景：用户问题 "显示高血压患者"

步骤 1: 将问题向量化
embedding = get_embedding_model()
query_vector = embedding.embed_query("高血压患者")
# → [0.123, -0.456, 0.789, ...]

步骤 2: 将所有列的描述向量化（预先计算）
columns = [
  {"name": "SYSBP", "desc": "收缩压测量值"},
  {"name": "DIABP", "desc": "舒张压测量值"},
  {"name": "AETERM", "desc": "不良事件术语"}
]

步骤 3: 计算相似度
相似度分数:
• SYSBP (收缩压) → 0.85  ✅ 高度相关
• DIABP (舒张压) → 0.78  ✅ 相关
• AETERM (不良事件) → 0.32  ❌ 不相关

步骤 4: 返回最相关的列
推荐使用: SYSBP, DIABP（VS 表）
""")

print("\n5️⃣ BM25 vs Embedding 对比:")
print("-" * 80)

comparison_data = """
特性              BM25（关键词）          Embedding（语义）
───────────────────────────────────────────────────────────────
查询方式          关键词精确匹配          语义相似度
"年龄" → "AGE"    ✅ 能匹配              ❌ 需要中英文映射
"高血压" → "血压"  ❌ 无法匹配            ✅ 能理解语义
"严重" → "SEVERE"  ❌ 语言不同            ✅ 跨语言理解
处理速度          ⚡ 快                  🐌 较慢（需调用 API）
成本              💰 免费                💰💰 需付费
准确性            ⭐⭐⭐                ⭐⭐⭐⭐⭐

推荐策略: 混合检索（BM25 + Embedding）
• BM25 快速筛选候选项
• Embedding 精确排序
• 结合两者的优势
"""

print(comparison_data)

print("\n6️⃣ 临床应用示例:")
print("-" * 80)

clinical_examples = [
    {
        "query": "血压异常的患者",
        "bm25_result": "['SYSBP', 'DIABP'] (关键词: 血压)",
        "embedding_result": "['SYSBP', 'DIABP', 'PULSE'] (语义: 生命体征相关)"
    },
    {
        "query": "药物副作用",
        "bm25_result": "[] (无'副作用'关键词)",
        "embedding_result": "['AETERM', 'AESEV', 'AEREL'] (理解为不良事件)"
    },
    {
        "query": "患者基本信息",
        "bm25_result": "[] (无精确匹配)",
        "embedding_result": "['AGE', 'SEX', 'RACE'] (DM 表人口统计学)"
    }
]

for i, ex in enumerate(clinical_examples, 1):
    print(f"\n示例 {i}: {ex['query']}")
    print(f"  BM25:      {ex['bm25_result']}")
    print(f"  Embedding: {ex['embedding_result']}")

print("\n\n💡 最佳实践:")
print("   • 使用 text-embedding-3-small（性价比最佳）")
print("   • 预先计算所有列的 embedding（避免重复计算）")
print("   • 结合 BM25 和 Embedding（混合检索）")
print("   • 缓存 embedding 结果（减少 API 调用）")

### 6.6 call_llm_chat_model_with_retry() - 智能重试机制

这是 OpenChatBI 的核心函数，提供带智能重试的 LLM 调用，确保系统的健壮性。

**核心功能**:
- 🔄 自动重试（最多 3 次）
- 🛡️ 验证工具调用的有效性
- 📊 防止多工具并行调用（可选）
- ⏱️ 记录执行时间
- 🐛 详细的错误日志

In [ ]:
from openchatbi.llm.llm import call_llm_chat_model_with_retry

print("=" * 80)
print("🔄 call_llm_chat_model_with_retry() - 智能重试机制")
print("=" * 80)

print("\n1️⃣ 函数签名:")
print("-" * 80)
print("""
def call_llm_chat_model_with_retry(
    chat_model: BaseChatModel,      # LLM 实例
    messages: list,                  # 对话消息列表
    streaming_tokens: bool = False,  # 是否流式输出
    bound_tools: list = None,        # 可用工具列表
    parallel_tool_call: bool = False # 是否允许并行调用多个工具
) -> AIMessage or None

返回值: AIMessage（LLM 的响应）或 None（所有重试失败）
""")

print("\n2️⃣ 重试场景:")
print("-" * 80)
print("""
场景 1: LLM API 调用失败
─────────────────────────────────────────────
• 网络错误
• API 限流（Rate Limit）
• 超时
→ 自动重试最多 3 次

场景 2: 工具调用无效
─────────────────────────────────────────────
LLM 调用了不存在的工具:
• 用户提问: "查询 DM 表"
• LLM 错误响应: 调用 non_existent_tool()
• 系统检测到错误
→ 提示 LLM 可用工具列表，重试

场景 3: 多工具并行调用（未启用）
─────────────────────────────────────────────
• parallel_tool_call=False 时
• LLM 同时调用多个工具
→ 提示只能调用一个，重试

场景 4: 成功调用
─────────────────────────────────────────────
• 工具调用有效
• 单个工具（或已启用并行）
→ 返回结果
""")

print("\n3️⃣ 工作流程示例:")
print("-" * 80)
print("""
尝试次数 1:
───────────────────────────────────────────────────────────
用户: "查询年龄大于60的受试者"
LLM 响应: 调用 query_database_tool(sql="SELECT * FROM dm WHERE AGE > 60")
验证: ✅ query_database_tool 在可用工具列表中
结果: 返回响应 ✅


尝试次数 1:
───────────────────────────────────────────────────────────
用户: "查询年龄大于60的受试者"
LLM 响应: 调用 execute_sql_query() + run_python_code()
验证: ❌ 同时调用了 2 个工具（parallel_tool_call=False）
系统提示: "You should only response with one tool call."

尝试次数 2:
───────────────────────────────────────────────────────────
LLM 响应: 调用 execute_sql_query(sql="...")
验证: ✅ 单个工具调用
结果: 返回响应 ✅


尝试次数 1:
───────────────────────────────────────────────────────────
用户: "查询数据"
LLM 响应: 调用 invalid_tool()
验证: ❌ invalid_tool 不在可用工具列表中
系统提示: "You should not use tool that does not exist: invalid_tool.
          Available tools are: [run_sql, run_code, search].
          Please choose a valid tool and try again."

尝试次数 2:
───────────────────────────────────────────────────────────
LLM 响应: 调用 run_sql(query="SELECT * FROM dm")
验证: ✅ run_sql 在可用工具列表中
结果: 返回响应 ✅
""")

print("\n4️⃣ 参数详解:")
print("-" * 80)
print("""
streaming_tokens: bool
────────────────────────────────────────────
• True:  流式输出（实时显示生成的文本）
• False: 等待完整响应
• 用途: UI 中的实时反馈

bound_tools: list
────────────────────────────────────────────
可用工具列表，可以是:
• 工具名称字符串: ["run_sql", "run_code"]
• StructuredTool 对象: [sql_tool, code_tool]
• 工具类: [AskHuman]

parallel_tool_call: bool
────────────────────────────────────────────
• False: 一次只能调用一个工具（默认，更安全）
• True:  允许同时调用多个工具（高级用法）
""")

print("\n5️⃣ 在 Agent 中的实际使用:")
print("-" * 80)
print("""
# 在 openchatbi/agent_graph.py 中

from openchatbi.llm.llm import get_default_llm, call_llm_chat_model_with_retry

def agent_node(state):
    '''Agent 节点：决策下一步操作'''
    
    # 获取 LLM
    llm = get_default_llm()
    
    # 绑定可用工具
    tools = [run_sql_tool, run_code_tool, search_tool, ask_human_tool]
    llm_with_tools = llm.bind_tools(tools)
    
    # 准备消息
    messages = state['messages']
    
    # 调用 LLM（带重试）
    response = call_llm_chat_model_with_retry(
        chat_model=llm_with_tools,
        messages=messages,
        streaming_tokens=True,         # 实时显示
        bound_tools=tools,             # 验证工具调用
        parallel_tool_call=False       # 禁止并行调用
    )
    
    if response is None:
        # 所有重试失败
        return {"error": "LLM call failed after 3 retries"}
    
    # 返回响应
    return {"messages": [response]}
""")

print("\n6️⃣ 临床应用场景:")
print("-" * 80)
print("""
场景: 复杂的多步骤查询
────────────────────────────────────────────
用户: "找出治疗组中有严重不良事件的受试者，并分析他们的年龄分布"

第 1 步: Agent 决策
  • LLM 分析: 需要查询 DM 和 AE 表
  • 工具选择: run_sql
  • 如果 LLM 错误选择了多个工具 → 重试

第 2 步: 执行 SQL
  • 获取数据

第 3 步: Agent 决策（数据分析）
  • LLM 分析: 需要计算年龄分布
  • 工具选择: run_code
  • 如果 LLM 选择了无效工具 → 重试

第 4 步: 生成结果
  • 返回分析报告
""")

print("\n7️⃣ 错误处理和日志:")
print("-" * 80)
print("""
自动记录的信息:
• 每次调用的时间戳
• 执行时间（秒）
• 重试次数和原因
• 错误详情（如果失败）

示例日志输出:
────────────────────────────────────────────
Call LLM chat model with retry 0 times.
LLM response after 2 seconds.

Call LLM chat model with retry 1 times.
Invalid tool non_existent_tool, retry 1 times.
LLM response after 3 seconds.
""")

print("\n💡 最佳实践:")
print("   • 始终使用此函数而非直接调用 llm.invoke()")
print("   • 设置合理的 bound_tools（防止无效调用）")
print("   • 生产环境禁用 parallel_tool_call（更稳定）")
print("   • 启用 streaming_tokens 提升用户体验")
print("   • 监控重试次数（频繁重试说明配置有问题）")

### 6.7 完整的临床数据分析示例

结合 SDTM 数据，演示 LLM 模块在实际应用中的完整工作流程。

In [ ]:
print("=" * 80)
print("🏥 完整的临床数据分析流程示例")
print("=" * 80)

print("\n场景: 用户通过自然语言查询临床数据")
print("问题: '分析治疗组和安慰剂组的不良事件发生情况'")
print("=" * 80)

print("\n【步骤 1】加载 SDTM 数据")
print("-" * 80)
import pandas as pd

dm = pd.read_csv('data/sdtm/dm.csv')
ae = pd.read_csv('data/sdtm/ae.csv')

print(f"✅ DM: {len(dm)} 条记录")
print(f"✅ AE: {len(ae)} 条记录")

print("\n【步骤 2】用户问题 → LLM 理解意图")
print("-" * 80)
print("""
用户输入: "分析治疗组和安慰剂组的不良事件发生情况"

LLM (get_default_llm) 分析:
┌────────────────────────────────────────────────┐
│ 意图识别:                                      │
│   • 任务类型: 数据分析 + 对比                  │
│   • 涉及表: dm (治疗组信息), ae (不良事件)     │
│   • 分组维度: 治疗组 vs 安慰剂组               │
│   • 分析指标: AE 发生率、人数、事件数          │
│                                                │
│ 决策:                                          │
│   • 第一步: 使用 run_sql 工具查询数据          │
│   • 第二步: 使用 run_code 工具进行统计分析     │
└────────────────────────────────────────────────┘
""")

print("\n【步骤 3】生成 SQL（使用 get_text2sql_llm）")
print("-" * 80)
print("""
Text2SQL LLM 生成查询:

-- 合并 DM 和 AE 表
SELECT 
    dm.ARM,
    dm.USUBJID,
    ae.AETERM,
    ae.AESER,
    ae.AESEV
FROM dm
LEFT JOIN ae ON dm.USUBJID = ae.USUBJID

这个 SQL 由 get_text2sql_llm() 生成，temperature=0 确保准确性
""")

print("\n【步骤 4】执行 SQL 并分析")
print("-" * 80)

# 模拟 SQL 执行结果
merged = dm.merge(ae, on='USUBJID', how='left')

# 统计分析
stats = []
for arm in ['Treatment', 'Placebo']:
    arm_dm = dm[dm['ARM'] == arm]
    arm_ae = merged[merged['ARM'] == arm]
    
    subjects_total = len(arm_dm)
    subjects_with_ae = arm_ae[arm_ae['AETERM'].notna()]['USUBJID'].nunique()
    ae_total = arm_ae['AETERM'].notna().sum()
    sae_count = (arm_ae['AESER'] == 'Y').sum()
    
    stats.append({
        '治疗组': arm,
        '受试者数': subjects_total,
        '发生AE人数': subjects_with_ae,
        'AE发生率': f"{subjects_with_ae/subjects_total*100:.1f}%",
        'AE总数': ae_total,
        'SAE数': sae_count
    })

result_df = pd.DataFrame(stats)
print("\n统计结果:")
print(result_df.to_string(index=False))

print("\n【步骤 5】LLM 生成分析报告")
print("-" * 80)
print("""
LLM (get_default_llm) 基于数据生成报告:

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
不良事件发生情况分析报告
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. 总体情况
   • 研究共入组 50 名受试者
   • 治疗组: 25 人
   • 安慰剂组: 25 人

2. 不良事件发生率
""")

trt_rate = float(result_df[result_df['治疗组'] == 'Treatment']['AE发生率'].values[0].rstrip('%'))
plb_rate = float(result_df[result_df['治疗组'] == 'Placebo']['AE发生率'].values[0].rstrip('%'))

print(f"   • 治疗组: {trt_rate:.1f}%")
print(f"   • 安慰剂组: {plb_rate:.1f}%")

if abs(trt_rate - plb_rate) < 10:
    print(f"   • 两组发生率相近，差异不大")
else:
    higher_group = "治疗组" if trt_rate > plb_rate else "安慰剂组"
    print(f"   • {higher_group}的发生率较高")

trt_sae = result_df[result_df['治疗组'] == 'Treatment']['SAE数'].values[0]
plb_sae = result_df[result_df['治疗组'] == 'Placebo']['SAE数'].values[0]

print(f"""
3. 严重不良事件（SAE）
   • 治疗组 SAE: {trt_sae} 例
   • 安慰剂组 SAE: {plb_sae} 例
   
4. 结论
   • 不良事件发生情况在两组间无显著差异
   • 建议持续监测严重不良事件
   • 需关注高发的不良事件类型
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
""")

print("\n【完整流程图】")
print("-" * 80)
print("""
┌─────────────────────────────────────────────────────┐
│              用户自然语言问题                        │
│    "分析治疗组和安慰剂组的不良事件发生情况"         │
└────────────────┬────────────────────────────────────┘
                 │
                 ▼
┌─────────────────────────────────────────────────────┐
│          get_default_llm() - 意图理解                │
│  • 识别任务类型: 数据分析                           │
│  • 识别涉及表: dm, ae                               │
│  • 决策工具: run_sql → run_code                     │
└────────────────┬────────────────────────────────────┘
                 │
                 ▼
┌─────────────────────────────────────────────────────┐
│        get_text2sql_llm() - 生成 SQL                │
│  SELECT dm.ARM, ae.AETERM, ae.AESER ...             │
│  FROM dm LEFT JOIN ae ON dm.USUBJID = ae.USUBJID    │
└────────────────┬────────────────────────────────────┘
                 │
                 ▼
┌─────────────────────────────────────────────────────┐
│              执行 SQL 查询数据                       │
│  返回: 合并后的 DM + AE 数据                        │
└────────────────┬────────────────────────────────────┘
                 │
                 ▼
┌─────────────────────────────────────────────────────┐
│       get_default_llm() - 生成分析代码               │
│  生成 Python 代码计算:                              │
│  • 各组受试者数、AE 发生率                          │
│  • SAE 数量、对比分析                               │
└────────────────┬────────────────────────────────────┘
                 │
                 ▼
┌─────────────────────────────────────────────────────┐
│            执行代码（RestrictedExecutor）            │
│  计算统计指标                                       │
└────────────────┬────────────────────────────────────┘
                 │
                 ▼
┌─────────────────────────────────────────────────────┐
│       get_default_llm() - 生成报告                  │
│  基于数据生成专业的临床报告                         │
└────────────────┬────────────────────────────────────┘
                 │
                 ▼
┌─────────────────────────────────────────────────────┐
│              返回给用户                              │
│  结构化报告 + 数据表格                              │
└─────────────────────────────────────────────────────┘

在整个流程中:
• call_llm_chat_model_with_retry() 确保每次 LLM 调用的可靠性
• get_embedding_model() 用于 schema retrieval（找到相关表和列）
• 三种 LLM 各司其职，协同完成复杂任务
""")

### 6.8 LLM 模块总结与最佳实践

In [ ]:
print("=" * 80)
print("📚 LLM 模块总结")
print("=" * 80)

print("""
┌──────────────────────────────────────────────────────────┐
│                   LLM 模块架构                           │
└──────────────────────────────────────────────────────────┘

              openchatbi/llm/llm.py
                      │
        ┌─────────────┼─────────────┐
        │             │             │
        ▼             ▼             ▼
get_default_llm  get_text2sql  get_embedding
                    _llm          _model
        │             │             │
        └─────────────┴─────────────┘
                      │
                      ▼
       call_llm_chat_model_with_retry
              (智能重试包装器)
""")

print("\n" + "=" * 80)
print("🗂️ 函数功能速查表")
print("=" * 80)

summary = """
函数名                          用途                        配置来源
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
get_default_llm()              通用 LLM                    config.default_llm
                               • Agent 对话推理
                               • 意图识别
                               • 报告生成

get_text2sql_llm()             SQL 生成专用                config.text2sql_llm
                               • 自然语言转 SQL             (或 default_llm)
                               • Schema linking
                               • 确定性输出 (temp=0)

get_embedding_model()          文本向量化                  config.embedding_model
                               • 语义检索
                               • 相似度计算
                               • 知识库搜索

call_llm_chat_model_with_      智能调用包装器              N/A (工具函数)
retry()                        • 自动重试 (3次)
                               • 验证工具调用
                               • 错误日志
"""

print(summary)

print("\n" + "=" * 80)
print("⚙️ 配置最佳实践")
print("=" * 80)

best_practices = """
1. 模型选择
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
任务                推荐模型                    Temperature
────────────────────────────────────────────────────────────
通用对话            Claude 3.5 Sonnet           0.1-0.3
SQL 生成            Claude 3.5 Sonnet           0.0 ⭐⭐⭐
报告生成            GPT-4 Turbo                 0.3-0.5
向量检索            text-embedding-3-small      N/A
中文理解            Claude 3.5 Sonnet           0.0-0.2

2. Temperature 设置原则
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
0.0     完全确定性    • SQL 生成 ✅
                      • 数据分析 ✅
                      • 需要可重复的结果

0.1-0.3 轻微创造性    • Agent 对话
                      • 意图理解
                      • 工具选择

0.5-0.7 中等创造性    • 报告撰写
                      • 内容生成
                      • 多样化表达

0.8-1.0 高度创造性    • 不推荐用于数据分析
                      • 可能产生不准确结果

3. 成本优化
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
• default_llm: Claude 3.5 Sonnet (综合性能最佳)
• text2sql_llm: 可使用 GPT-3.5-turbo (成本更低)
• embedding: text-embedding-3-small (性价比高)
• 缓存 embedding 结果 (减少重复调用)
• 使用 BM25 + Embedding 混合检索 (减少 API 调用)

4. 安全性
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
• API Key 使用环境变量 (不要硬编码)
• 启用 call_llm_chat_model_with_retry (自动重试)
• 设置 bound_tools (验证工具调用)
• 禁用 parallel_tool_call (生产环境)
• 记录和监控 LLM 调用 (发现异常)
"""

print(best_practices)

print("\n" + "=" * 80)
print("🎯 临床数据分析的 LLM 使用模式")
print("=" * 80)

patterns = """
模式 1: 单步查询
─────────────────────────────────────────────────────────
用户: "有多少受试者？"
流程: get_text2sql_llm() → 生成 SQL → 执行 → 返回结果
特点: 简单快速，适合明确的查询

模式 2: 多步分析
─────────────────────────────────────────────────────────
用户: "分析各组的不良事件情况"
流程: 
  1. get_default_llm() - 理解意图
  2. get_text2sql_llm() - 生成 SQL 查询数据
  3. get_default_llm() - 生成分析代码
  4. get_default_llm() - 生成报告
特点: 复杂任务，需要多次 LLM 调用

模式 3: 交互式对话
─────────────────────────────────────────────────────────
用户: "显示严重不良事件"
Agent: "已找到 5 例，需要详细信息吗？"
用户: "是的，显示详情"
流程: get_default_llm() 保持对话上下文
特点: 需要记忆和上下文管理

模式 4: 知识检索增强
─────────────────────────────────────────────────────────
用户: "什么是 CDISC SDTM？"
流程:
  1. get_embedding_model() - 向量化问题
  2. 检索知识库
  3. get_default_llm() - 生成回答（基于检索结果）
特点: RAG 模式，结合检索和生成
"""

print(patterns)

print("\n" + "=" * 80)
print("✅ 学习检查清单")
print("=" * 80)

checklist = """
□ 理解三个 LLM getter 函数的区别和用途
□ 知道何时使用 default_llm vs text2sql_llm
□ 掌握 temperature 参数对输出的影响
□ 理解 embedding_model 在检索中的作用
□ 会使用 call_llm_chat_model_with_retry
□ 了解重试机制的触发条件
□ 知道如何配置不同的 LLM 提供商
□ 能够根据任务选择合适的模型
□ 理解 LLM 在 Agent 中的角色
□ 掌握成本优化策略
"""

print(checklist)

print("\n" + "=" * 80)
print("🚀 下一步学习建议")
print("=" * 80)

next_steps = """
1. 实践 LLM 配置
   • 申请 Anthropic/OpenAI API Key
   • 配置 config.yaml
   • 测试不同的 temperature 设置

2. 探索 Prompt Engineering
   • 查看 openchatbi/prompts/ 目录
   • 学习如何优化提示词
   • 提高 SQL 生成准确性

3. 深入 Agent 工作流
   • 学习 openchatbi/agent_graph.py
   • 理解 LangGraph 的节点和边
   • 掌握工具调用机制

4. 监控和优化
   • 记录 LLM 调用日志
   • 分析成本和性能
   • 优化 prompt 和模型选择

5. 高级特性
   • 实现流式输出
   • 添加缓存机制
   • 自定义 LLM 包装器
"""

print(next_steps)

print("\n✅ openchatbi/llm/ 模块学习完成！")
print("=" * 80)

In [ ]:
# 创建临床数据监控配置
clinical_config = {
    "default_llm": {
        "provider": "anthropic",
        "model": "claude-3-5-sonnet-20241022",
        "api_key": "your-api-key-here",
        "temperature": 0,
        "max_tokens": 4096
    },
    
    "catalog": {
        "type": "file_system",
        "catalog_dir": "data/sdtm"
    },
    
    "local_datasets": {
        "enabled": True,
        "datasets": [
            {
                "name": "dm",
                "path": "data/sdtm/dm.csv",
                "description": "Demographics domain - 受试者基本信息（年龄、性别、入组日期等）",
                "file_type": "csv"
            },
            {
                "name": "ae",
                "path": "data/sdtm/ae.csv",
                "description": "Adverse Events domain - 不良事件（症状、严重程度、因果关系等）",
                "file_type": "csv"
            },
            {
                "name": "vs",
                "path": "data/sdtm/vs.csv",
                "description": "Vital Signs domain - 生命体征（血压、脉搏、体温、体重等）",
                "file_type": "csv"
            }
        ]
    },
    
    "code_executor": {
        "type": "restricted_local",
        "allowed_packages": ["pandas", "numpy", "matplotlib", "seaborn", "plotly"]
    }
}

# 保存配置
config_file = Path('data/clinical_config.yaml')
with open(config_file, 'w', encoding='utf-8') as f:
    yaml.dump(clinical_config, f, default_flow_style=False, allow_unicode=True)

print(f"✓ 配置文件已创建: {config_file}")
print("\n配置内容:")
print(yaml.dump(clinical_config, default_flow_style=False, allow_unicode=True))

### 4.2 创建表结构元数据

In [ ]:
# 创建表信息配置（用于 schema linking）
table_info = {
    "dm": {
        "display_name": "受试者人口统计学",
        "description": "包含受试者的基本信息，如年龄、性别、种族、入组日期、分组等",
        "columns": [
            {"name": "USUBJID", "display_name": "唯一受试者ID", "description": "研究中受试者的唯一标识符"},
            {"name": "AGE", "display_name": "年龄", "description": "受试者年龄"},
            {"name": "SEX", "display_name": "性别", "description": "M=男性, F=女性"},
            {"name": "RACE", "display_name": "种族", "description": "受试者种族"},
            {"name": "ARMCD", "display_name": "治疗组代码", "description": "TRT=治疗组, PLB=安慰剂组"},
            {"name": "RFSTDTC", "display_name": "首次用药日期", "description": "受试者首次接受研究药物的日期"}
        ]
    },
    "ae": {
        "display_name": "不良事件",
        "description": "记录研究期间发生的所有不良事件，包括症状、严重程度、因果关系判断等",
        "columns": [
            {"name": "USUBJID", "display_name": "唯一受试者ID", "description": "发生AE的受试者ID"},
            {"name": "AETERM", "display_name": "不良事件术语", "description": "不良事件的描述性名称"},
            {"name": "AESEV", "display_name": "严重程度", "description": "MILD=轻度, MODERATE=中度, SEVERE=重度"},
            {"name": "AESER", "display_name": "严重不良事件标志", "description": "Y=是严重AE, N=非严重AE"},
            {"name": "AEREL", "display_name": "因果关系", "description": "与研究药物的因果关系评估"},
            {"name": "AESTDTC", "display_name": "开始日期", "description": "不良事件开始日期"}
        ]
    },
    "vs": {
        "display_name": "生命体征",
        "description": "记录受试者的生命体征测量值，如血压、脉搏、体温、体重等",
        "columns": [
            {"name": "USUBJID", "display_name": "唯一受试者ID", "description": "测量的受试者ID"},
            {"name": "VSTESTCD", "display_name": "检查代码", "description": "SYSBP=收缩压, DIABP=舒张压, PULSE=脉搏, TEMP=体温"},
            {"name": "VSTEST", "display_name": "检查名称", "description": "生命体征检查的完整名称"},
            {"name": "VSORRES", "display_name": "原始结果", "description": "测量的原始数值"},
            {"name": "VSORRESU", "display_name": "单位", "description": "测量值的单位"},
            {"name": "VISIT", "display_name": "访视", "description": "测量时的访视名称"}
        ]
    }
}

table_info_file = Path('data/sdtm/table_info.yaml')
with open(table_info_file, 'w', encoding='utf-8') as f:
    yaml.dump(table_info, f, default_flow_style=False, allow_unicode=True)

print(f"✓ 表结构元数据已创建: {table_info_file}")

## 5. 测试本地数据集加载

In [ ]:
# 测试加载 SDTM 数据
from openchatbi.local_dataset_loader import LocalDatasetManager

# 创建数据集管理器
manager = LocalDatasetManager(clinical_config['local_datasets']['datasets'])

print("可用数据集:")
for name in manager.list_datasets():
    info = manager.get_dataset_info(name)
    print(f"\n{name}:")
    print(f"  描述: {info['description']}")
    print(f"  路径: {info['path']}")
    print(f"  类型: {info['file_type']}")

In [ ]:
# 加载并查看 DM 数据
dm = manager.load_dataset('dm')
print(f"DM 数据集: {len(dm)} 条记录")
print(f"\n列: {', '.join(dm.columns)}")
print(f"\n前5条:")
dm.head()

In [ ]:
# 加载并查看 AE 数据
ae = manager.load_dataset('ae')
print(f"AE 数据集: {len(ae)} 条记录")
print(f"\n严重 AE 统计:")
print(ae['AESER'].value_counts())
print(f"\n严重程度分布:")
print(ae['AESEV'].value_counts())

## 6. 常见临床监控查询示例

### 6.1 使用 pandas 进行数据分析

In [ ]:
# 示例1: 统计各组的受试者数量
print("各治疗组受试者数量:")
print(dm.groupby('ARM').size())

# 示例2: 年龄分布
print("\n年龄统计:")
print(dm['AGE'].describe())

# 示例3: 性别分布
print("\n性别分布:")
print(dm['SEX'].value_counts())

In [ ]:
# 示例4: 不良事件分析
print("最常见的不良事件 (Top 5):")
print(ae['AETERM'].value_counts().head())

print("\n严重不良事件数量:")
sae = ae[ae['AESER'] == 'Y']
print(f"SAE: {len(sae)} 例")
print(f"受试者数: {sae['USUBJID'].nunique()} 人")

print("\n与药物相关的 AE:")
related_ae = ae[ae['AEREL'].isin(['POSSIBLY RELATED', 'RELATED'])]
print(f"相关 AE: {len(related_ae)} 例")

In [ ]:
# 示例5: 生命体征趋势
vs = manager.load_dataset('vs')

# 计算收缩压的平均值（按访视）
sysbp = vs[vs['VSTESTCD'] == 'SYSBP']
print("收缩压平均值（按访视）:")
print(sysbp.groupby('VISIT')['VSORRES'].mean())

# 识别异常值（收缩压 > 140 或 < 90）
abnormal_bp = sysbp[(sysbp['VSORRES'] > 140) | (sysbp['VSORRES'] < 90)]
print(f"\n血压异常记录: {len(abnormal_bp)} 条")

### 6.2 数据可视化

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# 可视化1: 不良事件严重程度分布
ae_severity = ae.groupby(['AESEV', 'AESER']).size().reset_index(name='count')
fig = px.bar(ae_severity, x='AESEV', y='count', color='AESER',
             title='不良事件严重程度分布',
             labels={'AESEV': '严重程度', 'count': '数量', 'AESER': '是否严重AE'})
fig.show()

In [ ]:
# 可视化2: 各治疗组的 AE 发生率
dm_ae = dm.merge(ae[['USUBJID']], on='USUBJID', how='left')
dm_ae['has_ae'] = dm_ae['USUBJID'].duplicated(keep=False)
ae_rate = dm_ae.groupby('ARM')['has_ae'].mean() * 100

fig = px.bar(x=ae_rate.index, y=ae_rate.values,
             title='各治疗组 AE 发生率',
             labels={'x': '治疗组', 'y': 'AE 发生率 (%)'})
fig.show()

In [ ]:
# 可视化3: 生命体征趋势
sysbp_trend = sysbp.groupby('VISIT')['VSORRES'].agg(['mean', 'std']).reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(x=sysbp_trend['VISIT'], y=sysbp_trend['mean'],
                        mode='lines+markers', name='平均值',
                        error_y=dict(type='data', array=sysbp_trend['std'])))
fig.update_layout(title='收缩压趋势（各访视）',
                 xaxis_title='访视', yaxis_title='收缩压 (mmHg)')
fig.show()

## 7. 如何修改和扩展

### 7.1 添加新的 SDTM 域

1. 创建数据文件 (如 `lb.csv` 用于实验室检查)
2. 在配置文件的 `local_datasets.datasets` 中添加条目
3. 在 `table_info.yaml` 中添加表结构信息

### 7.2 自定义监控规则

在 `openchatbi/tool/` 下创建新的工具，例如：

In [ ]:
# 示例：创建数据质量检查工具
def check_data_quality(dataset_name: str) -> dict:
    """检查数据集的质量问题"""
    manager = LocalDatasetManager(config['local_datasets']['datasets'])
    df = manager.load_dataset(dataset_name)
    
    issues = {
        'missing_values': df.isnull().sum().to_dict(),
        'duplicates': df.duplicated().sum(),
        'total_records': len(df)
    }
    
    return issues

# 使用示例
print("DM 数据质量检查:")
print(check_data_quality('dm'))

### 7.3 集成到 Agent

修改 `agent_graph.py`，添加临床特定的节点：

```python
# 添加安全性监控节点
def safety_monitoring_node(state: AgentState):
    """检查严重不良事件"""
    ae = manager.load_dataset('ae')
    sae = ae[ae['AESER'] == 'Y']
    
    if len(sae) > threshold:
        return {"alert": "严重不良事件超过阈值！"}
    return {"status": "正常"}
```

## 8. 实战演示：启动 Streamlit UI

运行以下命令启动 Web 界面：

# Section 7: Text2SQL 模块详解

本节将详细介绍 `openchatbi/text2sql` 模块的使用方法。该模块是 OpenChatBI 的核心功能，负责将自然语言问题转换为 SQL 查询并执行。

## 7.1 模块概述

`text2sql` 模块包含以下核心组件：

1. **extraction.py** - 信息抽取：从用户问题中提取关键信息（维度、指标、时间范围等）
2. **schema_linking.py** - 模式关联：根据问题选择相关的表和列
3. **generate_sql.py** - SQL生成与执行：生成SQL查询并执行
4. **visualization.py** - 可视化：为查询结果推荐和生成图表
5. **sql_graph.py** - 工作流编排：构建完整的Text2SQL处理流程
6. **data.py** - 示例数据管理：管理SQL示例和表选择示例
7. **text2sql_utils.py** - 工具函数：向量检索等辅助功能

我们将使用 SDTM 临床试验数据作为示例，逐步演示每个组件的使用方法。

## 7.2 准备工作：导入模块和加载数据

首先导入必要的模块并加载 SDTM 数据。

In [ ]:
# 导入 text2sql 模块的各个组件
from openchatbi.text2sql.extraction import (
    generate_extraction_prompt,
    parse_extracted_info_json,
    information_extraction
)
from openchatbi.text2sql.schema_linking import schema_linking
from openchatbi.text2sql.generate_sql import create_sql_nodes
from openchatbi.text2sql.visualization import VisualizationService, ChartType
from openchatbi.text2sql.sql_graph import build_sql_graph
from openchatbi.text2sql.text2sql_utils import (
    init_sql_example_retriever,
    init_table_selection_example_dict
)

# 导入其他必要的模块
from openchatbi.graph_state import SQLGraphState
from openchatbi.llm.llm import get_default_llm, get_text2sql_llm, get_embedding_model
from langchain_core.messages import HumanMessage, SystemMessage
import pandas as pd

print("✓ Text2SQL 模块导入成功")

# 加载 SDTM 数据
dm_df = pd.read_csv('data/sdtm/dm.csv')
ae_df = pd.read_csv('data/sdtm/ae.csv')
vs_df = pd.read_csv('data/sdtm/vs.csv')

print(f"✓ 数据加载成功: DM({len(dm_df)}行), AE({len(ae_df)}行), VS({len(vs_df)}行)")

## 7.3 extraction.py - 信息抽取

`extraction.py` 负责从用户的自然语言问题中提取结构化信息，包括：
- **keywords**: 关键词列表
- **dimensions**: 维度字段（如 ARM, SEX, AESER）
- **metrics**: 指标字段（如受试者数量、AE发生率）
- **filter**: 过滤条件
- **start_time/end_time**: 时间范围
- **rewrite_question**: 重写后的明确问题

### 7.3.1 生成抽取提示词

`generate_extraction_prompt()` 函数生成用于信息抽取的系统提示词。

In [ ]:
# 示例：生成信息抽取提示词
extraction_prompt = generate_extraction_prompt()

# 显示提示词的前500个字符
print("信息抽取提示词（前500字符）:")
print("=" * 80)
print(extraction_prompt[:500])
print("..." if len(extraction_prompt) > 500 else "")
print("=" * 80)
print(f"提示词总长度: {len(extraction_prompt)} 字符")

### 7.3.2 解析抽取结果

`parse_extracted_info_json()` 函数从 LLM 的返回结果中提取 JSON 数据。

In [ ]:
# 示例：模拟 LLM 返回的 JSON 字符串
mock_llm_response = '''
```json
{
  "reasoning": "用户想查询各治疗组的严重不良事件发生率",
  "keywords": ["治疗组", "ARM", "严重不良事件", "SAE", "AESER", "发生率"],
  "dimensions": ["ARM", "AESER"],
  "metrics": ["SAE incidence rate", "subjects with SAE", "total subjects"],
  "filter": ["AESER='Y'"],
  "rewrite_question": "计算各治疗组（ARM）的严重不良事件（AESER='Y'）发生率，显示有SAE的受试者数除以该组总受试者数"
}
```
'''

# 解析 JSON
parsed_info = parse_extracted_info_json(mock_llm_response)

print("解析后的信息:")
print("=" * 80)
for key, value in parsed_info.items():
    print(f"{key}: {value}")
print("=" * 80)

### 7.3.3 完整的信息抽取流程

`information_extraction()` 函数创建一个节点函数，用于从 SQLGraphState 中提取信息。

**注意**: 这个函数需要在 LangGraph 流程中使用，这里我们演示其基本原理。

In [ ]:
# 演示信息抽取的工作原理
print("信息抽取模块的核心功能:")
print("=" * 80)
print("1. generate_extraction_prompt() - 生成包含SDTM知识的系统提示词")
print("2. information_extraction(llm) - 创建信息抽取节点函数")
print("3. parse_extracted_info_json() - 解析LLM返回的JSON结果")
print()
print("处理流程:")
print("  用户问题 → 系统提示词 + 用户问题 → LLM → JSON响应 → 解析 → 结构化信息")
print()
print("抽取的信息包括:")
print("  - keywords: 关键词列表")
print("  - dimensions: SDTM维度字段 (USUBJID, ARM, SITEID, AESER等)")
print("  - metrics: 指标和计算 (受试者数、AE发生率等)")
print("  - filter: SQL过滤条件")
print("  - start_time/end_time: 时间范围")
print("  - rewrite_question: 明确化的问题描述")
print("=" * 80)

## 7.4 schema_linking.py - 模式关联

`schema_linking.py` 根据提取的信息选择相关的数据库表和列。

In [ ]:
# 演示模式关联原理
print("模式关联（Schema Linking）:")
print("=" * 80)
print("功能: 根据关键词、维度、指标选择相关的表和列")
print()
print("示例输入:")
print("  keywords: ['治疗组', 'ARM', '严重不良事件', 'AESER']")
print("  dimensions: ['ARM', 'AESER']")
print("  metrics: ['SAE发生率']")
print()
print("示例输出:")
print('  {"tables": [')
print('    {"table": "dm", "columns": ["USUBJID", "ARM"]},')
print('    {"table": "ae", "columns": ["USUBJID", "AESER", "AETERM"]}')
print('  ]}')
print("=" * 80)

## 7.5 generate_sql.py - SQL生成与执行

`create_sql_nodes()` 创建4个节点函数: generate_sql_node, execute_sql_node, regenerate_sql_node, generate_visualization_node

In [ ]:
# SQL生成示例
print("SQL生成模块:")
print("=" * 80)
print()
print("示例生成的SDTM SQL:")
print("-" * 80)
sql_example = """
SELECT 
    dm.ARM,
    COUNT(DISTINCT dm.USUBJID) as total_subjects,
    COUNT(DISTINCT CASE WHEN ae.AESER = 'Y' 
          THEN ae.USUBJID END) as subjects_with_sae,
    COUNT(DISTINCT CASE WHEN ae.AESER = 'Y' 
          THEN ae.USUBJID END) * 100.0 
        / COUNT(DISTINCT dm.USUBJID) as sae_rate
FROM dm
LEFT JOIN ae ON dm.USUBJID = ae.USUBJID
GROUP BY dm.ARM
"""
print(sql_example)
print("=" * 80)

## 7.6 visualization.py - 可视化服务

`VisualizationService` 类负责为查询结果推荐合适的图表类型。

In [ ]:
# 演示可视化服务
from openchatbi.text2sql.visualization import VisualizationService, ChartType

# 创建可视化服务（规则模式）
viz_service = VisualizationService(llm=None)

print("可视化服务支持的图表类型:")
print("=" * 80)
for chart_type in ChartType:
    print(f"  - {chart_type.value}: {chart_type.name}")
print()
print("图表推荐规则:")
print("  - 趋势/时间序列 → LINE (折线图)")
print("  - 分布/频率 → HISTOGRAM (直方图)")  
print("  - 分类对比 → BAR (柱状图)")
print("  - 占比分析 → PIE (饼图)")
print("  - 相关性 → SCATTER (散点图)")
print("  - 数值范围 → BOX (箱线图)")
print("  - 矩阵数据 → HEATMAP (热力图)")
print("  - 详细数据 → TABLE (表格)")
print("=" * 80)

## 7.7 sql_graph.py - 工作流编排

`build_sql_graph()` 函数使用 LangGraph 构建完整的 Text2SQL 处理流程。

**流程图**:
```
START → information_extraction → table_selection → generate_sql 
                ↓                                       ↓
         search_knowledge                        execute_sql
         ask_human                                     ↓
                                              generate_visualization → END
                                                      ↓
                                              regenerate_sql (if error)
```

In [ ]:
# Text2SQL 工作流说明
print("Text2SQL 完整工作流:")
print("=" * 80)
print()
print("节点 (Nodes):")
print("  1. information_extraction - 提取问题中的关键信息")
print("  2. search_knowledge - 搜索知识库（如需要）")
print("  3. ask_human - 向用户请求澄清（如需要）")
print("  4. table_selection (schema_linking) - 选择相关表和列")
print("  5. generate_sql - 生成SQL查询")
print("  6. execute_sql - 执行SQL")
print("  7. regenerate_sql - 重新生成SQL（如执行失败）")
print("  8. generate_visualization - 推荐图表")
print()
print("条件边 (Conditional Edges):")
print("  - information_extraction → search_knowledge | ask_human | table_selection")
print("  - execute_sql → generate_visualization | regenerate_sql | end")
print()
print("重试机制:")
print("  - SQL执行失败时自动重试，最多3次")
print("  - 每次重试会将错误信息反馈给LLM")
print("=" * 80)

## 7.8 text2sql_utils.py - 工具函数

提供向量检索相关的工具函数，用于检索相似的SQL示例和表选择示例。

**主要函数**:
- `init_sql_example_retriever(catalog)` - 初始化SQL示例检索器
- `init_table_selection_example_dict(catalog)` - 初始化表选择示例检索器

In [ ]:
# 工具函数说明
print("Text2SQL 工具函数:")
print("=" * 80)
print()
print("1. init_sql_example_retriever(catalog)")
print("   - 从catalog中加载SQL示例")
print("   - 使用embedding模型创建向量数据库")
print("   - 返回检索器，用于查找相似的SQL示例")
print()
print("2. init_table_selection_example_dict(catalog)")
print("   - 从catalog中加载表选择示例")
print("   - 创建问题→表映射的向量检索器")
print("   - 用于schema_linking时参考历史选择")
print()
print("检索配置:")
print("  - 检索算法: MMR (Maximal Marginal Relevance)")
print("  - 相似度度量: cosine distance")
print("  - fetch_k: 30 (初始候选数)")
print("  - k: 10 (最终返回数)")
print()
print("这些检索器帮助模型:")
print("  ✓ 学习类似问题的SQL写法")
print("  ✓ 选择正确的表和列")
print("  ✓ 提高SQL生成的准确性")
print("=" * 80)

## 7.9 完整示例：Text2SQL处理流程

下面演示如何使用 text2sql 模块处理一个完整的SDTM查询。

**场景**: 用户问"各治疗组的严重不良事件发生率是多少？"

我们将展示从问题到SQL的完整转换过程。

In [ ]:
# 完整的 Text2SQL 处理示例
print("=" * 80)
print("完整的 Text2SQL 处理流程示例")
print("=" * 80)
print()
print("用户问题: '各治疗组的严重不良事件发生率是多少？'")
print()
print("步骤1: 信息抽取 (extraction.py)")
print("-" * 80)
extraction_result = {
    "keywords": ["治疗组", "ARM", "严重不良事件", "SAE", "AESER", "发生率"],
    "dimensions": ["ARM", "AESER"],
    "metrics": ["SAE发生率", "有SAE的受试者数", "总受试者数"],
    "filter": ["AESER='Y'"],
    "rewrite_question": "计算各治疗组的严重不良事件发生率"
}
for key, value in extraction_result.items():
    print(f"  {key}: {value}")
print()

print("步骤2: 模式关联 (schema_linking.py)")
print("-" * 80)
selected_tables = [
    {"table": "dm", "columns": ["USUBJID", "ARM"]},
    {"table": "ae", "columns": ["USUBJID", "AESER", "AETERM"]}
]
print(f"  选中的表: {[t['table'] for t in selected_tables]}")
print(f"  dm表列: {selected_tables[0]['columns']}")
print(f"  ae表列: {selected_tables[1]['columns']}")
print()

print("步骤3: SQL生成 (generate_sql.py)")
print("-" * 80)
generated_sql = """SELECT 
    dm.ARM,
    COUNT(DISTINCT dm.USUBJID) as total_subjects,
    COUNT(DISTINCT CASE WHEN ae.AESER = 'Y' 
          THEN ae.USUBJID END) as subjects_with_sae,
    COUNT(DISTINCT CASE WHEN ae.AESER = 'Y' 
          THEN ae.USUBJID END) * 100.0 
        / COUNT(DISTINCT dm.USUBJID) as sae_rate
FROM dm
LEFT JOIN ae ON dm.USUBJID = ae.USUBJID
GROUP BY dm.ARM"""
print(generated_sql)
print()

print("步骤4: SQL执行")
print("-" * 80)
# 使用pandas模拟SQL执行结果
result_data = {
    'ARM': ['治疗组A', '治疗组B', '对照组'],
    'total_subjects': [30, 32, 28],
    'subjects_with_sae': [3, 5, 2],
    'sae_rate': [10.0, 15.6, 7.1]
}
result_df = pd.DataFrame(result_data)
print(result_df.to_string(index=False))
print()

print("步骤5: 可视化推荐 (visualization.py)")
print("-" * 80)
print("  推荐图表类型: BAR (柱状图)")
print("  X轴: ARM (治疗组)")
print("  Y轴: sae_rate (SAE发生率 %)")
print("  图表标题: '各治疗组严重不良事件发生率对比'")
print()
print("=" * 80)
print("✓ Text2SQL 流程完成！")
print("=" * 80)

## 7.10 小结：text2sql 模块的关键点

### 模块架构
- **分层设计**: 信息抽取 → 模式关联 → SQL生成 → 执行 → 可视化
- **LangGraph编排**: 使用状态图管理复杂的处理流程
- **向量检索**: 利用历史示例提高准确性

### SDTM特定优化
1. **提示词**: 已更新为包含SDTM IG 3.4知识
2. **域关联**: 自动识别需要通过USUBJID关联的表
3. **SDTM规范**: 遵循变量命名、日期格式、控制术语等规范

### 最佳实践
- 配置高质量的SQL示例和表选择示例
- 使用合适的embedding模型提高检索准确性
- 设置适当的重试次数处理SQL错误
- 选择合适的可视化模式（rule或llm）

### 下一步
Section 8将演示如何启动完整的Streamlit UI界面，使用所有这些模块。

# Section 8: Tool 模块详解

本节将详细介绍 `openchatbi/tool` 模块的使用方法。该模块提供了一系列工具函数，供 Agent 在处理用户请求时调用。

## 8.1 模块概述

`tool` 模块包含以下核心工具：

1. **ask_human.py** - 向用户请求澄清信息
2. **search_knowledge.py** - 搜索知识库和模式信息
3. **run_python_code.py** - 执行Python代码
4. **memory.py** - 长期记忆管理（用户偏好、上下文）
5. **save_report.py** - 保存报告到文件
6. **timeseries_forecast.py** - 时间序列预测
7. **mcp_tools.py** - Model Context Protocol工具集成

这些工具使 Agent 能够与用户交互、访问知识库、执行代码、管理记忆等。

## 8.2 准备工作：导入工具模块

In [ ]:
# 导入 tool 模块的各个工具
from openchatbi.tool.ask_human import AskHuman
from openchatbi.tool.search_knowledge import search_knowledge, show_schema
from openchatbi.tool.run_python_code import run_python_code
from openchatbi.tool.save_report import save_report
from openchatbi.tool.memory import get_sync_memory_store
import pandas as pd

print("✓ Tool 模块导入成功")
print()
print("可用工具:")
print("  1. AskHuman - 向用户请求澄清")
print("  2. search_knowledge - 搜索知识库")
print("  3. show_schema - 显示表结构")
print("  4. run_python_code - 执行Python代码")
print("  5. save_report - 保存报告")
print("  6. memory - 记忆管理")

# 加载 SDTM 数据供后续演示使用
dm_df = pd.read_csv('data/sdtm/dm.csv')
ae_df = pd.read_csv('data/sdtm/ae.csv')
vs_df = pd.read_csv('data/sdtm/vs.csv')
print(f"\n✓ SDTM数据已加载")

## 8.3 ask_human.py - 请求用户澄清

`AskHuman` 工具用于当信息不明确或缺失时向用户请求澄清。

**使用场景**:
- 用户问题模糊（如"那个研究中心"但未指定哪个）
- 需要用户选择（提供选项列表）
- 缺少必要参数

**重要**: 只在**确实需要**时使用，优先尝试从上下文推断。

In [ ]:
# 演示 AskHuman 工具的使用
print("AskHuman 工具示例:")
print("=" * 80)
print()

# 示例1: 开放式问题
print("示例1: 请求用户指定研究中心")
print("-" * 80)
ask_tool_1 = AskHuman(
    question="请问您想查询哪个研究中心的数据？",
    options=[]  # 空列表表示开放式问题
)
print(f"  问题: {ask_tool_1.question}")
print(f"  选项: {ask_tool_1.options if ask_tool_1.options else '(开放式回答)'}")
print()

# 示例2: 选择题
print("示例2: 让用户选择治疗组")
print("-" * 80)
treatment_arms = dm_df['ARM'].unique().tolist()[:3]  # 获取前3个治疗组
ask_tool_2 = AskHuman(
    question="请选择要分析的治疗组：",
    options=treatment_arms
)
print(f"  问题: {ask_tool_2.question}")
print(f"  选项: {ask_tool_2.options}")
print()

# 示例3: SDTM 场景 - 选择严重程度
print("示例3: 选择不良事件严重程度")
print("-" * 80)
severities = ae_df['AESEV'].dropna().unique().tolist()[:3]
ask_tool_3 = AskHuman(
    question="请选择要分析的不良事件严重程度：",
    options=severities
)
print(f"  问题: {ask_tool_3.question}")
print(f"  选项: {ask_tool_3.options}")
print()

print("=" * 80)
print("注意: 在实际Agent中，这个工具会触发 LangGraph 的 interrupt，")
print("      等待用户输入后再继续执行。")
print("=" * 80)

## 8.4 search_knowledge.py - 搜索知识库

提供两个工具函数：
- **search_knowledge** - 搜索列定义、业务知识
- **show_schema** - 显示完整的表结构

这些工具帮助Agent理解数据库结构和业务术语。

In [ ]:
# 演示知识库搜索工具
print("知识库搜索工具:")
print("=" * 80)
print()

print("工具1: search_knowledge")
print("-" * 80)
print("功能: 搜索列描述、别名和业务知识")
print()
print("参数:")
print("  - reasoning: 搜索原因")
print("  - query_list: 搜索词列表（最多5个）")
print("  - knowledge_bases: ['columns', 'business']")
print("  - with_table_list: 是否包含表列表")
print()
print("示例调用（SDTM场景）:")
print("  search_knowledge(")
print('    reasoning="用户询问严重不良事件，需要了解AESER变量",')
print('    query_list=["AESER", "严重不良事件", "SAE"],')
print('    knowledge_bases=["columns"],')
print("    with_table_list=True")
print("  )")
print()
print("返回示例:")
print("  {")
print('    "columns": "# Relevant Columns and Description:"')
print('               "AESER: Serious Event (Y/N), Tables: [ae]"')
print('               "AETERM: Adverse Event Term, Tables: [ae]"')
print("  }")
print()

print("工具2: show_schema")
print("-" * 80)
print("功能: 显示完整表结构")
print()
print("参数:")
print("  - reasoning: 查看原因")
print("  - tables: 表名列表（最多5个）")
print()
print("示例调用（SDTM场景）:")
print("  show_schema(")
print('    reasoning="需要了解DM和AE表的完整结构",')
print('    tables=["dm", "ae"]')
print("  )")
print()
print("返回: 表描述、所有列、派生指标等")
print()
print("=" * 80)
print("注意: 这些工具在配置了catalog后才能正常工作")
print("=" * 80)

## 8.5 run_python_code.py - 执行Python代码

`run_python_code` 工具允许Agent执行Python代码进行数据分析、计算等操作。

**支持的执行器**:
- **LocalExecutor** - 本地执行（默认）
- **RestrictedLocalExecutor** - 受限本地执行（安全模式）
- **DockerExecutor** - Docker容器执行（隔离环境）

**重要**: 只有 `print()` 输出可见，函数返回值会被忽略。

In [ ]:
# 演示 run_python_code 工具
print("run_python_code 工具示例:")
print("=" * 80)
print()

# 示例1: 使用SDTM数据计算SAE发生率
print("示例1: 计算严重不良事件发生率")
print("-" * 80)
sdtm_code_1 = """
import pandas as pd

# 加载数据
dm = pd.read_csv('data/sdtm/dm.csv')
ae = pd.read_csv('data/sdtm/ae.csv')

# 计算SAE发生率
total_subjects = dm['USUBJID'].nunique()
sae_subjects = ae[ae['AESER'] == 'Y']['USUBJID'].nunique()
sae_rate = (sae_subjects / total_subjects) * 100

print(f"总受试者数: {total_subjects}")
print(f"发生SAE的受试者数: {sae_subjects}")
print(f"SAE发生率: {sae_rate:.2f}%")
"""
print("代码:")
print(sdtm_code_1)
print()

# 示例2: 各治疗组的受试者数分布
print("示例2: 各治疗组受试者数分布")
print("-" * 80)
sdtm_code_2 = """
import pandas as pd

dm = pd.read_csv('data/sdtm/dm.csv')

# 按治疗组统计
arm_counts = dm['ARM'].value_counts()

print("各治疗组受试者数:")
for arm, count in arm_counts.items():
    print(f"  {arm}: {count}名")
"""
print("代码:")
print(sdtm_code_2)
print()

# 示例3: 计算年龄统计
print("示例3: 年龄统计")
print("-" * 80)
sdtm_code_3 = """
import pandas as pd

dm = pd.read_csv('data/sdtm/dm.csv')

print(f"平均年龄: {dm['AGE'].mean():.1f}岁")
print(f"年龄范围: {dm['AGE'].min()}-{dm['AGE'].max()}岁")
print(f"中位年龄: {dm['AGE'].median():.1f}岁")
"""
print("代码:")
print(sdtm_code_3)
print()

print("=" * 80)
print("工具调用格式:")
print("  run_python_code(")
print('    reasoning="需要计算SAE发生率",')
print('    code=sdtm_code_1')
print("  )")
print()
print("注意事项:")
print("  ✓ 必须使用 print() 输出结果")
print("  ✓ 函数返回值不会被显示")
print("  ✓ 支持 pandas, numpy 等常用库")
print("  ✓ 可以读取文件和数据")
print("=" * 80)

## 8.6 save_report.py - 保存报告

`save_report` 工具将分析结果保存为文件，支持多种格式。

**支持的格式**: markdown (.md), CSV (.csv), 文本 (.txt), JSON (.json)

文件名格式: `timestamp_title.format` (如: `20251215_143025_SAE分析报告.md`)

In [ ]:
# 演示 save_report 工具
print("save_report 工具示例:")
print("=" * 80)
print()

# 示例1: 保存Markdown格式的SDTM分析报告
print("示例1: 保存SAE分析报告 (Markdown)")
print("-" * 80)
sae_report_content = """# 严重不良事件(SAE)分析报告

## 1. 总体概况

- 总受试者数: 100名
- 发生SAE的受试者数: 12名
- SAE发生率: 12.00%

## 2. 各治疗组SAE发生率

| 治疗组 | 受试者数 | SAE受试者数 | 发生率 |
|--------|---------|------------|--------|
| 治疗组A | 30 | 3 | 10.0% |
| 治疗组B | 35 | 5 | 14.3% |
| 对照组 | 35 | 4 | 11.4% |

## 3. 最常见的SAE

1. 心血管事件 - 5例
2. 感染 - 3例
3. 肝功能异常 - 2例

## 4. 结论

治疗组B的SAE发生率略高，但各组间无显著差异。
"""

print("报告内容:")
print(sae_report_content[:200] + "...")
print()
print("调用示例:")
print("  save_report(")
print('    content=sae_report_content,')
print('    title="SAE分析报告",')
print('    file_format="md"')
print("  )")
print()
print("生成文件名: 20251215_143025_SAE分析报告.md")
print()

# 示例2: 保存CSV格式的数据
print("示例2: 保存治疗组统计数据 (CSV)")
print("-" * 80)
csv_content = """ARM,Total_Subjects,SAE_Subjects,SAE_Rate
治疗组A,30,3,10.0
治疗组B,35,5,14.3
对照组,35,4,11.4
"""
print("CSV内容:")
print(csv_content)
print("调用示例:")
print("  save_report(")
print('    content=csv_content,')
print('    title="治疗组SAE统计",')
print('    file_format="csv"')
print("  )")
print()

# 示例3: 保存JSON格式
print("示例3: 保存结构化数据 (JSON)")
print("-" * 80)
import json
json_data = {
    "study_id": "STUDY001",
    "analysis_date": "2025-12-15",
    "total_subjects": 100,
    "sae_subjects": 12,
    "sae_rate": 12.0,
    "by_arm": [
        {"arm": "治疗组A", "sae_rate": 10.0},
        {"arm": "治疗组B", "sae_rate": 14.3},
        {"arm": "对照组", "sae_rate": 11.4}
    ]
}
json_content = json.dumps(json_data, indent=2, ensure_ascii=False)
print("JSON内容:")
print(json_content)
print()
print("调用示例:")
print("  save_report(")
print('    content=json_content,')
print('    title="SAE统计数据",')
print('    file_format="json"')
print("  )")
print()

print("=" * 80)
print("配置:")
print("  报告保存目录由 config.report_directory 配置")
print("  默认目录: ./reports/")
print("=" * 80)

## 8.7 memory.py - 记忆管理

`memory` 模块提供长期记忆功能，使Agent能够记住用户偏好和上下文信息。

**核心功能**:
- 存储用户配置（语言、时区、术语偏好）
- 保存对话历史和重要信息
- 向量搜索历史记忆
- 跨会话持久化

**存储后端**: SQLite + 向量索引（1536维embedding）

In [ ]:
# 演示 memory 模块
print("Memory 记忆管理模块:")
print("=" * 80)
print()

print("核心组件:")
print("-" * 80)
print("1. UserProfile - 用户配置数据模型")
print("   - name: 用户名称")
print("   - language: 语言偏好 (中文/English)")
print("   - timezone: 时区 (Asia/Shanghai)")
print("   - jargon: 术语偏好 (clinical trial专业术语)")
print()

print("2. get_sync_memory_store() - 获取同步记忆存储")
print("   - 使用 SQLite 作为持久化存储")
print("   - 支持向量索引（1536维embedding）")
print("   - 用于搜索相关历史记忆")
print()

print("3. get_async_memory_store() - 获取异步记忆存储")
print("   - 用于异步操作场景")
print("   - 与同步版本共享同一数据库")
print()

print("使用场景 (SDTM临床试验):")
print("-" * 80)
print()
print("场景1: 记住用户的分析偏好")
print('  - "我通常关注严重不良事件" → 记忆中保存')
print('  - 下次用户问"分析不良事件" → 自动关注SAE')
print()

print("场景2: 记住研究上下文")
print('  - "我在分析STUDY001" → 记忆中保存study_id')
print('  - 后续查询自动过滤该研究的数据')
print()

print("场景3: 记住术语映射")
print('  - "受试者 = Subject = USUBJID"')
print('  - "严重AE = SAE = AESER=\'Y\'"')
print('  - 用户用中文提问时自动转换为SDTM术语')
print()

print("场景4: 跨会话记忆")
print("  - 会话1: 分析了治疗组A的数据")
print("  - 会话2: '继续上次的分析' → 自动加载治疗组A上下文")
print()

print("工具函数:")
print("-" * 80)
print("  - create_search_memory_tool(store, llm)")
print("    搜索相关历史记忆")
print()
print("  - create_manage_memory_tool(store, llm)")
print("    管理记忆（添加、更新、删除）")
print()

print("=" * 80)
print("数据库文件: memory.db")
print("表结构: 包含文本内容、向量embedding、metadata")
print("=" * 80)

## 8.8 timeseries_forecast.py - 时间序列预测

`timeseries_forecast` 工具提供时间序列预测功能，适用于趋势分析。

**支持的频率**: hourly, daily, weekly, monthly

**使用场景 (SDTM)**:
- 预测未来访视的生命体征趋势
- 预测AE累积发生率
- 预测受试者入组速度

**注意**: 需要启动独立的预测服务（Docker容器）

In [ ]:
# 演示时间序列预测工具
print("时间序列预测工具:")
print("=" * 80)
print()

print("使用场景 (SDTM临床试验):")
print("-" * 80)
print()

# 示例1: 预测生命体征趋势
print("示例1: 预测受试者体重变化趋势")
print("-" * 80)
print("输入: 某受试者前8次访视的体重数据")
weight_data = [75.2, 74.8, 74.5, 74.0, 73.8, 73.5, 73.2, 73.0]
print(f"历史体重: {weight_data}")
print()
print("调用示例:")
print("  timeseries_forecast(")
print('    reasoning="预测未来4次访视的体重变化",')
print(f"    input_data={weight_data},")
print("    forecast_window=4,")
print('    frequency="weekly"')
print("  )")
print()
print("预期输出:")
print("  未来4周体重预测: [72.7, 72.5, 72.2, 72.0]")
print("  趋势: 持续下降")
print()

# 示例2: 预测AE累积发生数
print("示例2: 预测不良事件累积发生数")
print("-" * 80)
print("输入: 前12周每周新增AE数量")
ae_weekly = [2, 3, 5, 4, 6, 5, 4, 3, 4, 5, 3, 4]
print(f"每周新增AE: {ae_weekly}")
print()
print("调用示例:")
print("  timeseries_forecast(")
print('    reasoning="预测未来4周的AE发生趋势",')
print(f"    input_data={ae_weekly},")
print("    forecast_window=4,")
print('    frequency="weekly"')
print("  )")
print()
print("预期输出:")
print("  未来4周预测: [4, 4, 3, 4]")
print("  趋势: 稳定")
print()

# 示例3: 结构化数据预测
print("示例3: 使用结构化数据（带时间戳）")
print("-" * 80)
structured_data = [
    {"date": "2024-01-01", "sysbp": 120},
    {"date": "2024-01-08", "sysbp": 122},
    {"date": "2024-01-15", "sysbp": 125},
]
print("血压数据:")
for record in structured_data:
    print(f"  {record}")
print()
print("调用示例:")
print("  timeseries_forecast(")
print('    reasoning="预测未来血压趋势",')
print("    input_data=structured_data,")
print("    forecast_window=3,")
print('    frequency="weekly",')
print('    target_column="sysbp"')
print("  )")
print()

print("=" * 80)
print("前置条件:")
print("  1. 启动预测服务:")
print("     docker run -p 8765:8765 timeseries-forecasting")
print()
print("  2. 配置服务URL:")
print("     config.timeseries_forecasting_service_url = 'http://localhost:8765'")
print()
print("  3. 检查服务健康:")
print("     check_forecast_service_health()")
print("=" * 80)

## 8.9 mcp_tools.py - MCP工具集成

Model Context Protocol (MCP) 工具集成模块，允许Agent使用外部MCP服务器提供的工具。

**功能**:
- 连接多个MCP服务器
- 自动发现服务器提供的工具
- 同步/异步调用兼容
- 超时控制

**应用场景**: 扩展Agent能力，如文件操作、数据库访问、API调用等。

In [ ]:
# 演示 MCP 工具集成
print("MCP (Model Context Protocol) 工具集成:")
print("=" * 80)
print()

print("什么是MCP?")
print("-" * 80)
print("MCP是一个标准协议，允许AI应用连接到外部数据源和工具。")
print("通过MCP，Agent可以访问:")
print("  - 文件系统")
print("  - 数据库")
print("  - API服务")
print("  - 其他外部工具")
print()

print("核心功能:")
print("-" * 80)
print("1. MultiServerMCPClient - 连接多个MCP服务器")
print("2. make_tool_sync_compatible() - 使异步工具同步兼容")
print("3. 自动工具发现和注册")
print("4. 超时控制（默认30秒）")
print()

print("配置示例:")
print("-" * 80)
mcp_config_example = """
# config.yaml
mcp_servers:
  filesystem:
    command: "npx"
    args: ["-y", "@modelcontextprotocol/server-filesystem", "/path/to/data"]
  database:
    command: "mcp-server-sqlite"
    args: ["--db", "clinical_trial.db"]
"""
print(mcp_config_example)
print()

print("使用场景 (SDTM临床试验):")
print("-" * 80)
print()
print("场景1: 文件系统访问")
print("  - 读取本地SDTM CSV文件")
print("  - 列出数据目录中的文件")
print("  - 写入分析结果")
print()

print("场景2: 数据库操作")
print("  - 查询临床试验数据库")
print("  - 执行复杂的SQL JOIN")
print("  - 更新数据质量标志")
print()

print("场景3: API集成")
print("  - 调用CRO的数据API")
print("  - 获取最新的受试者数据")
print("  - 提交安全性报告")
print()

print("工具使用流程:")
print("-" * 80)
print("1. 启动MCP服务器")
print("2. 配置连接信息")
print("3. Agent自动发现可用工具")
print("4. 在对话中使用工具")
print()

print("示例工具调用:")
print("-" * 80)
print("  # 通过MCP文件系统工具读取文件")
print('  mcp_tool = agent.get_tool("read_file")')
print('  result = mcp_tool.invoke({"path": "data/sdtm/dm.csv"})')
print()
print("  # 通过MCP数据库工具查询")
print('  db_tool = agent.get_tool("query_database")')
print('  result = db_tool.invoke({"sql": "SELECT * FROM ae WHERE AESER=\'Y\'"})')
print()

print("=" * 80)
print("优势:")
print("  ✓ 标准化的工具接口")
print("  ✓ 易于扩展新工具")
print("  ✓ 支持多种数据源")
print("  ✓ 安全的访问控制")
print("=" * 80)

## 8.10 工具协同使用示例

展示多个工具如何协同工作完成复杂的SDTM数据分析任务。

**场景**: 分析各治疗组的SAE发生率，并生成报告

In [ ]:
# 工具协同使用完整示例
print("=" * 80)
print("完整工作流: SAE发生率分析与报告生成")
print("=" * 80)
print()

print("场景: 用户问'各治疗组的严重不良事件发生率是多少？并生成报告'")
print()

print("步骤1: search_knowledge - 搜索SDTM术语")
print("-" * 80)
print("Agent调用:")
print("  search_knowledge(")
print('    reasoning="用户询问SAE，需要了解AESER变量",')
print('    query_list=["AESER", "严重不良事件"],')
print('    knowledge_bases=["columns"]')
print("  )")
print("结果: 了解到AESER='Y'表示严重AE")
print()

print("步骤2: run_python_code - 执行数据分析")
print("-" * 80)
analysis_code = """
import pandas as pd

dm = pd.read_csv('data/sdtm/dm.csv')
ae = pd.read_csv('data/sdtm/ae.csv')

# 计算各治疗组SAE发生率
result = dm.groupby('ARM').agg(
    total_subjects=('USUBJID', 'nunique')
).reset_index()

sae_counts = ae[ae['AESER'] == 'Y'].groupby('ARM').agg(
    sae_subjects=('USUBJID', 'nunique')
).reset_index()

result = result.merge(sae_counts, on='ARM', how='left')
result['sae_subjects'] = result['sae_subjects'].fillna(0)
result['sae_rate'] = (result['sae_subjects'] / result['total_subjects'] * 100).round(2)

print(result.to_markdown(index=False))
"""
print("Agent调用:")
print("  run_python_code(")
print('    reasoning="计算各组SAE发生率",')
print("    code=analysis_code")
print("  )")
print()
print("结果:")
print("| ARM      | total_subjects | sae_subjects | sae_rate |")
print("|----------|----------------|--------------|----------|")
print("| 治疗组A  | 30             | 3            | 10.00    |")
print("| 治疗组B  | 35             | 5            | 14.29    |")
print("| 对照组   | 35             | 4            | 11.43    |")
print()

print("步骤3: save_report - 生成并保存报告")
print("-" * 80)
report_content = """# 严重不良事件(SAE)分析报告

## 分析时间
2025-12-15

## 分析结果

各治疗组SAE发生率统计:

| 治疗组 | 受试者数 | SAE受试者数 | 发生率(%) |
|--------|---------|------------|-----------|
| 治疗组A | 30 | 3 | 10.00 |
| 治疗组B | 35 | 5 | 14.29 |
| 对照组 | 35 | 4 | 11.43 |

## 结论

- 总体SAE发生率: 12.00%
- 治疗组B发生率最高(14.29%)
- 各组间差异无统计学意义

## 数据来源
- DM域: data/sdtm/dm.csv
- AE域: data/sdtm/ae.csv
"""

print("Agent调用:")
print("  save_report(")
print("    content=report_content,")
print('    title="各治疗组SAE发生率分析",')
print('    file_format="md"')
print("  )")
print()
print("结果: 报告已保存至 reports/20251215_143025_各治疗组SAE发生率分析.md")
print()

print("可选步骤4: timeseries_forecast - 预测AE趋势")
print("-" * 80)
print("如果用户问'未来4周AE发生趋势如何？'")
print("Agent会:")
print("  1. 使用run_python_code提取每周AE数量")
print("  2. 调用timeseries_forecast预测")
print("  3. 将预测结果加入报告")
print()

print("可选步骤5: AskHuman - 请求进一步指示")
print("-" * 80)
print("Agent调用:")
print("  AskHuman(")
print('    question="报告已生成，是否需要进一步分析？",')
print('    options=["分析SAE具体类型", "对比不同访视", "不需要"]')
print("  )")
print()

print("=" * 80)
print("工具协同优势:")
print("  ✓ search_knowledge 提供领域知识")
print("  ✓ run_python_code 执行灵活的数据分析")
print("  ✓ save_report 持久化结果")
print("  ✓ AskHuman 引导用户深入探索")
print("  ✓ timeseries_forecast 提供预测能力")
print()
print("完整流程展示了OpenChatBI如何通过工具组合")
print("从问题理解 → 数据分析 → 结果呈现 → 用户交互")
print("=" * 80)

## 8.11 小结：Tool 模块的关键点

### 工具分类

**信息交互类**:
- `AskHuman` - 向用户请求澄清
- `search_knowledge` / `show_schema` - 搜索知识库

**数据处理类**:
- `run_python_code` - 执行数据分析代码
- `timeseries_forecast` - 时间序列预测

**结果管理类**:
- `save_report` - 保存分析报告
- `memory` - 长期记忆管理

**扩展集成类**:
- `mcp_tools` - 外部工具集成

### SDTM应用最佳实践

1. **使用search_knowledge**: 查询SDTM变量定义和控制术语
2. **使用run_python_code**: 灵活处理临床试验数据分析
3. **使用save_report**: 生成符合规范的分析报告
4. **使用memory**: 记住研究上下文和用户偏好

### 工具选择指南

- **简单查询** → Text2SQL (不需要工具)
- **复杂计算** → run_python_code
- **预测分析** → timeseries_forecast
- **信息不明** → AskHuman
- **术语不清** → search_knowledge
- **结果保存** → save_report

### 下一步

Section 9将演示如何使用完整的Agent系统，整合所有模块。

In [ ]:
# 注意：在 notebook 中运行会阻塞，建议在终端中运行
# !conda run -n chatbi streamlit run sample_ui/streamlit_ui.py --server.port 8506

print("请在终端中运行:")
print("conda activate chatbi")
print("streamlit run sample_ui/streamlit_ui.py")
print("\n然后在浏览器中打开 http://localhost:8501")
print("\n可以尝试的查询:")
print("- 有多少受试者出现了严重不良事件？")
print("- 治疗组和安慰剂组的 AE 发生率对比")
print("- 显示收缩压异常的受试者")
print("- 年龄超过 60 岁的受试者有哪些？")

# Section 9: 核心模块 - openchatbi/__init__.py 使用详解

## 9.1 模块概述

`openchatbi/__init__.py` 是 OpenChatBI 框架的**核心初始化模块**，负责：

1. **全局配置管理** - 自动加载配置文件
2. **提供便捷的 Agent Graph 构建函数** - 开箱即用
3. **导出核心组件** - 供外部使用

这是整个框架的**入口点**，所有应用都从这里开始。

## 9.2 核心组件 1: 全局配置实例 `config`

In [ ]:
# 全局配置实例的使用
from openchatbi import config

# 获取完整配置对象
cfg = config.get()

print("=== 基础配置 ===")
print(f"组织名称: {cfg.organization}")
print(f"SQL方言: {cfg.dialect}")
print(f"报告目录: {cfg.report_directory}")
print(f"Python执行器: {cfg.python_executor}")
print(f"可视化模式: {cfg.visualization_mode}")

print("\n=== LLM 配置 ===")
print(f"默认 LLM: {type(cfg.default_llm).__name__}")
print(f"Embedding 模型: {type(cfg.embedding_model).__name__ if cfg.embedding_model else 'None'}")
print(f"Text2SQL LLM: {type(cfg.text2sql_llm).__name__ if cfg.text2sql_llm else 'None'}")

print("\n=== Catalog Store ===")
print(f"Catalog Store 类型: {type(cfg.catalog_store).__name__}")
tables = cfg.catalog_store.get_all_tables()
print(f"已加载表数量: {len(tables)}")

print("\n=== 本地数据集管理器 ===")
if cfg.local_dataset_manager:
    datasets = cfg.local_dataset_manager.list_datasets()
    print(f"可用数据集: {len(datasets)} 个")
    for ds in datasets:
        print(f"  - {ds['name']}: {ds['description']}")
else:
    print("未配置本地数据集管理器")

### 配置项完整说明

| 配置项 | 类型 | 说明 | 默认值 |
|--------|------|------|--------|
| `organization` | str | 组织名称 | "The Company" |
| `dialect` | str | SQL方言 | "presto" |
| `default_llm` | BaseChatModel | 默认语言模型 | 必需 |
| `embedding_model` | BaseModel | 嵌入模型 | None |
| `text2sql_llm` | BaseChatModel | Text2SQL专用模型 | None |
| `bi_config` | dict | BI配置 | {} |
| `data_warehouse_config` | dict | 数据仓库配置 | {} |
| `catalog_store` | CatalogStore | 目录存储 | 自动创建 |
| `mcp_servers` | list | MCP服务器配置 | [] |
| `report_directory` | str | 报告保存目录 | "./data" |
| `python_executor` | str | Python执行器类型 | "local" |
| `visualization_mode` | str | 可视化模式 | "rule" |
| `context_config` | dict | 上下文管理配置 | {} |
| `timeseries_forecasting_service_url` | str | 时序预测服务URL | "http://localhost:8765" |
| `local_dataset_manager` | LocalDatasetManager | 本地数据集管理器 | None |

## 9.3 核心组件 2: `get_default_graph()` 函数

`get_default_graph()` 是最常用的函数，用于快速构建一个**同步模式**的 Agent Graph。

### 特点

- ✅ **开箱即用** - 无需手动配置
- ✅ **自动记忆管理** - 短期记忆（MemorySaver）+ 长期记忆（memory store）
- ✅ **使用默认配置** - 从 config.yaml 自动加载
- ✅ **适合大多数场景** - 快速原型、标准应用

In [ ]:
# 使用 get_default_graph() 构建 Agent
from openchatbi import get_default_graph

print("=== 构建 Agent Graph ===")
graph = get_default_graph()
print("✅ Agent Graph 构建成功！")
print(f"Graph 类型: {type(graph).__name__}")

# 准备测试输入
thread_id = "init_module_demo"
config_dict = {"configurable": {"thread_id": thread_id}}

# 简单测试
print("\n=== 测试 Agent 响应 ===")
test_question = "你好，请介绍一下自己"
result = graph.invoke(
    {"messages": [{"role": "user", "content": test_question}]},
    config_dict
)

print(f"用户: {test_question}")
print(f"助手: {result.get('final_answer', '未获取到回答')}")

## 9.4 完整示例：SDTM 数据分析

In [ ]:
# 完整的 SDTM 数据分析流程
from openchatbi import config, get_default_graph

# ===== 1. 检查配置 =====
print("="*60)
print("1. 配置信息检查")
print("="*60)
cfg = config.get()
print(f"组织: {cfg.organization}")
print(f"SQL方言: {cfg.dialect}")
print(f"报告目录: {cfg.report_directory}")

# 检查本地数据集
if cfg.local_dataset_manager:
    datasets = cfg.local_dataset_manager.list_datasets()
    print(f"\n可用数据集: {len(datasets)} 个")
    for ds in datasets:
        print(f"  ✓ {ds['name']}: {ds['description']}")
        
        # 加载数据查看样本
        data = cfg.local_dataset_manager.get_dataset(ds['name'])
        print(f"    记录数: {len(data)}, 列数: {len(data.columns)}")

# ===== 2. 构建 Agent Graph =====
print(f"\n{'='*60}")
print("2. 构建 Agent Graph")
print("="*60)
graph = get_default_graph()
print("✅ Agent Graph 构建成功！")

# ===== 3. 创建会话 =====
thread_id = "sdtm_complete_demo"
config_dict = {"configurable": {"thread_id": thread_id}}
print(f"\n会话ID: {thread_id}")

# ===== 4. 多轮对话分析 =====
print(f"\n{'='*60}")
print("3. 开始多轮数据分析")
print("="*60)

questions = [
    "各治疗组（ARM）有多少受试者？",
    "计算各治疗组的严重不良事件（SAE）发生率"
]

for i, question in enumerate(questions, 1):
    print(f"\n--- 问题 {i} ---")
    print(f"用户: {question}")
    
    result = graph.invoke(
        {"messages": [{"role": "user", "content": question}]},
        config_dict
    )
    
    answer = result.get("final_answer", "未获取到回答")
    print(f"助手: {answer[:200]}...")  # 显示前200字符
    
    if "sql" in result and result["sql"]:
        print(f"\n生成的 SQL: {result['sql'][:100]}...")

print(f"\n{'='*60}")
print("✅ 分析完成")
print("="*60)

## 9.5 使用场景对比

### 场景 1: 快速开始（推荐）✅

**适用于**: 初次使用、快速原型、标准场景

In [ ]:
# 场景 1: 最简单的使用方式
from openchatbi import get_default_graph

# 一行代码获取 Agent
graph = get_default_graph()

# 直接使用
result = graph.invoke(
    {"messages": [{"role": "user", "content": "统计DM表的受试者数量"}]},
    {"configurable": {"thread_id": "quick_start"}}
)

print("场景 1: 快速开始")
print(f"回答: {result.get('final_answer', '未获取到回答')[:150]}...")

### 场景 2: 自定义 Graph 构建

**适用于**: 需要自定义 checkpointer、memory store、上下文管理等

In [ ]:
# 场景 2: 自定义配置
from openchatbi import config
from openchatbi.agent_graph import build_agent_graph_sync
from langgraph.checkpoint.memory import MemorySaver
from openchatbi.tool.memory import get_sync_memory_store

# 1. 获取配置
cfg = config.get()

# 2. 自定义 checkpointer（短期记忆）
checkpointer = MemorySaver()

# 3. 自定义 memory store（长期记忆）
memory_store = get_sync_memory_store()

# 4. 构建 Graph，可控制上下文管理
print("场景 2: 自定义构建")
graph_custom = build_agent_graph_sync(
    catalog=cfg.catalog_store,
    checkpointer=checkpointer,
    memory_store=memory_store,
    enable_context_management=True  # 可以控制是否启用上下文管理
)

print("✅ 自定义 Graph 构建成功")
print("可控制的选项:")
print("  - checkpointer: 短期记忆存储（可切换到 SQLite、PostgreSQL）")
print("  - memory_store: 长期记忆存储（可切换到 Redis、PostgreSQL）")
print("  - enable_context_management: 上下文管理开关")

### 场景 3: 流式输出

**适用于**: 需要实时显示 AI 回复过程的应用

In [ ]:
# 场景 3: 流式输出
from openchatbi import get_default_graph

graph = get_default_graph()

config_dict = {"configurable": {"thread_id": "stream_demo"}}
input_msg = {"messages": [{"role": "user", "content": "DM表有哪些列？"}]}

print("场景 3: 流式输出")
print("实时显示每个节点的执行过程:\n")

# 流式输出，实时打印每个节点的输出
for i, chunk in enumerate(graph.stream(input_msg, config_dict), 1):
    print(f"步骤 {i}: {list(chunk.keys())}")
    
print("\n✅ 流式执行完成")

## 9.6 配置文件示例

OpenChatBI 使用 YAML 配置文件管理所有设置。以下是一个完整的配置示例：

```yaml
# openchatbi/config.yaml

# 组织信息
organization: "临床试验研究中心"

# SQL 方言
dialect: "presto"

# LLM 配置
default_llm:
  provider: "openai"
  model: "gpt-4"
  temperature: 0.1

embedding_model:
  provider: "openai"
  model: "text-embedding-3-small"

# 数据仓库配置
data_warehouse_config:
  type: "presto"
  host: "localhost"
  port: 8080
  catalog: "hive"
  schema: "default"

# Catalog Store 配置
catalog_store:
  store_type: "file_system"
  auto_load: true

# 报告配置
report_directory: "./data/reports"

# Python 执行器
python_executor: "local"  # local, restricted_local, docker

# 可视化配置
visualization_mode: "rule"  # rule, llm, null

# 上下文管理配置
context_config:
  enabled: true
  summary_trigger_tokens: 12000
  keep_recent_messages: 20
  enable_summarization: true

# 本地数据集配置（SDTM 示例）
local_datasets:
  enabled: true
  datasets:
    - name: "dm"
      path: "data/sdtm/dm.csv"
      description: "Demographics - 人口统计学数据"
      file_type: "csv"
    - name: "ae"
      path: "data/sdtm/ae.csv"
      description: "Adverse Events - 不良事件数据"
      file_type: "csv"
    - name: "vs"
      path: "data/sdtm/vs.csv"
      description: "Vital Signs - 生命体征数据"
      file_type: "csv"
```

## 9.7 环境变量配置

In [ ]:
# 环境变量配置示例
import os

print("=== 环境变量配置 ===\n")

# 1. 指定自定义配置文件
print("1. 自定义配置文件路径:")
print("   Windows PowerShell:")
print('   $env:CONFIG_FILE = "C:\\my_config\\custom_config.yaml"')
print()
print("   Linux/Mac:")
print('   export CONFIG_FILE="/path/to/custom_config.yaml"')

# 2. 文档构建模式
print("\n2. 文档构建模式（跳过配置加载）:")
print("   $env:SPHINX_BUILD = '1'")

# 3. 检查当前环境变量
print("\n3. 当前环境变量:")
print(f"   CONFIG_FILE: {os.getenv('CONFIG_FILE', '未设置（使用默认）')}")
print(f"   SPHINX_BUILD: {os.getenv('SPHINX_BUILD', '未设置')}")

## 9.8 常见问题 (FAQ)

### Q1: 如何更换 LLM 模型？

在 `config.yaml` 中修改配置：

```yaml
# 使用 Azure OpenAI
default_llm:
  provider: "azure_openai"
  model: "gpt-4"
  api_key: "your_key"
  endpoint: "https://your-endpoint.openai.azure.com/"

# 使用本地模型
default_llm:
  provider: "ollama"
  model: "llama3"
  base_url: "http://localhost:11434"
```

### Q2: 如何禁用上下文管理？

**方式 1**: 配置文件
```yaml
context_config:
  enabled: false
```

**方式 2**: 代码
```python
from openchatbi.agent_graph import build_agent_graph_sync
graph = build_agent_graph_sync(
    catalog=cfg.catalog_store,
    enable_context_management=False
)
```

### Q3: 如何切换到持久化存储？

```python
from langgraph.checkpoint.sqlite import SqliteSaver

checkpointer = SqliteSaver.from_conn_string("./data/checkpoints.db")
graph = build_agent_graph_sync(
    catalog=cfg.catalog_store,
    checkpointer=checkpointer,
    memory_store=get_sync_memory_store()
)
```

### Q4: 如何加载自定义数据集？

在配置文件中添加：
```yaml
local_datasets:
  enabled: true
  datasets:
    - name: "my_data"
      path: "data/my_data.csv"
      description: "自定义数据集"
      file_type: "csv"
```

### Q5: 如何查看 Graph 的执行流程？

使用流式输出或启用日志：
```python
# 流式输出
for event in graph.stream(input_msg, config_dict):
    print(f"节点: {event}")

# 启用详细日志
import logging
logging.basicConfig(level=logging.DEBUG)
```

## 9.9 最佳实践

### ✅ 推荐做法

1. **使用 `get_default_graph()` 快速开始**
2. **为每个用户/会话使用唯一的 thread_id**
3. **在配置文件中管理所有设置**
4. **启用上下文管理处理长对话**
5. **使用本地数据集简化开发**

In [ ]:
# 最佳实践示例
print("=== 最佳实践演示 ===\n")

# ✅ 推荐：使用 get_default_graph()
from openchatbi import get_default_graph
graph = get_default_graph()
print("✅ 1. 使用 get_default_graph() - 简单高效")

# ✅ 推荐：唯一的 thread_id
import uuid
user_id = "user_001"
session_id = str(uuid.uuid4())[:8]
thread_id = f"user_{user_id}_session_{session_id}"
print(f"✅ 2. 唯一 thread_id: {thread_id}")

# ✅ 推荐：配置文件管理
print("✅ 3. 所有配置在 config.yaml 中管理")

# ✅ 推荐：启用上下文管理
print("✅ 4. 上下文管理已启用（自动处理长对话）")

# ✅ 推荐：使用本地数据集
from openchatbi import config
cfg = config.get()
if cfg.local_dataset_manager:
    print(f"✅ 5. 本地数据集: {len(cfg.local_dataset_manager.list_datasets())} 个")

print("\n" + "="*60)
print("❌ 避免的做法:")
print("  1. ❌ 在循环中重复创建 Graph")
print("  2. ❌ 忽略配置文件加载失败")
print("  3. ❌ 在生产环境使用 MemorySaver（应该用持久化存储）")
print("="*60)

## 9.10 模块依赖关系

```
openchatbi/__init__.py
├── config_loader.py (ConfigLoader)
│   ├── 加载 config.yaml
│   ├── 初始化 LLM 模型
│   ├── 创建 Catalog Store
│   └── 配置本地数据集
│
├── agent_graph.py (build_agent_graph_sync/async)
│   ├── 构建 SQL Graph
│   ├── 集成工具（search_knowledge, run_python_code 等）
│   ├── 配置上下文管理
│   └── 编译 Agent Graph
│
├── tool/memory.py (get_sync_memory_store, get_async_memory_store)
│   ├── SQLite 存储
│   └── 向量索引（1536维）
│
└── langgraph (CompiledStateGraph, MemorySaver)
    ├── 状态管理
    ├── Checkpointer（短期记忆）
    └── Memory Store（长期记忆）
```

## 9.11 小结

### 核心要点

`openchatbi/__init__.py` 提供了两个核心组件：

#### 1. `config` - 全局配置管理器
- ✅ 自动加载配置文件（默认 `openchatbi/config.yaml`）
- ✅ 提供统一的配置访问接口
- ✅ 管理 LLM、数据仓库、Catalog Store、本地数据集等
- ✅ 支持环境变量自定义配置文件路径

#### 2. `get_default_graph()` - 快速构建 Agent Graph
- ✅ 开箱即用，无需手动配置
- ✅ 自动配置短期记忆（MemorySaver）
- ✅ 自动配置长期记忆（Memory Store）
- ✅ 适合大多数应用场景

### 推荐使用流程

1. **准备配置文件** - `openchatbi/config.yaml`
2. **导入模块** - `from openchatbi import get_default_graph`
3. **构建 Agent** - `graph = get_default_graph()`
4. **执行对话** - 使用唯一的 `thread_id`

### 适用场景

- 🎯 **快速原型** - 几行代码即可启动
- 🎯 **SDTM 数据分析** - 配合本地数据集管理器
- 🎯 **临床试验 BI** - 自动化数据查询和分析
- 🎯 **多轮对话** - 自动管理上下文和记忆

### 下一步

- 学习更多核心模块：`config_loader.py`, `agent_graph.py`, `graph_state.py`
- 了解工具扩展：MCP 协议集成
- 部署生产环境：持久化存储配置

---

**Section 9 完成！** 您已经掌握了 OpenChatBI 的核心入口模块。

# 核心文件详解

## 1. config_loader.py - 配置加载器

### 概述
`ConfigLoader` 是 OpenChatBI 的配置管理核心，采用**单例模式**设计，负责加载、管理和分发系统的各项配置参数。

### 主要组件

#### 1.1 Config 类 (Pydantic BaseModel)
配置数据模型，定义了系统所有可配置项：

```python
class Config(BaseModel):
    # 基础配置
    organization: str = "The Company"           # 组织名称
    dialect: str = "presto"                      # SQL方言
    
    # LLM配置
    default_llm: BaseChatModel                   # 默认LLM（必需）
    embedding_model: BaseModel | None            # 嵌入模型（可选）
    text2sql_llm: BaseChatModel | None           # Text2SQL专用LLM（可选）
    
    # BI配置
    bi_config: dict[str, Any] = {}               # 业务智能配置（词汇表、规则等）
    
    # 数据仓库配置
    data_warehouse_config: dict[str, Any] = {}   # 数据库连接信息
    
    # Catalog Store（数据目录）
    catalog_store: Any = None                    # 存储表结构、列信息等元数据
    
    # MCP服务器配置
    mcp_servers: list[dict[str, Any]] = []       # Model Context Protocol服务器列表
    
    # 报告配置
    report_directory: str = "./data"             # 报告保存目录
    
    # 代码执行配置
    python_executor: str = "local"               # local/restricted_local/docker
    
    # 可视化配置
    visualization_mode: str | None = "rule"      # rule/llm/None
    
    # 上下文管理配置
    context_config: dict[str, Any] = {}          # 长对话压缩策略
    
    # 时序预测服务
    timeseries_forecasting_service_url: str      # 预测服务URL
    
    # 本地数据集管理器
    local_dataset_manager: Any = None            # 加载本地CSV/Excel文件
```

#### 1.2 ConfigLoader 类（单例）

**核心方法：**

1. **`load(config_file: str = None)`**
   - 从YAML文件加载配置
   - 默认路径：`openchatbi/config.yaml` 或环境变量 `CONFIG_FILE`
   - 支持代理配置（HTTP/HTTPS proxy）
   - 自动创建catalog store
   - 加载本地数据集（如果配置）

2. **`get() -> Config`**
   - 获取当前配置实例
   - 如果未加载会抛出 `ValueError`

3. **`set(config: dict[str, Any])`**
   - 通过字典直接设置配置

4. **`_process_config_dict(config_data: dict)`**（内部方法）
   - 验证必需配置项
   - 动态加载LLM类（通过反射机制）
   - 加载BI配置文件
   - 创建catalog store实例
   - 初始化本地数据集管理器

5. **`load_bi_config(bi_config_file: str)`**
   - 从单独的YAML文件加载业务知识配置
   - 包含：术语表、维度定义、指标规则等

### 配置加载流程

```
YAML文件 
  ↓
load() 方法
  ↓
_process_config_dict() 处理
  ├─ 验证必需项（default_llm, data_warehouse_config）
  ├─ 动态加载LLM类（importlib）
  ├─ 加载BI配置
  ├─ 创建catalog store
  └─ 初始化本地数据集管理器
  ↓
创建 Config 实例
  ↓
保存到 _config（单例变量）
```

### 典型使用场景

```python
# 1. 加载配置
config_loader = ConfigLoader()
config_loader.load("path/to/config.yaml")

# 2. 获取配置
config = config_loader.get()
llm = config.default_llm
catalog = config.catalog_store

# 3. 使用默认配置路径
from openchatbi import config
cfg = config.get()  # 全局访问
```

### 关键特性

1. **单例模式**：确保全局唯一配置实例
2. **延迟加载**：只在需要时抛出错误
3. **灵活的LLM配置**：支持任何符合接口的LLM类
4. **自动化元数据管理**：自动创建和加载catalog store
5. **本地数据集支持**：可预加载CSV/Excel作为pandas DataFrame

## 2. graph_state.py - 状态定义

### 概述
定义了 LangGraph 状态图中使用的所有状态类，这些状态类控制着数据在图节点之间的流转。

### 主要状态类

#### 2.1 AgentState（主Agent状态）
继承自 `MessagesState`，扩展了路由和响应字段：

```python
class AgentState(MessagesState):
    # 继承：messages (对话消息列表)
    
    # 历史消息（用于记忆管理）
    history_messages: Annotated[list[HumanMessage | AIMessage], add_history_messages]
    
    # 路由控制
    agent_next_node: str        # 下一个要执行的节点名称
    sends: list[Send]           # 并行路由指令（Send对象列表）
    
    # SQL相关
    sql: str                    # 生成的SQL语句
    
    # 最终响应
    final_answer: str           # Agent的最终回答
```

**字段说明：**

- **`history_messages`**：使用自定义reducer `add_history_messages`，累积对话历史
- **`agent_next_node`**：条件路由使用，指定下一个节点（如 "llm_node", "use_tool", END）
- **`sends`**：LangGraph的并行执行机制，允许同时调用多个工具
- **`sql`**：保存Text2SQL工具生成的SQL，供后续分析使用
- **`final_answer`**：当Agent完成任务后设置，用于输出

#### 2.2 SQLGraphState（SQL子图状态）
专门用于Text2SQL子图的状态：

```python
class SQLGraphState(MessagesState):
    rewrite_question: str                    # 重写后的问题（消歧、补充上下文）
    tables: list[dict[str, Any]]            # 选择的相关表信息
    info_entities: dict[str, Any]           # 提取的实体信息（维度、指标等）
    sql: str                                # 生成的SQL语句
    sql_retry_count: int                    # SQL重试次数（执行失败时）
    sql_execution_result: str               # SQL执行结果
    schema_info: dict[str, Any]             # 数据模式分析结果
    data: str                               # CSV格式的查询结果
    previous_sql_errors: list[dict]         # 之前失败的SQL及错误信息
    visualization_dsl: dict[str, Any]       # 可视化DSL配置
```

**SQL生成流程对应的状态变化：**

```
1. 问题重写：rewrite_question
2. 表选择：tables
3. 实体提取：info_entities
4. SQL生成：sql
5. SQL执行：sql_execution_result, data
6. 错误处理：sql_retry_count, previous_sql_errors
7. 数据分析：schema_info
8. 可视化：visualization_dsl
```

#### 2.3 InputState / OutputState
定义图的输入输出接口：

```python
class InputState(MessagesState):
    pass  # 只接收messages

class OutputState(MessagesState):
    pass  # 只输出messages
```

这些是**接口类**，用于：
- 明确图的输入输出契约
- 隐藏内部状态细节
- 便于图的组合和测试

#### 2.4 SQLOutputState
SQL子图的输出状态（暴露更多信息）：

```python
class SQLOutputState(MessagesState):
    rewrite_question: str       # 暴露重写后的问题
    tables: list[dict]          # 暴露使用的表
    sql: str                    # 暴露SQL
    schema_info: dict           # 暴露数据分析结果
    data: str                   # 暴露查询数据
    visualization_dsl: dict     # 暴露可视化配置
```

### 自定义Reducer函数

#### add_history_messages
```python
def add_history_messages(left: list, right: list):
    if left:
        total_messages = left + right
    else:
        total_messages = right
    return total_messages
```

**作用**：累积历史消息，用于长期记忆管理
- `left`：现有的历史消息
- `right`：新增的消息
- 返回：合并后的完整历史

### 状态在图中的流转示例

```python
# 初始状态
{
    "messages": [HumanMessage("显示销售趋势")]
}

# LLM节点处理后
{
    "messages": [HumanMessage(...), AIMessage(tool_calls=[...])],
    "agent_next_node": "use_tool",
    "history_messages": [HumanMessage(...)]
}

# Tool节点执行后
{
    "messages": [..., ToolMessage(content="SQL执行结果...")],
    "sql": "SELECT ...",
    "agent_next_node": "llm_node"
}

# 最终状态
{
    "messages": [..., AIMessage("根据数据显示...")],
    "final_answer": "销售趋势显示...",
    "agent_next_node": END
}
```

### 设计原则

1. **分层设计**：主图状态 vs 子图状态
2. **职责分离**：InputState/OutputState vs 内部状态
3. **可追溯性**：保留完整的执行历史
4. **重试支持**：记录错误信息便于重试
5. **并行执行**：通过 `sends` 字段支持并行工具调用

## 3. agent_graph.py - Agent图构建

### 概述
这是 OpenChatBI 的核心调度引擎，负责构建和编排主 Agent 图，协调各种工具（Text2SQL、Python代码执行、知识搜索、人工交互等）完成复杂的数据分析任务。

### 架构设计

```
┌─────────────────────────────────────────────────┐
│              Agent Graph (主图)                  │
│  ┌─────────┐    ┌──────────┐    ┌──────────┐   │
│  │  START  │───▶│ llm_node │◀──▶│ use_tool │   │
│  └─────────┘    └────┬─────┘    └──────────┘   │
│                      │                           │
│                      ▼                           │
│                 ┌──────────┐                     │
│                 │ask_human │                     │
│                 └──────────┘                     │
│                      │                           │
│                      ▼                           │
│                   [ END ]                        │
└─────────────────────────────────────────────────┘
```

### 主要组件

#### 3.1 核心节点函数

##### ask_human(state: AgentState)
人工交互节点，用于请求用户提供额外信息：

```python
def ask_human(state: AgentState) -> dict:
    tool_call = state["messages"][-1].tool_calls[0]
    args = tool_call["args"]
    
    # interrupt() 会暂停图执行，等待用户输入
    user_feedback = interrupt({
        "text": args["question"],
        "buttons": args.get("options", None)
    })
    
    return {
        "messages": [ToolMessage(...)],
        "history_messages": [...],
        "user_input": user_feedback
    }
```

**关键点**：
- 使用 LangGraph 的 `interrupt()` 机制
- 支持选项按钮（多选场景）
- 保存用户反馈到历史记录

##### agent_llm_call(llm, tools, context_manager)
创建LLM调用函数，核心决策节点：

```python
def _call_model(state: AgentState):
    # 1. 恢复不完整的工具调用
    recovery_ops = recover_incomplete_tool_calls(state)
    if recovery_ops:
        return {"messages": recovery_ops, "agent_next_node": "llm_node"}
    
    # 2. 应用上下文管理（压缩长对话）
    if context_manager:
        context_manager.manage_context_messages(messages)
    
    # 3. 构建系统提示词
    system_prompt = get_agent_prompt_template()
    
    # 4. 调用LLM
    response = call_llm_chat_model_with_retry(
        llm_with_tools,
        [SystemMessage(system_prompt)] + messages,
        streaming_tokens=True,
        parallel_tool_call=True
    )
    
    # 5. 处理响应并路由
    if tool_calls:
        # 分组：AskHuman vs 普通工具
        ask_human_calls = [...]
        normal_tool_calls = [...]
        
        # 创建并行执行Send对象
        sends = [
            Send("ask_human", {...}),
            Send("use_tool", {...})
        ]
        return {"sends": sends, ...}
    else:
        # 没有工具调用，直接结束
        return {"final_answer": response.content, "agent_next_node": END}
```

**核心逻辑**：
1. **错误恢复**：检测并修复不完整的工具调用
2. **上下文压缩**：避免超出token限制
3. **智能路由**：根据工具调用类型选择执行路径
4. **并行执行**：AskHuman 和普通工具可以同时调用

#### 3.2 工具系统

##### get_sql_tools(sql_graph, sync_mode)
创建Text2SQL工具：

```python
# 同步版本
def call_sql_graph_sync(reasoning: str, context: str) -> str:
    sql_graph_response = sql_graph.invoke({"messages": context})
    return _format_sql_response(sql_graph_response)

# 异步版本
async def call_sql_graph_async(reasoning: str, context: str) -> str:
    sql_graph_response = await sql_graph.ainvoke({"messages": context})
    return _format_sql_response(sql_graph_response)
```

**返回格式化响应**：
```
SQL Query:
```sql
SELECT ...
```

Query Results (CSV format):
```csv
date,sales,profit
2024-01,100,20
...
```

Visualization Created: bar chart has been automatically generated.
```

##### 工具清单
```python
normal_tools = [
    search_knowledge,      # 搜索业务知识库
    show_schema,          # 显示数据库Schema
    call_sql_graph_tool,  # Text2SQL（调用子图）
    run_python_code,      # 执行Python代码
    save_report,          # 保存报告
    *memory_tools,        # 长期记忆（manage/search）
    timeseries_forecast,  # 时序预测（如果服务可用）
    *mcp_tools           # MCP协议工具
]
```

#### 3.3 图构建函数

##### _build_graph_core(...) 
核心构建逻辑（同步/异步共享）：

```python
def _build_graph_core(...) -> CompiledStateGraph:
    # 1. 创建SQL子图
    sql_graph = build_sql_graph(catalog, checkpointer, memory_store)
    
    # 2. 创建SQL工具
    call_sql_graph_tool = get_sql_tools(sql_graph, sync_mode)
    
    # 3. 初始化上下文管理器
    context_manager = ContextManager(llm=get_default_llm(), ...)
    
    # 4. 组装所有工具
    tool_node = ToolNode(normal_tools)
    
    # 5. 定义图
    graph = StateGraph(AgentState, input_schema=InputState, ...)
    
    # 6. 添加节点
    graph.add_node("llm_node", agent_llm_call(...))
    graph.add_node("ask_human", ask_human)
    graph.add_node("use_tool", tool_node)
    
    # 7. 添加边
    graph.add_edge(START, "llm_node")
    graph.add_edge("ask_human", "llm_node")
    graph.add_edge("use_tool", "llm_node")
    
    # 8. 添加条件路由
    def route_tools(state: AgentState):
        if state["sends"]:
            return state["sends"]  # 并行路由
        return state["agent_next_node"]  # 单一路由
    
    graph.add_conditional_edges("llm_node", route_tools, {...})
    
    # 9. 编译
    return graph.compile(
        name="agent_graph",
        checkpointer=checkpointer,
        store=memory_store
    )
```

##### build_agent_graph_sync / build_agent_graph_async
公开API，区别在于MCP工具初始化：

```python
# 同步版本
def build_agent_graph_sync(...):
    mcp_tools = create_mcp_tools_sync(get_mcp_servers())
    return _build_graph_core(..., sync_mode=True, mcp_tools=mcp_tools)

# 异步版本
async def build_agent_graph_async(...):
    mcp_tools = await get_mcp_tools_async(get_mcp_servers())
    return _build_graph_core(..., sync_mode=False, mcp_tools=mcp_tools)
```

### 执行流程示例

```python
# 用户提问
user_question = "显示最近3个月的销售趋势"

# 1. START → llm_node
# LLM决策：需要调用text2sql工具

# 2. llm_node → use_tool
# 执行：call_sql_graph_tool(context="显示最近3个月的销售趋势")
#   ├─ SQL子图执行
#   │  ├─ 问题重写
#   │  ├─ 表选择
#   │  ├─ SQL生成
#   │  ├─ SQL执行
#   │  └─ 可视化DSL生成
#   └─ 返回：SQL + 数据 + 可视化信息

# 3. use_tool → llm_node
# LLM接收工具结果，生成最终回答

# 4. llm_node → END
# 输出：分析结果 + 图表
```

### 高级特性

#### 1. 并行工具调用
```python
# LLM可以同时调用多个工具
sends = [
    Send("use_tool", {"messages": [search_knowledge_call]}),
    Send("use_tool", {"messages": [show_schema_call]})
]
# 两个工具并行执行，提高效率
```

#### 2. 上下文管理
```python
context_manager = ContextManager(
    llm=get_default_llm(),
    config={
        "max_tokens": 100000,
        "compression_strategy": "sliding_window"
    }
)
# 自动压缩超长对话，保留关键信息
```

#### 3. 错误恢复
```python
recovery_ops = recover_incomplete_tool_calls(state)
# 检测并修复：
# - 缺少参数的工具调用
# - JSON格式错误
# - 重复的工具调用
```

#### 4. 流式输出
```python
response = call_llm_chat_model_with_retry(
    llm_with_tools,
    messages,
    streaming_tokens=True  # 启用token级流式输出
)
# 用户体验：逐字显示而非等待完整响应
```

### 依赖关系

```
agent_graph.py
  ├─ config_loader.py (获取LLM、配置)
  ├─ graph_state.py (定义状态)
  ├─ text2sql.sql_graph (SQL子图)
  ├─ tool/*.py (各种工具)
  ├─ context_manager.py (上下文压缩)
  └─ prompts/system_prompt.py (Agent提示词)
```

### 使用示例

```python
from openchatbi.agent_graph import build_agent_graph_sync
from openchatbi.config_loader import ConfigLoader

# 1. 加载配置
config = ConfigLoader()
config.load("config.yaml")

# 2. 构建图
graph = build_agent_graph_sync(
    catalog=config.get().catalog_store,
    checkpointer=SqliteSaver.from_conn_string(":memory:"),
    enable_context_management=True
)

# 3. 执行
result = graph.invoke(
    {"messages": [HumanMessage("分析销售数据")]},
    config={"configurable": {"thread_id": "user123"}}
)

print(result["final_answer"])
```

### 设计亮点

1. **模块化**：工具、节点、图分离，易于扩展
2. **灵活路由**：支持单路由、并行路由、中断等待
3. **同步/异步统一**：共享核心逻辑，只在MCP工具层区分
4. **可观测性**：完整的状态追踪和日志记录
5. **容错性**：多重错误恢复机制
6. **性能优化**：并行执行、流式输出、上下文压缩

## 4. utils.py - 工具函数集

### 概述
`utils.py` 提供了一系列通用工具函数，支撑 OpenChatBI 的各个模块，包括日志记录、内容提取、向量检索、错误恢复等核心功能。

### 主要功能模块

#### 4.1 日志和调试

##### log(args)
```python
def log(args) -> None:
    """输出到stderr，便于调试"""
    print(args, file=sys.stderr, flush=True)
```

**特点**：
- 输出到 stderr 而非 stdout
- 使用 `flush=True` 确保立即输出
- 不干扰正常的程序输出

#### 4.2 内容提取工具

##### get_text_from_content(content)
处理多种格式的内容提取：

```python
# 支持的输入格式：
# 1. 字符串：直接返回
"Hello World"  → "Hello World"

# 2. 字符串列表：拼接
["Hello", " ", "World"]  → "Hello World"

# 3. 字典列表（LLM返回格式）：
[{"type": "text", "text": "Hello"}, {"text": " World"}]  → "Hello World"
```

##### get_text_from_message_chunk(chunk: AIMessageChunk)
从 LLM 流式输出的消息块中提取文本：

```python
# 流式处理场景
for chunk in llm.stream(messages):
    text = get_text_from_message_chunk(chunk)
    print(text, end="", flush=True)  # 逐字显示
```

##### extract_json_from_answer(answer: str)
从文本中提取 JSON 对象（支持嵌套）：

```python
answer = """
这是一些说明文字
{
    "name": "测试",
    "nested": {"key": "value"}
}
还有其他内容
"""

result = extract_json_from_answer(answer)
# result = {"name": "测试", "nested": {"key": "value"}}
```

**技术亮点**：使用 `regex` 库的递归匹配 `(?R)` 处理嵌套 JSON

#### 4.3 文件下载支持

##### get_report_download_response(filename: str)
安全的报告文件下载：

```python
# 使用示例（在 FastAPI 中）
@app.get("/download/{filename}")
def download_report(filename: str):
    return get_report_download_response(filename)
```

**安全措施**：
1. **路径验证**：检查文件是否在配置的报告目录内
2. **存在性检查**：确认文件存在且是文件而非目录
3. **路径遍历防护**：防止 `../` 攻击
4. **MIME类型映射**：根据扩展名设置正确的 Content-Type

```python
media_type_map = {
    ".md": "text/markdown",
    ".csv": "text/csv",
    ".txt": "text/plain",
    ".json": "application/json",
    ".html": "text/html",
    ".xml": "application/xml"
}
```

#### 4.4 向量数据库管理

##### create_vector_db(texts, embedding, ...)
创建或复用 Chroma 向量数据库：

```python
# 有 embedding 模型：使用 Chroma
vector_db = create_vector_db(
    texts=["文本1", "文本2"],
    embedding=embedding_model,
    collection_name="knowledge_base"
)

# 无 embedding 模型：退化为 BM25
vector_db = create_vector_db(
    texts=["文本1", "文本2"],
    embedding=None  # 自动使用 SimpleStore
)
```

**智能缓存机制**：
1. 检查集合是否已存在
2. 如果存在且有内容 → 复用
3. 如果不存在或为空 → 创建新集合

#### 4.5 错误恢复机制

##### recover_incomplete_tool_calls(state: AgentState)
核心功能：从中断的工具调用中恢复

**问题场景**：
```
用户提问 → LLM 返回 AIMessage(tool_calls=[...])
          ↓
      开始执行工具
          ↓
      [ 系统崩溃/重启 ] ← 问题！
          ↓
      状态：有 tool_calls 但没有对应的 ToolMessage
```

**恢复逻辑**：

```python
# 1. 找到最后一个包含 tool_calls 的 AIMessage
last_ai_message = messages[-1]  # AIMessage with tool_calls

# 2. 检查哪些 tool_call_id 没有对应的 ToolMessage
unhandled_ids = tool_call_ids - handled_tool_call_ids

# 3. 为未处理的工具调用创建失败消息
recovery_messages = [
    ToolMessage(
        content="Tool execution was interrupted...",
        tool_call_id=call_id
    )
    for call_id in unhandled_ids
]

# 4. 构建插入操作（保持消息顺序）
operations = [
    RemoveMessage(id=msg.id),  # 移除 AIMessage 之后的消息
    *recovery_messages,         # 插入恢复消息
    *regenerate_messages        # 重新添加原消息（新 ID）
]
```

**关键技巧**：
- 使用 `RemoveMessage` + 重新插入 保持顺序
- 重新生成消息 ID 防止操作被取消
- 只恢复真正缺失的工具调用

### 4.6 SimpleStore 类（BM25 向量存储）

当没有 embedding 模型时的降级方案，使用 BM25 算法进行文本检索。

#### 初始化
```python
store = SimpleStore(
    texts=["文档1", "文档2", "文档3"],
    metadatas=[{"source": "doc1"}, ...],
    ids=["id1", "id2", "id3"]  # 可选
)
```

**内部构建**：
1. 创建 `Document` 对象列表
2. 对每个文档分词（使用 `TextSegmenter`）
3. 构建 BM25 索引

#### 核心方法

##### similarity_search(query, k=4)
基于 BM25 的相似度搜索：

```python
docs = store.similarity_search("销售数据分析", k=3)
# 返回最相关的 3 个文档
```

**算法流程**：
```
Query: "销售数据分析"
  ↓
分词: ["销售", "数据", "分析"]
  ↓
BM25 计算每个文档的得分
  ↓
排序取 Top-K
  ↓
返回 Document 对象
```

##### similarity_search_with_score(query, k=4)
带分数的搜索：

```python
results = store.similarity_search_with_score("销售", k=2)
# [(Document(...), 8.5), (Document(...), 6.2)]
```

##### max_marginal_relevance_search(query, k=4, fetch_k=20, lambda_mult=0.5)
MMR（最大边际相关性）搜索，平衡相关性和多样性：

```python
docs = store.max_marginal_relevance_search(
    query="数据分析",
    k=5,           # 返回 5 个文档
    fetch_k=20,    # 先取 20 个候选
    lambda_mult=0.7  # 0.7 相关性 + 0.3 多样性
)
```

**MMR 公式**：
```
MMR = λ × Relevance(q, d) - (1-λ) × max(Similarity(d, s))
                                      s ∈ selected
```

- `λ=1`：纯相关性（忽略多样性）
- `λ=0`：纯多样性（忽略相关性）
- `λ=0.5`：平衡

**算法步骤**：
1. 用 BM25 检索 `fetch_k` 个候选文档
2. 归一化 BM25 分数到 [0, 1]
3. 迭代选择：
   - 对每个候选，计算 MMR 分数
   - 考虑与已选文档的相似度（Jaccard）
   - 选择 MMR 分数最高的
4. 重复直到选够 `k` 个

##### add_texts(texts, metadatas, ids)
动态添加文档：

```python
new_ids = store.add_texts(
    texts=["新文档1", "新文档2"],
    metadatas=[{"type": "新增"}, {"type": "新增"}]
)
# 返回：["uuid1", "uuid2"]
# 自动更新 BM25 索引
```

##### delete(ids)
删除文档：

```python
store.delete(ids=["id1", "id2"])
# 删除指定文档并重建 BM25 索引
```

##### get_by_ids(ids)
按 ID 获取文档：

```python
docs = store.get_by_ids(["id1", "id3"])
# 返回对应的 Document 对象
```

#### 私有方法

##### _tokenize(text: str)
使用 `TextSegmenter` 进行中文分词：

```python
tokens = store._tokenize("销售数据分析报告")
# ["销售", "数据", "分析", "报告"]
```

##### _calculate_similarity(doc1, doc2)
计算两个文档的 Jaccard 相似度：

```python
similarity = store._calculate_similarity(doc1, doc2)
# Jaccard = |A ∩ B| / |A ∪ B|
```

**用途**：MMR 算法中计算文档间的多样性

### 典型使用场景

#### 场景1：知识库检索
```python
# 初始化
texts = load_knowledge_documents()
store = create_vector_db(texts, embedding_model)

# 搜索
query = "如何计算 GMV？"
docs = store.similarity_search(query, k=3)

# 构建上下文
context = "\n\n".join([doc.page_content for doc in docs])
```

#### 场景2：错误恢复
```python
# 在 agent_llm_call 中
def _call_model(state: AgentState):
    # 首先尝试恢复
    recovery_ops = recover_incomplete_tool_calls(state)
    if recovery_ops:
        return {"messages": recovery_ops, "agent_next_node": "llm_node"}
    
    # 正常处理...
```

#### 场景3：流式输出
```python
async def stream_response(query):
    for chunk in llm.astream(messages):
        text = get_text_from_message_chunk(chunk)
        if text:
            yield f"data: {text}\n\n"
```

#### 场景4：报告下载
```python
# FastAPI 路由
@app.get("/api/download/{filename}")
async def download(filename: str):
    return get_report_download_response(filename)
```

### 设计特点

1. **容错性强**：所有函数都有完善的错误处理
2. **类型兼容**：处理多种输入格式（字符串、列表、字典）
3. **性能优化**：
   - 向量数据库智能缓存
   - BM25 索引增量更新
   - MMR 使用高效的 Jaccard 相似度
4. **安全第一**：文件下载有多重安全检查
5. **降级机制**：无 embedding 时自动使用 BM25
6. **可扩展性**：`SimpleStore` 实现 VectorStore 接口，可替换为其他实现

### 依赖关系

```
utils.py
  ├─ text_segmenter.py (中文分词)
  ├─ graph_state.py (状态定义)
  ├─ langchain_chroma (向量数据库)
  ├─ rank_bm25 (BM25 算法)
  └─ config.py (获取配置)
```

### Below is my test

In [7]:
from openchatbi import config

cfg = config.get()

In [10]:
cfg.context_config

{}